<a href="https://colab.research.google.com/github/Imran012x/Transfer-Models/blob/main/Hilsha_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training..

In [ ]:
#1. IMPORTS AND INITIAL SETUP
# ================================================================================================================================
# Purpose: Import all required libraries and set up warnings to suppress unnecessary messages.

# # 1️⃣ Uninstall all broken/conflicting packages
# !pip uninstall -y numpy pandas matplotlib seaborn opencv-python-headless opencv-contrib-python -q

# # 2️⃣ Remove corrupted pandas folder
# !rm -rf /usr/local/lib/python3.12/dist-packages/~andas* -q

# # 3️⃣ Install compatible versions
# !pip install numpy==1.26.4 pandas==2.2.2 matplotlib seaborn opencv-python-headless opencv-contrib-python -q

# # 4️⃣ Restart runtime after this!


# # 2. Install latest compatible versions for Python 3.12
# !pip install --upgrade pip -q


import warnings
#For DeprecationWarning / FutureWarning specifically:
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=FutureWarning)
# Hide all pip warnings
warnings.filterwarnings("ignore", category=UserWarning)



!pip install pytorch-gradcam optuna captum -q  # Uncomment if running in a new environment
!pip install torchmetrics==0.11.4 -q # or whatever version you need
# # !!pip install --upgrade --force-reinstall numpy seaborn matplotlib -q


import sys
import numpy
import pandas
import seaborn as sns

print(f"python_version: {sys.version.split()[0]}")
print(f"numpy_version: {numpy.__version__}")
print(f"pandas_version: {pandas.__version__}")
print(f"seaborn_version: {sns.__version__}\n")



# !pip list | grep -E "numpy|seaborn|matplotlib|pandas"
# !pip uninstall -y numpy seaborn matplotlib
# !pip install numpy seaborn matplotlib



# !pip install captum  -q  #> /dev/null 2>&1
# !pip install --upgrade captum optuna
# Install missing libraries in Colab
# !pip install optuna torchmetrics captum -q
# !pip install --upgrade --force-reinstall numpy pandas
# !pip install --upgrade --force-reinstall numpy pandas scipy scikit-learn




import os
from google.colab import drive

drive_path = '/content/drive'

if os.path.exists(drive_path) and os.path.ismount(drive_path):
    print("Google Drive is already connected ✅")
else:
    drive.mount(drive_path)
    print("Google Drive connection done ✅")



# # Upload a file
# uploaded = files.upload()
# # Get the file name
# file_name = list(uploaded.keys())[0]
# print(f"Uploaded file: {file_name}")


# import zipfile
# import os
# # with zipfile.ZipFile('/content/drive/MyDrive/Hilsha/data_fish_224_11k.zip', 'r') as zip_ref:
# #     zip_ref.extractall('')
# with zipfile.ZipFile('/content/drive/MyDrive/Hilsha/data_fish_org_8407.zip', 'r') as zip_ref:
#     zip_ref.extractall('')



# ============================================================
# Standard Library
# ============================================================
import os
import sys
import gc
import time
import json
import logging
import random
import warnings
import subprocess
import threading
import traceback
from pathlib import Path
from itertools import combinations
from datetime import datetime, timedelta
import zipfile
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

# ============================================================
# Data Handling & Utilities
# ============================================================
import numpy as np
import pandas as pd
from tqdm import tqdm

# ============================================================
# Visualization
# ============================================================
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2

# ============================================================
# System & Resource Monitoring
# ============================================================
import psutil
import pynvml

# ============================================================
# Machine Learning
# ============================================================
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, f1_score, accuracy_score,
    precision_score, recall_score, roc_curve, auc
)
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import label_binarize

# Imbalanced data handling
from imblearn.over_sampling import SMOTE

# ============================================================
# Deep Learning - PyTorch
# ============================================================
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, SubsetRandomSampler
import torchvision.models as models
import torchvision.transforms as transforms

# ============================================================
# Augmentation
# ============================================================
import albumentations as A
from albumentations.pytorch import ToTensorV2


# ============================================================
# Explainable AI (XAI)
# ============================================================

import torch.autograd as autograd
from captum.attr import LRP

# Optuna
import optuna
import optuna.logging


# Compute metrics on GPU
# import torchmetrics.functional as tmf

# ============================================================
# Hyperparameter Optimization
# ============================================================
try:
    OPTUNA_AVAILABLE = True
except ImportError:
    OPTUNA_AVAILABLE = False
    print("Warning: Optuna not available. Using default hyperparameters.")








# ---
# 2. CONFIGURATION
# ================================================================================================================================
# Purpose: Define configuration settings and initialize the environment.

class Config:
    """Configuration settings"""
    # Paths - UPDATE THESE TO YOUR ACTUAL PATHS
    DATA_FILE = '/content/drive/MyDrive/Hilsha/X_data.npy'
    LABELS_FILE = '/content/drive/MyDrive/Hilsha/Y_labels.npy'
    OUTPUT_DIR = './fish_classification_results'

    # Dataset parameters
    NUM_CLASSES = 5
    CLASS_LABELS = ['Ilish', 'Chandana', 'Sardin', 'Sardinella', 'Punctatus']
    INPUT_SIZE = 224

    # Training parameters
    BATCH_SIZE = 32 #Will Change Dynamically
    DATALOADER_NUM_WORKERS = 1 #Will Change Dynamically
    # Dynamically adjust batch size and workers
    EPOCHS = 15
    PIN_MEMORY = True
    USE_MIXED_PRECISION = True #True
    COMPILE_MODEL = True
    PATIENCE = 4
    LEARNING_RATE = 1e-5
    WEIGHT_DECAY = 1e-4

    # Hyperparameter tuning
    OPTUNA_TRIALS = 12
    OPTUNA_EPOCHS = 7

    # Models to train
    MODELS = ['resnet50','efficientnet_b0','mobilenet_v3_large','vgg16', 'densenet121']

    # Ensemble methods
    ENSEMBLE_METHODS = ['simple_average', 'weighted_average', 'confidence_based', 'learnable_weighted']

    # Device
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    SEED = 42



# def get_available_cpu_memory():
#     """Get available CPU memory in GB."""
#     mem = psutil.virtual_memory()
#     return mem.available / 1024**3  # Convert bytes to GB

# def get_available_gpu_memory():
#     """Get available GPU memory in GB."""
#     pynvml.nvmlInit()
#     handle = pynvml.nvmlDeviceGetHandleByIndex(0)
#     mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
#     return mem_info.free / 1024**3  # Convert bytes to GB

# def adjust_batch_size_and_workers(base_batch_size, base_num_workers):
#     gpu_memory = get_available_gpu_memory()
#     print(f"GPU memory available: {gpu_memory:.2f} GB")

#     # Prioritize GPU memory for batch size
#     if gpu_memory > 24:  # High-end GPU
#         batch_size = base_batch_size * 4
#         num_workers = min(8, psutil.cpu_count())  # Use more workers for high GPU memory
#     elif gpu_memory > 12:  # Mid-range GPU
#         batch_size = base_batch_size * 2
#         num_workers = min(4, psutil.cpu_count())  # Moderate workers
#     elif gpu_memory > 6:  # Lower-end GPU
#         batch_size = base_batch_size
#         num_workers = min(2, psutil.cpu_count())  # Minimal workers
#     else:  # Very low GPU memory
#         batch_size = max(8, base_batch_size // 2)
#         num_workers = 0  # Disable workers to minimize CPU load

    # if batch_size is None:
    # if torch.cuda.is_available():
    #     gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    #     if gpu_memory_gb >= 24:
    #         batch_size = 128
    #     elif gpu_memory_gb >= 12:
    #         batch_size = 96
    #     elif gpu_memory_gb >= 8:
    #         batch_size = 64
    #     else:
    #         batch_size = 48
    # else:
    #     batch_size = Config.BATCH_SIZE


    # print(f"Adjusted batch_size: {batch_size}, num_workers: {num_workers} And GPU memory: {gpu_memory:.2f} GB)")
    # return batch_size, num_workers


def setup_environment():
    """Setup random seeds, directories, and dynamically adjust batch size and workers"""

    os.environ['PYTHONHASHSEED'] = str(Config.SEED)  # For hash seed reproducibility
    random.seed(Config.SEED)
    np.random.seed(Config.SEED)
    torch.manual_seed(Config.SEED)
    torch.cuda.manual_seed_all(Config.SEED)  # For multi-GPU if applicable
    #Guard for GPU determinism (optional, but helpful if you want exact reproducibility across runs):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    warnings.filterwarnings("ignore")


    torch.manual_seed(Config.SEED)
    np.random.seed(Config.SEED)

    directories = [
        Config.OUTPUT_DIR,
        f"{Config.OUTPUT_DIR}/models",
        f"{Config.OUTPUT_DIR}/visualizations",
        f"{Config.OUTPUT_DIR}/reports",
        f"{Config.OUTPUT_DIR}/xai_visualizations"
    ]

    for directory in directories:
        Path(directory).mkdir(parents=True, exist_ok=True)#With exist_ok=True:Python will not raise an error if already exists.Or else raise a FileExistsError
        #& parents=True → creates all missing parent directories in the path.

    # Ensure all output directories exist
    os.makedirs(f"{Config.OUTPUT_DIR}/best_model", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/model_results", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/kfold_results", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/visualizations", exist_ok=True)

    print(f"Using device: {Config.DEVICE}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"Dynamic BATCH_SIZE: {Config.BATCH_SIZE}, DATALOADER_NUM_WORKERS: {Config.DATALOADER_NUM_WORKERS}")
    print("-" * 70)


def worker_init_fn(worker_id):
    seed = Config.SEED + worker_id
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)


# class NpEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, torch.Tensor):
#             return obj.cpu().detach().numpy().tolist()  # Convert tensor to NumPy array, then to list
#         if isinstance(obj, np.integer):
#             return int(obj)
#         if isinstance(obj, np.floating):
#             return float(obj)
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return super(NpEncoder, self).default(obj)

# Config.BATCH_SIZE, Config.DATALOADER_NUM_WORKERS = adjust_batch_size_and_workers(Config.BATCH_SIZE, Config.DATALOADER_NUM_WORKERS)




# ---
# 3. TRAINING PROGRESS TRACKER
# ================================================================================================================================
# Purpose: Track and display training progress with progress bars and epoch summaries.

class TrainingProgressTracker:
    """Track and display detailed training progress"""

    def __init__(self, model_name, total_epochs, total_batches_per_epoch):
        self.model_name = model_name
        self.total_epochs = total_epochs
        self.total_batches_per_epoch = total_batches_per_epoch

        self.current_epoch = 0
        self.start_time = time.time()
        self.epoch_start_time = None
        self.batch_times = []



    def start_epoch(self, epoch):
        """Start tracking an epoch"""
        self.current_epoch = epoch
        self.epoch_start_time = time.time()
        self.batch_times = []
        print(f"\n{'='*60}")
        print(f"Model: {self.model_name.upper()} | Epoch: {epoch+1}/{self.total_epochs}")
        print(f"{'='*60}")



    def update_batch(self, batch_idx, batch_loss, batch_acc, is_training=True,total_batches=None):
        """Update progress for current batch"""
        batch_time = time.time()
        self.batch_times.append(batch_time)


        # Calculate timing estimates
        if len(self.batch_times) > 1:
            avg_batch_time = np.mean(np.diff(self.batch_times[-10:]))
        else:
            avg_batch_time = 1.0

        total_batches = total_batches if total_batches is not None else self.total_batches_per_epoch
        remaining_batches = total_batches - (batch_idx + 1)


        # remaining_batches = self.total_batches_per_epoch - (batch_idx + 1)
        eta_epoch = remaining_batches * avg_batch_time

        # Progress bar
        # progress_pct = (batch_idx + 1) / self.total_batches_per_epoch * 100
        progress_pct = (batch_idx + 1) / total_batches * 100
        bar_length = 30

        # filled_length = int(bar_length * (batch_idx + 1) // self.total_batches_per_epoch)
        filled_length = int(bar_length * (batch_idx + 1) // total_batches)
        bar = '█' * filled_length + '-' * (bar_length - filled_length)

        # Format time
        eta_str = str(timedelta(seconds=int(eta_epoch)))

        # Display progress
        mode = "TRAIN" if is_training else "VAL  "

        print(f"\r{mode} |{bar}| {progress_pct:5.1f}% | "
              f"Batch: {batch_idx+1:4d}/{total_batches} | "
              f"Loss: {batch_loss:.4f} | Acc: {batch_acc:.4f} | "
              f"ETA: {eta_str}", end='', flush=True)


    def finish_epoch(self, train_loss, train_acc, val_loss, val_acc, val_f1, is_best=False, lr=None):
        """Finish epoch and display summary"""
        epoch_time = time.time() - self.epoch_start_time
        total_time = time.time() - self.start_time

        print(f"\n{'-'*60}")
        print(f"EPOCH SUMMARY:")
        print(f"  Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.6f} | Val Acc:   {val_acc:.4f}")
        print(f"  Val F1:     {val_f1:.4f} | Epoch Time: {epoch_time:.1f}s")
        if lr:
            print(f"  Learning Rate: {lr:.2e}")

        if is_best:
            print(f"  ★ NEW BEST MODEL! (F1: {val_f1:.4f})")

        print(f"  Total Time: {str(timedelta(seconds=int(total_time)))}")
        print(f"{'-'*60}")


# ---
# 4. MODEL FACTORY
# ================================================================================================================================
# Purpose: Create different neural network models with customizable architectures.

class ModelFactory:
    @staticmethod
    def create_model(model_name, num_classes=Config.NUM_CLASSES, dropout_rate=0.5,hidden_dim_multiplier=0.5):
        """Create model with configurable architecture"""

        if model_name == 'resnet50':
            model = models.resnet50(weights='IMAGENET1K_V2')
            # Partial unfreeze for better accuracy: unfreeze layer4 and fc
            for name, param in model.named_parameters():
                param.requires_grad = False
                # if "layer4" in name or "fc" in name:
                if "layer3" in name or "layer4" in name or "fc" in name:
                    param.requires_grad = True

            num_features = model.fc.in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.fc = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'efficientnet_b0':
            model = models.efficientnet_b0(weights='IMAGENET1K_V1')
            # Partial unfreeze: last blocks
            for name, param in model.named_parameters():
                param.requires_grad = False
                if "_blocks.15" in name or "_blocks.16" in name or "classifier" in name:
                    param.requires_grad = True
            num_features = model.classifier[1].in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'mobilenet_v3_large':
            model = models.mobilenet_v3_large(weights='IMAGENET1K_V2')
            # Partial unfreeze: last features
            for name, param in model.named_parameters():
                param.requires_grad = False
                if "features.12" in name or "features.13" in name or "classifier" in name:
                    param.requires_grad = True
            num_features = 960
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'vgg16':
            model = models.vgg16(weights='IMAGENET1K_V1')
            # Partial unfreeze: classifier and last features
            for name, param in model.named_parameters():
                param.requires_grad = False
                if "classifier" in name or "features.28" in name:
                    param.requires_grad = True
            hidden_dim = int(4096 * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Linear(512 * 7 * 7, 4096),
                nn.ReLU(True),
                nn.Dropout(),
                nn.Linear(4096, hidden_dim),
                nn.ReLU(True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'densenet121':
            model = models.densenet121(weights='IMAGENET1K_V1')
            # Partial unfreeze: denseblock4 and classifier
            for name, param in model.named_parameters():
                param.requires_grad = False
                if "denseblock4" in name or "classifier" in name:
                    param.requires_grad = True
            num_features = model.classifier.in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )


        elif model_name == 'cnn':
            # class SimpleCNN(nn.Module):

            class SimpleCNN(nn.Module):
                def __init__(self, num_classes=5, dropout_rate=0.3, hidden_dim_multiplier=0.3):
                    super(SimpleCNN, self).__init__()

                    # More conservative feature extractor to prevent overfitting
                    self.features = nn.Sequential(
                        # Block 1 - Start small
                        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(32),
                        nn.ReLU(inplace=True),
                        nn.Dropout2d(0.1),  # Spatial dropout in conv layers
                        nn.MaxPool2d(2, 2),  # 224 -> 112

                        # Block 2
                        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(64),
                        nn.ReLU(inplace=True),
                        nn.Dropout2d(0.15),
                        nn.MaxPool2d(2, 2),  # 112 -> 56

                        # Block 3
                        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(128),
                        nn.ReLU(inplace=True),
                        nn.Dropout2d(0.2),
                        nn.MaxPool2d(2, 2),  # 56 -> 28

                        # Block 4 - Add one more conv before pooling
                        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(128),
                        nn.ReLU(inplace=True),
                        nn.Dropout2d(0.25),
                        nn.MaxPool2d(2, 2),  # 28 -> 14

                        # Block 5 - Final feature extraction
                        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(256),
                        nn.ReLU(inplace=True),
                        nn.Dropout2d(0.3),
                        nn.AdaptiveAvgPool2d((7, 7))  # Fixed spatial size
                    )

                    # Calculate features after adaptive pooling
                    conv_output_size = 256 * 7 * 7  # 12544

                    # Much smaller hidden dimension to prevent overfitting
                    hidden_dim = int(conv_output_size * hidden_dim_multiplier)
                    hidden_dim = max(64, min(hidden_dim, 512))  # Smaller range

                    # Simple but effective classifier
                    self.classifier = nn.Sequential(
                        nn.Dropout(dropout_rate),
                        nn.Linear(conv_output_size, hidden_dim),
                        nn.ReLU(inplace=True),
                        nn.BatchNorm1d(hidden_dim),
                        nn.Dropout(dropout_rate * 0.5),
                        nn.Linear(hidden_dim, num_classes)
                    )

                    # Initialize weights properly
                    self._initialize_weights()

                def _initialize_weights(self):
                    for m in self.modules():
                        if isinstance(m, nn.Conv2d):
                            # Use smaller initialization for better gradient flow
                            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                            if m.bias is not None:
                                nn.init.zeros_(m.bias)
                        elif isinstance(m, nn.Linear):
                            # Smaller initialization for linear layers
                            nn.init.xavier_uniform_(m.weight, gain=0.5)
                            if m.bias is not None:
                                nn.init.zeros_(m.bias)
                        elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
                            if m.weight is not None:
                                nn.init.ones_(m.weight)
                            if m.bias is not None:
                                nn.init.zeros_(m.bias)

                def forward(self, x):
                    # Feature extraction
                    x = self.features(x)

                    # Flatten
                    x = torch.flatten(x, 1)

                    # Classification with gradient clipping
                    x = self.classifier(x)

                    # Clip outputs to prevent extreme values
                    x = torch.clamp(x, min=-10, max=10)

                    return x


            # Example usage
            model = SimpleCNN(num_classes=num_classes, dropout_rate=dropout_rate, hidden_dim_multiplier=hidden_dim_multiplier)
            model = model.to(Config.DEVICE)  # Move to device right after creation


        else:
            raise ValueError(f"Unsupported model: {model_name}")

        return model



# ---
# 5. LEARNABLE WEIGHTED ENSEMBLE MODEL
# ===============================================================================================================================
# Purpose: Define a neural network for learning optimal ensemble weights.

class LearnableWeightedEnsemble(nn.Module):
    """Ensemble model with per-class adaptive weights and attention"""
    def __init__(self, num_models, num_classes, hidden_dim=128, num_heads=4):
        super(LearnableWeightedEnsemble, self).__init__()
        self.num_models = num_models
        self.num_classes = num_classes

        # Attention mechanism to learn relations between model predictions
        self.attention = nn.MultiheadAttention(embed_dim=num_classes, num_heads=num_heads, batch_first=True)

        # Weight network outputs per-class weights for each model
        self.weight_network = nn.Sequential(
            nn.Linear(num_classes, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes),
            nn.Sigmoid()  # Per-class weight scaling
        )

        # Prediction head: combines weighted predictions + raw predictions
        self.prediction_head = nn.Sequential(
            nn.Linear(num_classes * (num_models + 1), hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes * 2),
            nn.ReLU(),
            nn.Linear(num_classes * 2, num_classes)
        )

    def forward(self, model_predictions):
        """
        model_predictions: (batch, num_models, num_classes)
        Returns:
            final_predictions: logits for classification
            weights: learned per-class weights for each model
        """
        batch_size = model_predictions.size(0)

        # --- Step 1: Attention over model predictions --- #The model looks at how predictions of different models relate to each other.
        attn_output, _ = self.attention(model_predictions, model_predictions, model_predictions)
        # shape: (batch, num_models, num_classes)


        # --- Step 2: Per-class weights for each model ---
        #Learns a weight for each model for each class.
        #softmax ensures weights across models sum to 1 for each class.
        #Basically: “For class 0, I trust model 2 more; for class 1, I trust model 0 more.”
        weights = self.weight_network(attn_output)  # (batch, num_models, num_classes)
        weights = F.softmax(weights, dim=1)  # normalize over models


        # --- Step 3: Weighted average across models ---
        #Combines the models’ predictions using the learned weights → smarter than a plain average.
        weighted_avg = torch.sum(model_predictions * weights, dim=1)  # (batch, num_classes)


        # --- Step 4: Residual connection with raw predictions ---
        #Combines the weighted average and all raw predictions.Gives the network more info to refine the final prediction.
        flat_preds = model_predictions.view(batch_size, -1)  # (batch, num_models * num_classes)
        final_input = torch.cat([weighted_avg, flat_preds], dim=1)  # (batch, num_classes + num_models*num_classes)


        # --- Step 5: Final refined prediction ---
        #A small feed-forward network refines the predictions.Output: (batch_size, num_classes) → logits for each class.
        final_predictions = self.prediction_head(final_input)  # (batch, num_classes)

        return final_predictions, weights
        #It learns which model is best for each class, combines their predictions smartly using attention, and produces a refined final prediction.

    # def entropy_regularization(self, weights):
    #     """Encourage diverse weight usage (optional loss term)."""
    #     # weights: (batch, num_models, num_classes)
    #     entropy = -torch.sum(weights * torch.log(weights + 1e-8), dim=1)  # (batch, num_classes)
    #     return torch.mean(entropy)



# ---
# 6. ENSEMBLE METHODS
# ================================================================================================================================
# Purpose: Implement ensemble methods (simple, weighted, confidence-based, learnable).

class EnsembleManager:
    def __init__(self, models_dict, val_data):
        self.models = models_dict
        self.X_val, self.y_val = val_data
        self.model_predictions = self._get_predictions()
        self.histories = {}

    def _get_predictions(self):
        print("Getting model predictions for ensemble...")
        predictions = {}

        val_dataset = FishDataset(self.X_val, self.y_val, DataManager.get_transforms(False))
        val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

        for name, model in self.models.items():
            model.eval()
            all_preds = []
            all_probs = []
            all_losses = []
            all_labels = []
            total = 0
            correct = 0
            criterion = nn.CrossEntropyLoss()

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                    outputs = model(images)
                    probabilities = torch.softmax(outputs, dim=1)
                    loss = criterion(outputs, labels).item()

                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    all_preds.extend(predicted.cpu().numpy())
                    all_probs.extend(probabilities.cpu().numpy())
                    all_losses.append(loss)
                    all_labels.extend(labels.cpu().numpy())

            accuracy = correct / total
            f1 = f1_score(self.y_val, all_preds, average='macro')
            avg_loss = np.mean(all_losses)

            predictions[name] = {
                'predictions': np.array(all_preds),
                'probabilities': np.array(all_probs),
                'loss': avg_loss,
                'accuracy': accuracy,
                'f1': f1,
                'true_labels': np.array(all_labels)
            }

            print(f"  {name}: F1 = {f1:.4f}, Acc = {accuracy:.4f}, Loss = {avg_loss:.4f}")

        return predictions

    def simple_average_ensemble(self, model_combo):
        selected_probs = [self.model_predictions[name]['probabilities'] for name in model_combo]
        avg_probs = np.mean(selected_probs, axis=0)
        predictions = np.argmax(avg_probs, axis=1)

        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.mean([self.model_predictions[name]['loss'] for name in model_combo])

        avg_probs = np.mean(selected_probs, axis=0) if selected_probs else np.zeros((len(self.y_val), Config.NUM_CLASSES))

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'models': model_combo,
            # 'probabilities': avg_probs,
            'probabilities': avg_probs if avg_probs.ndim == 2 else np.zeros((0, Config.NUM_CLASSES)),
            'true_labels': self.y_val
        }

    def weighted_average_ensemble(self, model_combo):
        weights = []
        selected_probs = []

        for name in model_combo:
            f1 = self.model_predictions[name]['f1']
            weights.append(f1)
            selected_probs.append(self.model_predictions[name]['probabilities'])

        weights = np.array(weights) / np.sum(weights)
        weighted_probs = np.average(selected_probs, axis=0, weights=weights)
        predictions = np.argmax(weighted_probs, axis=1)

        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.average([self.model_predictions[name]['loss'] for name in model_combo], weights=weights)

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'weights': weights,
            'models': model_combo,
            'probabilities': weighted_probs,
            'true_labels': self.y_val
        }

    def confidence_based_ensemble(self, model_combo):
        final_predictions = []
        all_probs = []

        for i in range(len(self.y_val)):
            confidences = []
            probs = []

            for name in model_combo:
                prob = self.model_predictions[name]['probabilities'][i]
                confidence = np.max(prob)
                confidences.append(confidence)
                probs.append(prob)

            confidences = np.array(confidences)
            weights = confidences / np.sum(confidences) if np.sum(confidences) > 0 else np.ones(len(confidences)) / len(confidences)

            final_prob = np.average(probs, axis=0, weights=weights)
            final_predictions.append(np.argmax(final_prob))
            all_probs.append(final_prob)

        predictions = np.array(final_predictions)
        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.mean([self.model_predictions[name]['loss'] for name in model_combo])

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'models': model_combo,
            'probabilities': np.array(all_probs),
            'true_labels': self.y_val
        }

    def learnable_weighted_ensemble(self, model_combo, epochs=30):
        print(f"Training learnable weighted ensemble with {len(model_combo)} models...")

        selected_probs = []
        for name in model_combo:
            selected_probs.append(self.model_predictions[name]['probabilities'])

        ensemble_input = np.stack(selected_probs, axis=1)

        X_ensemble = torch.FloatTensor(ensemble_input).to(Config.DEVICE)
        y_ensemble = torch.LongTensor(self.y_val).to(Config.DEVICE)

        ensemble_model = LearnableWeightedEnsemble(
            num_models=len(model_combo),
            num_classes=Config.NUM_CLASSES
        ).to(Config.DEVICE)

        optimizer = optim.AdamW(ensemble_model.parameters(), lr=1e-3, weight_decay=1e-4)
        criterion = nn.CrossEntropyLoss()
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

        history = {'train_loss': [], 'train_acc': [], 'val_f1': []}
        best_loss = float('inf')

        for epoch in range(epochs):
            ensemble_model.train()
            optimizer.zero_grad()
            predictions, weights = ensemble_model(X_ensemble)
            loss = criterion(predictions, y_ensemble)
            loss.backward()
            optimizer.step()
            scheduler.step()

            accuracy = accuracy_score(y_ensemble.cpu().numpy(), predictions.argmax(dim=1).cpu().numpy())
            f1 = f1_score(y_ensemble.cpu().numpy(), predictions.argmax(dim=1).cpu().numpy(), average='macro')

            history['train_loss'].append(loss.item())
            history['train_acc'].append(accuracy)
            history['val_f1'].append(f1)

            print(f"Ensemble Epoch {epoch+1}/{epochs}: Loss = {loss.item():.4f}, "
                  f"Acc = {accuracy:.4f}, F1 = {f1:.4f}")

            if loss.item() < best_loss:
                best_loss = loss.item()
                torch.save(ensemble_model.state_dict(), f"{Config.OUTPUT_DIR}/models/learnable_ensemble_{'+'.join(model_combo)}.pt")

        ensemble_model.load_state_dict(torch.load(f"{Config.OUTPUT_DIR}/models/learnable_ensemble_{'+'.join(model_combo)}.pt"))
        ensemble_model.eval()
        with torch.no_grad():
            final_predictions, learned_weights = ensemble_model(X_ensemble)
            predictions = final_predictions.argmax(dim=1).cpu().numpy()
            probabilities = torch.softmax(final_predictions, dim=1).cpu().numpy()
            avg_weights = learned_weights.mean(dim=0).cpu().numpy()

        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.mean([self.model_predictions[name]['loss'] for name in model_combo])

        self.histories[f"learnable_weighted_{'+'.join(model_combo)}"] = history

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'models': model_combo,
            'learned_weights': avg_weights,
            'probabilities': probabilities,
            'true_labels': self.y_val
        }

    def test_ensemble_combinations(self):
        print("Testing ensemble combinations...")

        model_names = list(self.models.keys())
        all_results = {}
        best_result = None
        best_score = 0

        for size in range(2, min(len(model_names) + 1, 5)):
            print(f"Testing {size}-model combinations...")

            for combo in list(combinations(model_names, size))[:5]:
                combo_name = f"combo_{size}_{'+'.join(combo)}"

                for method_name in Config.ENSEMBLE_METHODS:
                    full_name = f"{combo_name}_{method_name}"

                    try:
                        if method_name == 'simple_average':
                            result = self.simple_average_ensemble(combo)
                        elif method_name == 'weighted_average':
                            result = self.weighted_average_ensemble(combo)
                        elif method_name == 'confidence_based':
                            result = self.confidence_based_ensemble(combo)
                        elif method_name == 'learnable_weighted':
                            result = self.learnable_weighted_ensemble(combo)

                        # Verify result contains required keys
                        required_keys = ['accuracy', 'f1', 'loss', 'predictions', 'models', 'probabilities', 'true_labels']
                        if not all(key in result for key in required_keys):
                            missing = [key for key in required_keys if key not in result]
                            print(f"  {full_name}: Missing keys {missing}")
                            continue
                        # Ensure probabilities is 2D
                        if 'probabilities' in result and (result['probabilities'].ndim != 2 or result['probabilities'].shape[1] != Config.NUM_CLASSES):
                            result['probabilities'] = np.zeros((len(result['true_labels']), Config.NUM_CLASSES))

                        all_results[full_name] = result
                        print(f"  {full_name}: F1 = {result['f1']:.4f}, Acc = {result['accuracy']:.4f}, "
                              f"Loss = {result['loss']:.4f}, True Labels Shape = {result['true_labels'].shape}")

                        if result['f1'] > best_score:
                            best_score = result['f1']
                            best_result = (full_name, result)

                    except Exception as e:
                        print(f"  {full_name}: FAILED - {str(e)}")

        if best_result:
            print(f"\n✓ Best ensemble: {best_result[0]} (F1: {best_result[1]['f1']:.4f})")
        else:
            print("\nNo valid ensemble results generated.")

        return all_results, best_result


# ---
# 7. ENHANCED VISUALIZATIONS
# ================================================================================================================================
# Purpose: Generate visualizations for training history, ROC curves, confusion matrices, model comparisons, and XAI visualizations.

class EnhancedVisualizations:
    def __init__(self):
        self.viz_dir = f"{Config.OUTPUT_DIR}/visualizations"
        Path(self.viz_dir).mkdir(parents=True, exist_ok=True)
        # Set better matplotlib parameters for spacing
        plt.rcParams.update({
            'figure.autolayout': True,
            'axes.titlepad': 20,
            'axes.labelpad': 10,
            'xtick.major.pad': 8,
            'ytick.major.pad': 8
        })


    def plot_single_model_history(self, history, model_name, alpha=0.2):  # (range: 0.1–0.3; lower = smoother, higher = more responsive)
        """Plot training history for individual model with better spacing"""
        if not history['train_loss']:
            print(f"Skipping {model_name}: No training data available")
            return

        # def smooth_ema(values, alpha):
        #     """Apply exponential moving average (EMA) smoothing to a list of values."""
        #     if not values:
        #         return values
        #     smoothed = [values[0]]
        #     for val in values[1:]:
        #         smoothed.append(alpha * val + (1 - alpha) * smoothed[-1])
        #     return smoothed
        def smooth_ema(values, alpha=0.1):
            """
            Apply exponential moving average smoothing to a list or tensor of values.

            Args:
                values: List, NumPy array, or PyTorch tensor of numerical values
                alpha: Smoothing factor (default: 0.1)

            Returns:
                List of smoothed values
            """
            # Convert tensor to list if necessary
            if torch.is_tensor(values):
                values = values.cpu().numpy().tolist()
            elif isinstance(values, np.ndarray):
                values = values.tolist()

            if not values:  # Check if the list is empty
                return []

            smoothed = [values[0]]
            for i in range(1, len(values)):
                smoothed.append(alpha * values[i] + (1 - alpha) * smoothed[-1])

            return smoothed



        epochs = range(1, len(history['train_loss']) + 1)

        # Create subplot with more space
        fig = plt.figure(figsize=(16, 10))
        gs = fig.add_gridspec(2, 2, hspace=0.4, wspace=0.3)

        # Loss plot
        ax1 = fig.add_subplot(gs[0, 0])
        smoothed_train_loss = smooth_ema(history['train_loss'], alpha)
        smoothed_val_loss = smooth_ema(history.get('val_loss', []), alpha)
        ax1.plot(epochs, smoothed_train_loss, 'b-', linewidth=2, label='Train Loss', marker='o', markersize=4)
        if smoothed_val_loss:
            ax1.plot(epochs, smoothed_val_loss, 'r-', linewidth=2, label='Val Loss', marker='s', markersize=4)
        ax1.set_title(f'{model_name} - Loss vs Epoch', fontsize=14, fontweight='bold', pad=20)
        ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Loss', fontsize=12, fontweight='bold')
        ax1.legend(frameon=True, shadow=True, fontsize=11)
        ax1.grid(True, alpha=0.3)
        ax1.tick_params(labelsize=10)

        # Accuracy plot
        ax2 = fig.add_subplot(gs[0, 1])
        ax2.plot(epochs, history['train_acc'], 'g-', linewidth=2, label='Train Acc', marker='o', markersize=4)
        if history.get('val_acc', []):
            ax2.plot(epochs, history['val_acc'], 'm-', linewidth=2, label='Val Acc', marker='s', markersize=4)
        ax2.set_title(f'{model_name} - Accuracy vs Epoch', fontsize=14, fontweight='bold', pad=20)
        ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
        ax2.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
        ax2.legend(frameon=True, shadow=True, fontsize=11)
        ax2.grid(True, alpha=0.3)
        ax2.tick_params(labelsize=10)

        # F1 Score plot
        ax3 = fig.add_subplot(gs[1, 0])
        smoothed_val_f1 = smooth_ema(history.get('val_f1', []), alpha)
        if smoothed_val_f1:
            ax3.plot(epochs, smoothed_val_f1, 'orange', linewidth=2, label='Val F1', marker='d', markersize=4)
            ax3.set_title(f'{model_name} - F1 Score vs Epoch', fontsize=14, fontweight='bold', pad=20)
            ax3.set_xlabel('Epoch', fontsize=12, fontweight='bold')
            ax3.set_ylabel('F1 Score', fontsize=12, fontweight='bold')
            ax3.legend(frameon=True, shadow=True, fontsize=11)
            ax3.grid(True, alpha=0.3)
            ax3.tick_params(labelsize=10)

        # Learning Rate plot
        ax4 = fig.add_subplot(gs[1, 1])
        smoothed_lr = smooth_ema(history.get('learning_rates', []), alpha)
        if smoothed_lr:
            ax4.plot(epochs, smoothed_lr, 'purple', linewidth=2, label='Learning Rate', marker='x', markersize=6)
            ax4.set_title(f'{model_name} - Learning Rate vs Epoch', fontsize=14, fontweight='bold', pad=20)
            ax4.set_xlabel('Epoch', fontsize=12, fontweight='bold')
            ax4.set_ylabel('Learning Rate', fontsize=12, fontweight='bold')
            ax4.legend(frameon=True, shadow=True, fontsize=11)
            ax4.grid(True, alpha=0.3)
            ax4.tick_params(labelsize=10)
            ax4.set_yscale('log')

        plt.suptitle(f'{model_name} Training Progress', fontsize=18, fontweight='bold', y=0.98)
        save_path = f"{self.viz_dir}/{model_name}_individual_training_history.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"Individual training history saved: {save_path}")



    def plot_single_model_xai(self, model, model_name, test_loader, max_images=5):
        """Generate XAI visualizations for a single model using Grad-CAM++, Integrated Gradients, and LRP"""
        print(f"Generating XAI visualizations for {model_name}...")
        xai_visualizer = XAIVisualizer()

        # # Get sample images
        # sample_images, sample_labels = next(iter(test_loader))
        # sample_images = sample_images[:max_images].to(Config.DEVICE)
        # sample_labels = sample_labels[:max_images].numpy()

        # Get sample images
        sample_images = []
        sample_labels = []
        for i in range(Config.NUM_CLASSES):
            class_mask = np.where(test_loader.dataset.labels == i)[0]
            if len(class_mask) > 0:
                idx = np.random.choice(class_mask)
                image, label = test_loader.dataset[idx]
                if image.dim() == 4:
                    image = image.squeeze(0)
                sample_images.append(image.to(Config.DEVICE))
                sample_labels.append(label)


        for idx, (image, true_label) in enumerate(zip(sample_images, sample_labels)):
            try:
                # Create figure with larger size for clearer images
                fig = plt.figure(figsize=(24, 8))  # Standard size for clear visualization
                gs = fig.add_gridspec(1, 4, wspace=0.15, hspace=0.2)  # 4 columns: Original, Grad-CAM++, Integrated Gradients, LRP

                # Prepare original image
                image_np = image.detach().permute(1, 2, 0).cpu().numpy()
                image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                image_np = np.clip(image_np, 0, 1)
                image_np = (image_np * 255).astype(np.uint8)

                # Plot original image
                ax_orig = fig.add_subplot(gs[0, 0])
                ax_orig.imshow(image_np)
                ax_orig.set_title(f'Original Image\nTrue: {Config.CLASS_LABELS[true_label]}',
                                 fontsize=14, fontweight='bold', pad=15)
                ax_orig.axis('off')

                # Forward pass for prediction
                model.eval()
                with torch.no_grad():
                    outputs = model(image.unsqueeze(0))
                    probabilities = torch.softmax(outputs, dim=1)
                    predicted_class = outputs.argmax(dim=1).item()
                    confidence = probabilities[0, predicted_class].item()
                    predicted_label = Config.CLASS_LABELS[predicted_class]

                # Generate Grad-CAM++ visualization
                gradcam_img, _ = xai_visualizer.grad_cam_plus_plus(model, image, predicted_class)
                ax_gradcam = fig.add_subplot(gs[0, 1])
                ax_gradcam.imshow(gradcam_img)
                ax_gradcam.set_title(f'{model_name}\nGrad-CAM++\nPred: {predicted_label}\nConf: {confidence:.2%}',
                                    fontsize=12, fontweight='bold', pad=10)
                ax_gradcam.axis('off')

                # Generate Integrated Gradients visualization
                ig_img, _ = xai_visualizer.integrated_gradients(model, image, predicted_class)
                ax_ig = fig.add_subplot(gs[0, 2])
                ax_ig.imshow(ig_img)
                ax_ig.set_title(f'{model_name}\nIntegrated Gradients', fontsize=12, fontweight='bold', pad=10)
                ax_ig.axis('off')

                # Generate LRP visualization
                lrp_img, _ = xai_visualizer.layer_wise_relevance_propagation(model, image, predicted_class)
                ax_lrp = fig.add_subplot(gs[0, 3])
                # ax_lrp.imshow(lrp_img)
                ax_lrp.imshow(cv2.cvtColor(lrp_img, cv2.COLOR_BGR2RGB))
                ax_lrp.set_title(f'{model_name}\nLRP', fontsize=12, fontweight='bold', pad=10)
                ax_lrp.axis('off')

                plt.suptitle(f'XAI Analysis for {model_name} - Image {idx+1}', fontsize=16, fontweight='bold', y=0.95)
                save_path = f"{self.viz_dir}/{model_name}_xai_image_{idx+1}.png"
                plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
                plt.show()
                print(f"XAI visualization saved: {save_path}")

            except Exception as e:
                print(f"Error generating XAI for {model_name}, image {idx+1}: {e}")
                continue

    def plot_confusion_matrix(self, results, model_name):
        """Enhanced confusion matrix with better spacing"""
        if 'true_labels' not in results or 'predictions' not in results:
            print(f"Error: Missing 'true_labels' or 'predictions' in results for {model_name}")
            return

        cm = confusion_matrix(results['true_labels'], results['predictions'])
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        # Create figure with better spacing
        fig, ax = plt.subplots(figsize=(12, 10))

        # Use better color scheme and formatting
        sns.heatmap(cm_normalized, annot=cm, fmt='d', cmap='Blues',
                    xticklabels=Config.CLASS_LABELS, yticklabels=Config.CLASS_LABELS,
                    cbar_kws={'label': 'Normalized Count', 'shrink': 0.8},
                    square=True, linewidths=0.5, annot_kws={'size': 12})

        # === Move colorbar label to the top ===
        cbar = ax.collections[0].colorbar
        cbar.ax.yaxis.set_label_position('left')   # keep label aligned left of bar
        cbar.set_label("Normalized Count", rotation=0, labelpad=15)
        cbar.ax.yaxis.set_label_coords(-1.2, 1.02)  # fine-tune position (x, y)

        ax.set_title(f'Confusion Matrix: {model_name}', fontsize=16, fontweight='bold', pad=25)
        ax.set_xlabel('Predicted Label', fontsize=14, fontweight='bold', labelpad=15)
        ax.set_ylabel('True Label', fontsize=14, fontweight='bold', labelpad=15)

        # Rotate labels for better readability
        plt.xticks(rotation=45, ha='right', fontsize=12)
        plt.yticks(rotation=0, fontsize=12)

        # Add performance metrics as text
        accuracy = accuracy_score(results['true_labels'], results['predictions'])
        f1_macro = f1_score(results['true_labels'], results['predictions'], average='macro')
        f1_weighted = f1_score(results['true_labels'], results['predictions'], average='weighted')

        metrics_text = f'Accuracy: {accuracy:.4f}\nF1 (Macro): {f1_macro:.4f}\nF1 (Weighted): {f1_weighted:.4f}'
        ax.text(1.15, 0.5, metrics_text, transform=ax.transAxes, fontsize=12,
                verticalalignment='center', bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))

        save_path = f"{self.viz_dir}/{model_name}_enhanced_confusion_matrix.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"Enhanced confusion matrix saved: {save_path}")





    def plot_roc_curves(self, results, model_name):
        """Plot ROC curves for multi-class classification."""
        import numpy as np
        """Enhanced ROC curves with better spacing and styling for multiclass"""
        if 'true_labels' not in results or 'probabilities' not in results:
            print(f"Error: Missing 'true_labels' or 'probabilities' in results for {model_name}")
            return

        fig, ax = plt.subplots(figsize=(12, 10))
        colors = plt.cm.Set3(np.linspace(0, 1, Config.NUM_CLASSES))
        probabilities = np.array(results['probabilities'])
        if len(results['true_labels']) == 0 or probabilities.size == 0 or probabilities.ndim != 2 or probabilities.shape[1] != Config.NUM_CLASSES:
            print(f"Warning: Invalid or empty probabilities shape {probabilities.shape} for {model_name}. Skipping ROC plot.")
            ax.text(0.5, 0.5, 'No data available for ROC', horizontalalignment='center', verticalalignment='center', fontsize=12)
            return
        colors = plt.cm.Set3(np.linspace(0, 1, Config.NUM_CLASSES))
        for i in range(Config.NUM_CLASSES):
            y_true_bin = (np.array(results['true_labels']) == i).astype(int)
            y_score = probabilities[:, i]

            fpr, tpr, _ = roc_curve(y_true_bin, y_score)
            roc_auc = auc(fpr, tpr)
            ax.plot(fpr, tpr, color=colors[i], linewidth=2,
                    label=f'{Config.CLASS_LABELS[i]} (AUC = {roc_auc:.4f})')

        # Diagonal line
        ax.plot([0, 1], [0, 1], 'k--', linewidth=2, alpha=0.6, label='Random Classifier')

        ax.set_title(f'{model_name} - ROC Curves', fontsize=16, fontweight='bold', pad=20)
        ax.set_xlabel('False Positive Rate', fontsize=14, fontweight='bold', labelpad=10)
        ax.set_ylabel('True Positive Rate', fontsize=14, fontweight='bold', labelpad=10)
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', frameon=True, shadow=True, fontsize=11)
        ax.grid(True, alpha=0.3)
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])

        # Compute micro-average ROC curve
        y_true_bin = label_binarize(results['true_labels'], classes=range(Config.NUM_CLASSES))
        fpr_micro, tpr_micro, _ = roc_curve(y_true_bin.ravel(), results['probabilities'].ravel())
        roc_auc_micro = auc(fpr_micro, tpr_micro)
        ax.plot(fpr_micro, tpr_micro, 'deeppink', linestyle=':', linewidth=4,
                label=f'Micro-average (AUC = {roc_auc_micro:.4f})')

        save_path = f"{self.viz_dir}/{model_name}_enhanced_roc_curves.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"Enhanced ROC curves saved: {save_path}")




    def plot_misclassified_images(self, misclassified, model_name):
        """Plot up to 4 misclassified images in a 2x2 grid with improved visibility."""
        import matplotlib.pyplot as plt
        import numpy as np

        if not misclassified:
            print(f"No misclassified images for {model_name}")
            return

        # Select up to 4 misclassified images
        misclassified = misclassified[:4]
        fig, axes = plt.subplots(2, 2, figsize=(10, 10))  # Slightly smaller size
        axes = axes.flatten()

        for idx, item in enumerate(misclassified):
            # image = item['image'].permute(1, 2, 0).cpu().numpy()  # Convert CHW -> HWC
            image = item['image']
            if isinstance(image, torch.Tensor):
                image = image.permute(1, 2, 0).cpu().numpy()  # CHW -> HWC
            elif isinstance(image, np.ndarray):
                if image.shape[0] in [1, 3]:  # If channel-first
                    image = np.transpose(image, (1, 2, 0))  # CHW -> HWC


            # Normalize for display
            if image.max() > 1.0:
                image = image / 255.0
            else:
                image = (image - image.min()) / (image.max() - image.min() + 1e-8)
            image = np.clip(image, 0, 1)

            # Grayscale vs RGB
            if image.shape[2] == 1:
                image = image.squeeze()
                axes[idx].imshow(image, cmap='gray')
            else:
                axes[idx].imshow(image)

            # Titles with color
            true_label = Config.CLASS_LABELS[item['true_label']]
            pred_label = Config.CLASS_LABELS[item['pred_label']]
            axes[idx].set_title(
                f"True: {true_label}\nPred: {pred_label}",
                fontsize=9,
                fontweight="bold",
                pad=10,
                color="black" #if true_label != pred_label else "darkgreen"
            )

            axes[idx].axis("off")
            # Add white border around each image
            for spine in axes[idx].spines.values():
                spine.set_edgecolor("lightgray")
                spine.set_linewidth(3)

        # Hide unused subplots (if <4 images)
        for idx in range(len(misclassified), 4):
            axes[idx].axis("off")

        plt.suptitle(
            f"Misclassified Images for {model_name}",
            fontsize=12,
            fontweight="bold",
            y=0.99,
            color="black"
        )
        plt.subplots_adjust(wspace=0.3, hspace=0.4)  # Increase padding between plots
        save_path = f"{self.viz_dir}/misclassified_{model_name}.png"
        plt.savefig(save_path, bbox_inches="tight", dpi=300, facecolor="white")
        plt.show()
        plt.close()
        print(f"Saved misclassified images plot for {model_name} at {save_path}")





    def plot_kfold_results(self, fold_results, model_name, test_loader):
        """Plot k-fold results: 2x2 grid per fold and 1x2 comparison across folds."""
        n_folds = len(fold_results)
        plt.style.use('seaborn-v0_8')  # Use modern Seaborn style

        # 2x2 Grid Plot for Each Fold
        for fold in range(n_folds):
            fig, axes = plt.subplots(2, 2, figsize=(12, 10))
            fig.suptitle(f'{model_name} - Fold {fold+1} Metrics', fontsize=16)
            history = fold_results[fold]['history']

            # Plot Validation Accuracy
            axes[0, 0].plot(history['val_acc'], label='Validation Accuracy', marker='s')
            axes[0, 0].set_title('Validation Accuracy')
            axes[0, 0].set_xlabel('Epoch')
            axes[0, 0].set_ylabel('Accuracy')
            axes[0, 0].legend()
            axes[0, 0].grid(True)

            # Plot Validation Loss
            axes[0, 1].plot(history['val_loss'], label='Validation Loss', marker='s')
            axes[0, 1].set_title('Validation Loss')
            axes[0, 1].set_xlabel('Epoch')
            axes[0, 1].set_ylabel('Loss')
            axes[0, 1].legend()
            axes[0, 1].grid(True)

            # Hide unused subplots
            axes[1, 0].set_visible(False)
            axes[1, 1].set_visible(False)

            plt.tight_layout()
            plt.show()

        # 1x2 Comparison Plot Across Folds
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        fig.suptitle(f'{model_name} - Cross-Fold Comparison', fontsize=16)

        # Plot Validation Accuracy Across Folds
        for fold in range(n_folds):
            history = fold_results[fold]['history']
            axes[0].plot(history['val_acc'], label=f'Fold {fold+1}', marker='s')
        axes[0].set_title('Validation Accuracy Across Folds')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True)

        # Plot Validation Loss Across Folds
        for fold in range(n_folds):
            history = fold_results[fold]['history']
            axes[1].plot(history['val_loss'], label=f'Fold {fold+1}', marker='s')
        axes[1].set_title('Validation Loss Across Folds')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True)

        plt.tight_layout()
        plt.show()


    def plot_model_results(self, model_name, fold_results, test_loader):
        """Plot model results: ROC curves, confusion matrix, misclassified images, XAI, and k-fold results."""
        print(f"Generating plots for {model_name}")

        # Plot single-model results using first fold's result
        first_result = fold_results[0]['result']
        self.plot_roc_curves(first_result, model_name)
        self.plot_confusion_matrix(first_result, model_name)
        self.plot_misclassified_images(first_result['misclassified'], model_name)

        # Recreate model for XAI plotting
        model = ModelFactory.create_model(
            model_name,
            num_classes=Config.NUM_CLASSES,
            dropout_rate=0.5,  # Use default or pass from hyperparameters
            hidden_dim_multiplier=0.5
        ).to(Config.DEVICE)
        self.plot_single_model_xai(model, model_name, test_loader)

        # Plot k-fold results
        self.plot_kfold_results(fold_results, model_name, test_loader)

        print("Completed plotting k-fold results")

        # Clean up
        del model, fold_results, first_result
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        gc.collect()
        # print(f"Plotting memory cleared: {torch.cuda.memory_summary()}")






    def generate_comprehensive_report(self, single_results, ensemble_results, best_ensemble):
        """Generate a comprehensive visual report"""
        # fig = plt.figure(figsize=(24, 16))
        fig = plt.figure(figsize=(30, 20))
        gs = fig.add_gridspec(3, 4, hspace=0.4, wspace=0.3, top=0.92, bottom=0.08, left=0.05, right=0.95)

        # Title
        fig.suptitle('Fish Species Classification - Comprehensive Analysis Report',
                    fontsize=24, fontweight='bold', y=0.96)

        # 1. Model Performance Comparison
        ax1 = fig.add_subplot(gs[0, :2])
        model_names = list(single_results.keys())
        accuracies = [single_results[name]['accuracy'] for name in model_names]
        f1_scores = [single_results[name]['f1_macro'] for name in model_names]

        x = np.arange(len(model_names))
        width = 0.35

        bars1 = ax1.bar(x - width/2, accuracies, width, label='Accuracy', alpha=0.8, color='skyblue')
        bars2 = ax1.bar(x + width/2, f1_scores, width, label='F1 Score', alpha=0.8, color='lightcoral')

        ax1.set_xlabel('Models', fontweight='bold', fontsize=12)
        ax1.set_ylabel('Score', fontweight='bold', fontsize=12)
        ax1.set_title('Individual Model Performance', fontweight='bold', fontsize=14, pad=15)
        ax1.set_xticks(x)
        ax1.set_xticklabels(model_names, rotation=45, ha='right')
        ax1.legend(fontsize=11)
        ax1.grid(True, alpha=0.3)
        ax1.set_ylim(0, 1.1)

        # Add value labels on bars
        for bar in bars1 + bars2:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.4f}', ha='center', va='bottom', fontsize=9)

        # 2. Best vs Worst Model Comparison
        ax2 = fig.add_subplot(gs[0, 2:])
        if ensemble_results and best_ensemble:
            best_single_f1 = max(f1_scores)
            worst_single_f1 = min(f1_scores)
            best_ensemble_f1 = best_ensemble[1]['f1']

            categories = ['Worst Single', 'Best Single', 'Best Ensemble']
            values = [worst_single_f1, best_single_f1, best_ensemble_f1]
            colors = ['lightcoral', 'lightblue', 'gold']

            bars = ax2.bar(categories, values, color=colors, alpha=0.8)
            ax2.set_ylabel('F1 Score', fontweight='bold', fontsize=12)
            ax2.set_title('Performance Comparison', fontweight='bold', fontsize=14, pad=15)
            ax2.grid(True, alpha=0.3)
            ax2.set_ylim(0, max(values) * 1.1)

            for bar, value in zip(bars, values):
                ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{value:.4f}', ha='center', va='bottom', fontweight='bold')

        # 3. Per-class Performance Heatmap
        ax3 = fig.add_subplot(gs[1, :2])
        per_class_f1 = []
        for model_name in model_names:
            per_class_f1.append(single_results[model_name]['f1_per_class'])

        per_class_f1 = np.array(per_class_f1)
        im = ax3.imshow(per_class_f1, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)

        ax3.set_xticks(range(len(Config.CLASS_LABELS)))
        ax3.set_xticklabels(Config.CLASS_LABELS, rotation=45, ha='right')
        ax3.set_yticks(range(len(model_names)))
        ax3.set_yticklabels(model_names)
        ax3.set_title('Per-Class F1 Scores Heatmap', fontweight='bold', fontsize=14, pad=15)

        # Add colorbar
        cbar = plt.colorbar(im, ax=ax3, shrink=0.8)
        cbar.set_label('F1 Score', rotation=270, labelpad=15)

        # Add text annotations
        for i in range(len(model_names)):
            for j in range(len(Config.CLASS_LABELS)):
                text_color = 'white' if per_class_f1[i, j] < 0.5 else 'black'
                ax3.text(j, i, f'{per_class_f1[i, j]:.2f}',
                        ha="center", va="center", color=text_color, fontsize=8, fontweight='bold')

        # 4. Ensemble Methods Performance
        ax4 = fig.add_subplot(gs[1, 2:])
        if ensemble_results:
            ensemble_items = list(ensemble_results.items())
            ensemble_items.sort(key=lambda x: x[1]['f1'], reverse=True)
            top_ensembles = ensemble_items[:8]

            ensemble_names = [name.split('_')[-1] for name, _ in top_ensembles]
            ensemble_f1s = [result['f1'] for _, result in top_ensembles]

            bars = ax4.barh(range(len(ensemble_names)), ensemble_f1s, alpha=0.8, color='lightgreen')
            ax4.set_yticks(range(len(ensemble_names)))
            ax4.set_yticklabels(ensemble_names)
            ax4.set_xlabel('F1 Score', fontweight='bold', fontsize=12)
            ax4.set_title('Top Ensemble Methods', fontweight='bold', fontsize=14, pad=15)
            ax4.grid(True, alpha=0.3)

            for bar, value in zip(bars, ensemble_f1s):
                ax4.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2,
                        f'{value:.4f}', ha='left', va='center', fontsize=9, fontweight='bold')

        # 5. Summary Statistics Table
        ax5 = fig.add_subplot(gs[2, :])
        ax5.axis('off')

        # Create summary data
        summary_data = []
        for model_name in model_names:
            result = single_results[model_name]
            summary_data.append([
                model_name,
                f"{result['accuracy']:.4f}",
                f"{result['f1_macro']:.4f}",
                f"{result['f1_weighted']:.4f}",
                f"{result['precision_macro']:.4f}",
                f"{result['recall_macro']:.4f}"
            ])

        if ensemble_results and best_ensemble:
            best_result = best_ensemble[1]
            summary_data.append([
                f"Best Ensemble\n({best_ensemble[0]})",
                f"{best_result['accuracy']:.4f}",
                f"{best_result['f1']:.4f}",
                "N/A",
                "N/A",
                "N/A"
            ])

        columns = ['Model', 'Accuracy', 'F1 (Macro)', 'F1 (Weighted)', 'Precision', 'Recall']

        table = ax5.table(cellText=summary_data, colLabels=columns, loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 2)

        # Style the table
        for i in range(len(columns)):
            table[(0, i)].set_facecolor('#4CAF50')
            table[(0, i)].set_text_props(weight='bold', color='white')

        for i in range(1, len(summary_data) + 1):
            for j in range(len(columns)):
                if i % 2 == 0:
                    table[(i, j)].set_facecolor('#f0f0f0')
                else:
                    table[(i, j)].set_facecolor('white')

        ax5.set_title('Model Performance Summary', fontweight='bold', fontsize=16, pad=20)

        save_path = f"{self.viz_dir}/comprehensive_report.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"Comprehensive report saved: {save_path}")
        return save_path





    def plot_model_comparison(self, single_results, ensemble_results):
        """Enhanced model comparison with better spacing"""
        # fig = plt.figure(figsize=(20, 16))
        fig = plt.figure(figsize=(30, 20))
        gs = fig.add_gridspec(2, 2, hspace=0.4, wspace=0.3, top=0.92, bottom=0.08)

        model_names = list(single_results.keys())
        accuracies = [single_results[name]['accuracy'] for name in model_names]
        f1_scores = [single_results[name]['f1_macro'] for name in model_names]
        f1_weighted = [single_results[name]['f1_weighted'] for name in model_names]
        # losses = [single_results[name]['loss'] for name in model_names]
        # Use f1_macro as substitute since loss is not available in single_results
        losses = [1 - single_results[name]['f1_macro'] for name in model_names]  # Convert F1 to loss-like metric

        x = np.arange(len(model_names))
        width = 0.2

        ax1 = fig.add_subplot(gs[0, 0])
        bars1 = ax1.bar(x - width*1.5, accuracies, width, label='Accuracy', alpha=0.8, color='skyblue')
        bars2 = ax1.bar(x - width/2, f1_scores, width, label='F1 (Macro)', alpha=0.8, color='lightcoral')
        bars3 = ax1.bar(x + width/2, f1_weighted, width, label='F1 (Weighted)', alpha=0.8, color='lightgreen')
        bars4 = ax1.bar(x + width*1.5, losses, width, label='Loss', alpha=0.8, color='salmon')

        ax1.set_xlabel('Models', fontweight='bold', fontsize=12)
        ax1.set_ylabel('Score', fontweight='bold', fontsize=12)
        ax1.set_title('Individual Model Performance', fontweight='bold', fontsize=14, pad=20)
        ax1.set_xticks(x)
        ax1.set_xticklabels(model_names, rotation=45, ha='right', fontsize=10)
        ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
        ax1.grid(True, alpha=0.3)
        ax1.set_ylim(0, max(max(accuracies), max(f1_scores), max(f1_weighted), max(losses)) * 1.1)

        if ensemble_results:
            ensemble_items = list(ensemble_results.items())
            ensemble_items.sort(key=lambda x: x[1]['f1'], reverse=True)
            top_ensembles = ensemble_items[:10]

            ensemble_names = [name.split('_')[-1] for name, _ in top_ensembles]
            ensemble_f1s = [result['f1'] for _, result in top_ensembles]
            ensemble_accs = [result['accuracy'] for _, result in top_ensembles]
            ensemble_losses = [result['loss'] for _, result in top_ensembles]

            x = np.arange(len(ensemble_names))
            ax2 = fig.add_subplot(gs[0, 1])
            ax2.bar(x - width, ensemble_accs, width, label='Accuracy', alpha=0.8, color='skyblue')
            ax2.bar(x, ensemble_f1s, width, label='F1 (Macro)', alpha=0.8, color='lightcoral')
            ax2.bar(x + width, ensemble_losses, width, label='Loss', alpha=0.8, color='salmon')

            ax2.set_xticks(x)
            ax2.set_xticklabels([f"{name}" for name in ensemble_names], rotation=45, ha='right', fontsize=10)
            ax2.set_xlabel('Ensemble Methods', fontweight='bold', fontsize=12)
            ax2.set_ylabel('Score', fontweight='bold', fontsize=12)
            ax2.set_title('Top Ensemble Methods', fontweight='bold', fontsize=14, pad=20)
            ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
            ax2.grid(True, alpha=0.3)
            ax2.set_ylim(0, max(max(ensemble_accs), max(ensemble_f1s), max(ensemble_losses)) * 1.1)

        if single_results:
            per_class_f1 = []
            for model_name in model_names:
                per_class_f1.append(single_results[model_name]['f1_per_class'])

            per_class_f1 = np.array(per_class_f1)
            ax3 = fig.add_subplot(gs[1, 0])
            im = ax3.imshow(per_class_f1, cmap='RdYlBu_r', aspect='auto', vmin=0, vmax=1)

            ax3.set_xticks(range(len(Config.CLASS_LABELS)))
            ax3.set_xticklabels(Config.CLASS_LABELS, rotation=45, ha='right', fontsize=10)
            ax3.set_yticks(range(len(model_names)))
            ax3.set_yticklabels(model_names, fontsize=10)
            ax3.set_title('Per-Class F1 Scores', fontweight='bold', fontsize=14, pad=20)

            cbar = plt.colorbar(im, ax=ax3, shrink=0.8)
            cbar.set_label('F1 Score', rotation=270, labelpad=15)

            for i in range(len(model_names)):
                for j in range(len(Config.CLASS_LABELS)):
                    text_color = 'white' if per_class_f1[i, j] < 0.5 else 'black'
                    ax3.text(j, i, f'{per_class_f1[i, j]:.2f}',
                            ha="center", va="center", color=text_color, fontsize=8)

        if ensemble_results:
            best_single_f1 = max(f1_scores)
            best_single_name = model_names[f1_scores.index(best_single_f1)]
            best_ensemble_f1 = max([result['f1'] for result in ensemble_results.values()])
            best_ensemble_name = max(ensemble_results.items(), key=lambda x: x[1]['f1'])[0]

            ax4 = fig.add_subplot(gs[1, 1])
            categories = ['Best Single\nModel', 'Best Ensemble']
            values = [best_single_f1, best_ensemble_f1]
            colors = ['lightblue', 'gold']

            bars = ax4.bar(categories, values, color=colors, alpha=0.8, width=0.6)
            ax4.set_ylabel('F1 Score', fontweight='bold', fontsize=12)
            ax4.set_title('Single vs Ensemble Comparison', fontweight='bold', fontsize=14, pad=20)
            ax4.grid(True, alpha=0.3)
            ax4.set_ylim(0, 1)

            for bar, value in zip(bars, values):
                ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                        f'{value:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=12)

            improvement = ((best_ensemble_f1 - best_single_f1) / best_single_f1) * 100
            ax4.text(0.5, 0.5, f'Improvement:\n{improvement:.2f}%',
                    transform=ax4.transAxes, ha='center', va='center',
                    fontsize=14, fontweight='bold',
                    bbox=dict(boxstyle="round,pad=0.5", facecolor="yellow", alpha=0.8))

        plt.suptitle('Fish Species Classification - Model Comparison Analysis',
                    fontsize=18, fontweight='bold', y=0.96)
        save_path = f"{self.viz_dir}/enhanced_model_comparison.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"Enhanced model comparison saved: {save_path}")
        return save_path




    def plot_lrp_grid(self, single_models, test_loader):
        print("Generating LRP grid visualization for TP, TN, FP, FN...")
        xai_visualizer = XAIVisualizer()

        # Compute predictions and confusion matrix
        model = list(single_models.values())[0]
        model.eval()
        true_labels = []
        pred_labels = []
        with torch.no_grad():
            for batch in test_loader:
                # images, labels = batch['image'].to(Config.DEVICE), batch['label'].to(Config.DEVICE)
                # images, labels = batch[0].to(Config.DEVICE), batch[1].to(Config.DEVICE)  # Access tuple elements
                # Make it consistent with your data loader format:
                if isinstance(batch, dict):
                    images, labels = batch['image'].to(Config.DEVICE), batch['label'].to(Config.DEVICE)
                elif isinstance(batch, (list, tuple)) and len(batch) == 2:
                    images, labels = batch[0].to(Config.DEVICE), batch[1].to(Config.DEVICE)
                else:
                    raise ValueError(f"Unexpected batch format: {type(batch)}")

                outputs = model(images)
                preds = outputs.argmax(dim=1) if outputs.dim() > 1 else (outputs > 0.5).float()
                true_labels.extend(labels.cpu().numpy())
                pred_labels.extend(preds.cpu().numpy())

        cm = confusion_matrix(true_labels, pred_labels)
        classes = Config.CLASS_LABELS
        tp = np.diag(cm)
        fp = cm.sum(axis=0) - tp
        fn = cm.sum(axis=1) - tp
        tn = cm.sum() - (fp + fn + tp)
        metrics = [tp, tn, fp, fn]

        # Collect one image per class per category (TP, TN, FP, FN)
        sample_images = {cls: {"TP": None, "TN": None, "FP": None, "FN": None} for cls in range(5)}
        with torch.no_grad():
            for batch in test_loader:
                # images, labels = batch['image'].to(Config.DEVICE), batch['label'].to(Config.DEVICE)
                # images, labels = batch[0].to(Config.DEVICE), batch[1].to(Config.DEVICE)  # Access tuple elements
                # Make it consistent with your data loader format:
                if isinstance(batch, dict):
                    images, labels = batch['image'].to(Config.DEVICE), batch['label'].to(Config.DEVICE)
                elif isinstance(batch, (list, tuple)) and len(batch) == 2:
                    images, labels = batch[0].to(Config.DEVICE), batch[1].to(Config.DEVICE)
                else:
                    raise ValueError(f"Unexpected batch format: {type(batch)}")



                outputs = model(images)
                preds = outputs.argmax(dim=1) if outputs.dim() > 1 else (outputs > 0.5).float()
                for img, true, pred in zip(images, labels, preds):
                    cls = int(true.item())
                    pred_cls = int(pred.item())
                    for i in range(5):  # Check for each class
                        category = {
                            (1, 1): "TP",  # Predicted class i, true class i
                            (0, 0): "TN",  # Predicted not i, true not i
                            (1, 0): "FP",  # Predicted i, true not i
                            (0, 1): "FN"   # Predicted not i, true i
                        }[(1 if pred_cls == i else 0, 1 if cls == i else 0)]
                        if sample_images[cls][category] is None:
                            sample_images[cls][category] = (img, true)
                    if all(sample_images[c][cat] is not None for c in range(5) for cat in ["TP", "TN", "FP", "FN"]):
                        break
                if all(sample_images[c][cat] is not None for c in range(5) for cat in ["TP", "TN", "FP", "FN"]):
                    break

        # Visualization
        fig, axes = plt.subplots(5, 4, figsize=(30, 35))  # Large figure for clarity
        for i in range(5):  # Classes
            for j, category in enumerate(["TP", "TN", "FP", "FN"]):
                ax = axes[i, j]
                if sample_images[i][category] is not None:
                    img, true = sample_images[i][category]
                    img = img.squeeze().cpu()
                    lrp_img, _ = xai_visualizer.layer_wise_relevance_propagation(model, img, i)
                    img = img.permute(1, 2, 0).numpy()
                    img = np.clip(img, 0, 1)
                    img = img[..., [2, 1, 0]] if img.shape[2] == 3 else img  # Convert BGR to RGB
                    img = (img * 255).astype(np.uint8)
                    ax.imshow(img)
                    lrp_img = (lrp_img - lrp_img.min()) / (lrp_img.max() - lrp_img.min() + 1e-8)  # Normalize heatmap
                    ax.imshow(lrp_img, cmap='jet', alpha=0.4, vmin=0, vmax=np.percentile(lrp_img, 95))
                    ax.set_title(f'{classes[i]} - {category}: {metrics[j][i]}', fontsize=16, fontweight='bold', pad=10)
                ax.axis('off')

        plt.tight_layout()
        save_path = f"{self.viz_dir}/lrp_fish_metrics_grid.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"LRP grid visualization saved: {save_path}")




    def plot_comparative_xai(self, single_models, ensemble_results, test_loader, max_images=5):
        """Generate comparative XAI visualizations with Grad-CAM++, Integrated Gradients, and LRP in the same row"""
        print(f"Generating comparative XAI visualizations for {max_images} images...")
        xai_visualizer = XAIVisualizer()

        # Get sample images
        # sample_images, sample_labels = next(iter(test_loader))
        # sample_images = sample_images[:max_images].to(Config.DEVICE)
        # sample_labels = sample_labels[:max_images].numpy()
        sample_images = []
        sample_labels = []
        for i in range(Config.NUM_CLASSES):
            class_mask = np.where(test_loader.dataset.labels == i)[0]
            if len(class_mask) > 0:
                idx = np.random.choice(class_mask)
                image, label = test_loader.dataset[idx]
                if image.dim() == 4:
                    image = image.squeeze(0)
                sample_images.append(image.to(Config.DEVICE))
                sample_labels.append(label)
        sample_images = sample_images[:max_images]
        sample_labels = sample_labels[:max_images]

        # Get best ensemble if available
        best_ensemble_name = max(ensemble_results.items(), key=lambda x: x[1]['f1'])[0] if ensemble_results else None
        best_ensemble = ensemble_results.get(best_ensemble_name, None) if best_ensemble_name else None

        for idx, (image, true_label) in enumerate(zip(sample_images, sample_labels)):
            try:
                # Adjusted figure size with more height per row to prevent text cropping
                num_rows = len(single_models) + (1 if best_ensemble else 0)
                fig = plt.figure(figsize=(36, 14 * num_rows))  # Increased width and height per row
                gs = fig.add_gridspec(num_rows, 4, wspace=0.3, hspace=0.5)  # Increased wspace and hspace for better spacing

                # Prepare original image
                image_np = image.detach().permute(1, 2, 0).cpu().numpy()
                image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                image_np = np.clip(image_np, 0, 1)
                image_np = (image_np * 255).astype(np.uint8)

                # Plot original image spanning all rows in first column
                ax_orig = fig.add_subplot(gs[:, 0])
                ax_orig.imshow(image_np)
                ax_orig.set_title(f'Original Image\nTrue: {Config.CLASS_LABELS[true_label]}',
                                fontsize=16, fontweight='bold', pad=30)  # Increased pad for title
                ax_orig.axis('off')

                # Plot XAI for single models
                for row, (model_name, model) in enumerate(single_models.items()):
                    model.eval()
                    with torch.no_grad():
                        outputs = model(image.unsqueeze(0))
                        probabilities = torch.softmax(outputs, dim=1)
                        predicted_class = outputs.argmax(dim=1).item()
                        confidence = probabilities[0, predicted_class].item()
                        predicted_label = Config.CLASS_LABELS[predicted_class]

                    # Generate Grad-CAM++ visualization
                    gradcam_img, _ = xai_visualizer.grad_cam_plus_plus(model, image, predicted_class)

                    # Plot Grad-CAM++ in second column
                    ax_gradcam = fig.add_subplot(gs[row, 1])
                    ax_gradcam.imshow(gradcam_img)
                    ax_gradcam.set_title(f'{model_name}\nGrad-CAM++\nPred: {predicted_label}\nConf: {confidence:.2%}',
                                        fontsize=14, fontweight='bold', pad=30)  # Increased pad
                    ax_gradcam.axis('off')

                    # Generate Integrated Gradients visualization
                    ig_img, _ = xai_visualizer.integrated_gradients(model, image, predicted_class)

                    # Plot Integrated Gradients in third column
                    ax_ig = fig.add_subplot(gs[row, 2])
                    ax_ig.imshow(ig_img)
                    ax_ig.set_title(f'{model_name}\nIntegrated Gradients', fontsize=14, fontweight='bold', pad=30)
                    ax_ig.axis('off')

                    # Generate LRP visualization
                    lrp_img, _ = xai_visualizer.layer_wise_relevance_propagation(model, image, predicted_class)

                    # Plot LRP in fourth column
                    ax_lrp = fig.add_subplot(gs[row, 3])
                    # ax_lrp.imshow(lrp_img)
                    ax_lrp.imshow(cv2.cvtColor(lrp_img, cv2.COLOR_BGR2RGB))
                    ax_lrp.set_title(f'{model_name}\nLRP', fontsize=14, fontweight='bold', pad=30)
                    ax_lrp.axis('off')

                # Plot XAI for best ensemble (if available)
                if best_ensemble:
                    ensemble_models = best_ensemble['models']
                    with torch.no_grad():
                        model_probs = []
                        for model_name in ensemble_models:
                            model = single_models[model_name]
                            outputs = model(image.unsqueeze(0))
                            probs = torch.softmax(outputs, dim=1)
                            model_probs.append(probs)
                        model_probs = torch.stack(model_probs, dim=1)

                        # Load learnable ensemble model if applicable
                        if 'learnable_weighted' in best_ensemble_name:
                            ensemble_model = LearnableWeightedEnsemble(
                                num_models=len(ensemble_models),
                                num_classes=Config.NUM_CLASSES
                            ).to(Config.DEVICE)
                            # ensemble_model.load_state_dict(
                            #     torch.load(f"{Config.OUTPUT_DIR}/models/learnable_ensemble_{'+'.join(ensemble_models)}.pt")
                            # )
                            # PROBLEM: This line tries to load a model that may not exist
                            # FIX: Add error handling and fallback
                            try:
                                model_path = f"{Config.OUTPUT_DIR}/models/learnable_ensemble_{'+'.join(ensemble_models)}.pt"
                                if os.path.exists(model_path):
                                    ensemble_model.load_state_dict(torch.load(model_path, map_location=Config.DEVICE))
                                else:
                                    # Fallback to simple weighted average if no saved learnable model
                                    weights = best_ensemble.get('weights', np.ones(len(ensemble_models)) / len(ensemble_models))
                                    outputs = torch.sum(model_probs * torch.tensor(weights, device=Config.DEVICE).view(1, -1, 1), dim=1)
                                    predicted_class = outputs.argmax(dim=1).item()
                                    confidence = torch.softmax(outputs, dim=1)[0, predicted_class].item()
                            except Exception as e:
                                print(f"Error loading learnable ensemble model: {e}, using simple averaging")
                                weights = best_ensemble.get('weights', np.ones(len(ensemble_models)) / len(ensemble_models))
                                outputs = torch.sum(model_probs * torch.tensor(weights, device=Config.DEVICE).view(1, -1, 1), dim=1)
                                predicted_class = outputs.argmax(dim=1).item()
                                confidence = torch.softmax(outputs, dim=1)[0, predicted_class].item()




                            ensemble_model.eval()
                            # outputs, _ = ensemble_model(model_probs)
                            # TO THIS (ensure correct input format):
                            if model_probs.dim() == 3:  # Already correct format
                                outputs, learned_weights = ensemble_model(model_probs)
                            else:  # Need to reshape
                                model_probs_reshaped = model_probs.view(1, len(ensemble_models), Config.NUM_CLASSES)
                                outputs, learned_weights = ensemble_model(model_probs_reshaped)

                        else:
                            weights = best_ensemble.get('weights', np.ones(len(ensemble_models)) / len(ensemble_models))
                            outputs = torch.sum(model_probs * torch.tensor(weights, device=Config.DEVICE).view(1, -1, 1), dim=1)

                        predicted_class = outputs.argmax(dim=1).item()
                        confidence = torch.softmax(outputs, dim=1)[0, predicted_class].item()
                        predicted_label = Config.CLASS_LABELS[predicted_class]

                    # Approximate ensemble Grad-CAM++ by averaging
                    gradcam_imgs = []
                    for model_name in ensemble_models:
                        gradcam_img, _ = xai_visualizer.grad_cam_plus_plus(single_models[model_name], image, predicted_class)
                        gradcam_imgs.append(gradcam_img)
                    ensemble_gradcam = np.mean(gradcam_imgs, axis=0).astype(np.uint8)

                    # Plot ensemble Grad-CAM++ in second column
                    ax_gradcam = fig.add_subplot(gs[num_rows-1, 1])
                    ax_gradcam.imshow(ensemble_gradcam)
                    ax_gradcam.set_title(f'Best Ensemble ({best_ensemble_name.split("_")[-1]})\nGrad-CAM++\nPred: {predicted_label}\nConf: {confidence:.2%}',
                                        fontsize=14, fontweight='bold', pad=30)
                    ax_gradcam.axis('off')

                    # Approximate ensemble Integrated Gradients by averaging
                    ig_imgs = []
                    for model_name in ensemble_models:
                        ig_img, _ = xai_visualizer.integrated_gradients(single_models[model_name], image, predicted_class)
                        ig_imgs.append(ig_img)
                    ensemble_ig = np.mean(ig_imgs, axis=0).astype(np.uint8)

                    # Plot ensemble Integrated Gradients in third column
                    ax_ig = fig.add_subplot(gs[num_rows-1, 2])
                    ax_ig.imshow(ensemble_ig)
                    ax_ig.set_title(f'Best Ensemble ({best_ensemble_name.split("_")[-1]})\nIntegrated Gradients', fontsize=14, fontweight='bold', pad=30)
                    ax_ig.axis('off')

                    # Approximate ensemble LRP by averaging
                    lrp_imgs = []
                    for model_name in ensemble_models:
                        lrp_img, _ = xai_visualizer.layer_wise_relevance_propagation(single_models[model_name], image, predicted_class)
                        lrp_imgs.append(lrp_img)
                    ensemble_lrp = np.mean(lrp_imgs, axis=0).astype(np.uint8)

                    # Plot ensemble LRP in fourth column
                    ax_lrp = fig.add_subplot(gs[num_rows-1, 3])
                    ax_lrp.imshow(ensemble_lrp)
                    ax_lrp.set_title(f'Best Ensemble ({best_ensemble_name.split("_")[-1]})\nLRP', fontsize=14, fontweight='bold', pad=30)
                    ax_lrp.axis('off')

                # Adjusted suptitle with increased padding
                plt.suptitle(f'Comparative XAI Analysis - Image {idx+1}', fontsize=18, fontweight='bold', y=0.97)
                plt.tight_layout(rect=[0, 0, 1, 0.95])  # Added rect to ensure suptitle is not cropped
                save_path = f"{self.viz_dir}/comparative_xai_image_{idx+1}.png"
                plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
                plt.show()
                print(f"Comparative XAI visualization saved: {save_path}")

            except Exception as e:
                print(f"Error generating comparative XAI for image {idx+1}: {e}")
                continue



# ---
# 8. FULLY FIXED XAI VISUALIZATIONS
# ================================================================================================================================
# Purpose: Implement corrected Grad-CAM++ and LRP with proper tensor handling.

class XAIVisualizer:
    @staticmethod
    def grad_cam_plus_plus(model, image, target_class):
        """Implement Grad-CAM++ for visualizing important image regions."""
        if image.dim() == 4:
            image = image.squeeze(0)  # Normalize to (C, H, W)

        model.eval()

        # Find the last convolutional layer
        target_layer = None
        for name, module in model.named_modules():  # Corrected: removed redundant reversed
            if isinstance(module, nn.Conv2d):
                target_layer = module

        if target_layer is None:
            raise ValueError("No convolutional layer found in the model")

        # Hook to capture gradients and activations
        gradients = []
        activations = []

        def forward_hook(module, input, output):
            activations.append(output.detach())

        def backward_hook(module, grad_input, grad_output):
            gradients.append(grad_output[0].detach())

        # Register hooks
        forward_handle = target_layer.register_forward_hook(forward_hook)
        backward_handle = target_layer.register_backward_hook(backward_hook)

        try:
            # Forward pass
            image_tensor = image.unsqueeze(0).requires_grad_(True)
            output = model(image_tensor)

            # Backward pass
            model.zero_grad()
            score = output[0, target_class]
            score.backward()

            # Get gradients and activations
            if not gradients or not activations:
                raise ValueError("No gradients or activations captured")

            grad = gradients[0]  # Shape: (1, C, H, W)
            act = activations[0]  # Shape: (1, C, H, W)

            # Grad-CAM++ calculations
            alpha_num = grad.pow(2)
            alpha_denom = 2.0 * grad.pow(2) + (act * grad.pow(3)).sum(dim=(2, 3), keepdim=True)
            alpha_denom = torch.where(alpha_denom != 0, alpha_denom, torch.ones_like(alpha_denom))
            alpha = alpha_num / alpha_denom

            weights = (alpha * F.relu(grad)).sum(dim=(2, 3))  # Shape: (1, C)
            cam = (weights.unsqueeze(-1).unsqueeze(-1) * act).sum(dim=1, keepdim=True)  # Shape: (1, 1, H, W)
            cam = F.relu(cam)

            # Normalize
            cam = cam - cam.min()
            cam = cam / (cam.max() + 1e-8)

            # Resize to input image size
            cam = F.interpolate(cam, size=(224, 224), mode='bilinear', align_corners=False)
            cam = cam.squeeze().detach().cpu().numpy()

            # Apply colormap
            cam = (cam * 255).astype(np.uint8)
            heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)

            # Convert original image for visualization
            image_np = image.detach().permute(1, 2, 0).cpu().numpy()
            image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
            image_np = np.clip(image_np, 0, 1)
            image_np = (image_np * 255).astype(np.uint8)

            # Superimpose heatmap
            superimposed_img = cv2.addWeighted(image_np, 0.6, heatmap, 0.4, 0)
            # superimposed_img = cv2.addWeighted(image_np[:, :, ::-1], 0.6, heatmap, 0.4, 0)  # Convert image_np to BGR for cv2
            # superimposed_img = superimposed_img[:, :, ::-1]  # Back to RGB for display

        finally:
            # Remove hooks
            forward_handle.remove()
            backward_handle.remove()

        return superimposed_img, heatmap




    @staticmethod
    def integrated_gradients(model, image, target_class):
        """Integrated Gradients implementation."""
        if image.dim() == 4:
            image = image.squeeze(0)  # Normalize to (C, H, W)


        model.eval()

        # Detach the image first to avoid gradient issues
        image_detached = image.detach()

        # Create baseline (black image)
        baseline = torch.zeros_like(image_detached)

        # Generate path from baseline to image
        num_steps = 30
        alphas = torch.linspace(0, 1, num_steps).to(image_detached.device)

        gradients = []
        for alpha in alphas:
            interpolated = baseline + alpha * (image_detached - baseline)
            interpolated = interpolated.requires_grad_(True)

            output = model(interpolated.unsqueeze(0))
            score = output[0, target_class]

            # Clear gradients
            model.zero_grad()
            if interpolated.grad is not None:
                interpolated.grad.zero_()

            score.backward()

            # Detach gradient before storing
            if interpolated.grad is not None:
                gradients.append(interpolated.grad.detach().clone())

            # Clear the gradient to free memory
            interpolated.grad = None

        if not gradients:
            # Fallback: simple gradient
            image_grad = image_detached.requires_grad_(True)
            output = model(image_grad.unsqueeze(0))
            score = output[0, target_class]
            model.zero_grad()
            score.backward()
            gradients = [image_grad.grad.detach().clone()]

        # Average gradients
        avg_gradients = torch.stack(gradients).mean(dim=0)

        # Compute integrated gradients
        integrated_gradients = (image_detached - baseline) * avg_gradients

        # Sum across color channels
        relevance = integrated_gradients.abs().sum(dim=0).cpu().numpy()

        # Normalize
        relevance = relevance / (relevance.max() + 1e-8)

        # Apply colormap
        relevance = (relevance * 255).astype(np.uint8)
        heatmap = cv2.applyColorMap(relevance, cv2.COLORMAP_JET)

        # Convert original image for visualization
        image_np = image_detached.permute(1, 2, 0).cpu().numpy()
        image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        image_np = np.clip(image_np, 0, 1)
        image_np = (image_np * 255).astype(np.uint8)

        # Superimpose heatmap
        superimposed_img = cv2.addWeighted(image_np, 0.6, heatmap, 0.4, 0)

        return superimposed_img, heatmap





    @staticmethod
    def layer_wise_relevance_propagation(
        model,
        image,
        target_class=None,
        device="cuda",
        input_size=None,
        epsilon=1e-6,
        imagenet_norm=True
    ):
        """
        Layer-wise Relevance Propagation (LRP) with Captum and gradient fallback.

        Args:
            model: torch.nn.Module - trained model
            image: torch.Tensor - input image (C, H, W) or (1, C, H, W)
            target_class: int or None - class index, if None will be predicted
            device: str - device ("cuda" or "cpu")
            input_size: int or None - resize for visualization
            epsilon: float - small value to avoid division by zero
            imagenet_norm: bool - whether to denormalize using ImageNet mean/std

        Returns:
            superimposed_img (np.ndarray), heatmap (np.ndarray)
        """
        import torch
        import numpy as np
        import cv2

        model.eval()
        model.to(device)

        # Ensure correct input shape
        if image.dim() == 4:
            image = image.squeeze(0)
        if image.dim() != 3:
            raise ValueError(f"Expected 3D image tensor (C,H,W), got {image.shape}")

        image_tensor = image.unsqueeze(0).to(device).requires_grad_(True)

        # Determine target class
        if target_class is None:
            with torch.no_grad():
                output = model(image_tensor)
                target_class = output.argmax(dim=1).item()

        try:
            from captum.attr import LRP

            lrp = LRP(model)
            attributions = lrp.attribute(image_tensor, target=target_class)
            if attributions is None:
                raise ValueError("LRP attribution returned None")

            relevance = attributions.detach().cpu().squeeze(0)  # (C,H,W)
            if relevance.dim() == 3:
                relevance = relevance.sum(dim=0)  # (H,W)

        except Exception as e:
            # Fallback: Gradient-based relevance
            try:
                model.zero_grad()
                image_tensor = image_tensor.detach().requires_grad_(True)
                output = model(image_tensor)

                target_score = output[0, target_class]
                target_score.backward()

                if image_tensor.grad is None:
                    raise ValueError("Gradients are None in fallback.")

                relevance = image_tensor.grad.detach().cpu().squeeze(0)
                if relevance.dim() == 3:
                    relevance = relevance.clamp(min=0).sum(dim=0)

            except Exception as fallback_e:
                print(f"LRP + fallback failed: {fallback_e}")
                # Return original image and blank heatmap
                image_np = image.permute(1, 2, 0).cpu().numpy()
                image_np = np.clip(image_np, 0, 1)
                image_np = (image_np * 255).astype(np.uint8)
                heatmap = np.zeros_like(image_np, dtype=np.uint8)
                return image_np, heatmap

        # Normalize relevance
        relevance = relevance.clamp(min=0)
        if relevance.max() > 0:
            relevance = relevance / (relevance.max() + epsilon)

        relevance_map = relevance.numpy()
        if input_size is not None and relevance_map.shape != (input_size, input_size):
            relevance_map = cv2.resize(relevance_map, (input_size, input_size))

        relevance_map = (relevance_map * 255).astype(np.uint8)
        heatmap = cv2.applyColorMap(relevance_map, cv2.COLORMAP_JET)

        # Prepare original image
        image_np = image_tensor.detach().squeeze(0).permute(1, 2, 0).cpu().numpy()

        if imagenet_norm:
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            image_np = image_np * std + mean

        image_np = np.clip(image_np, 0, 1)
        image_np = (image_np * 255).astype(np.uint8)

        # Superimpose heatmap
        superimposed_img = cv2.addWeighted(image_np, 0.7, heatmap, 0.3, 0.0)

        return superimposed_img, heatmap



# ---
# 9. DATA LOADING AND PREPROCESSING
# ================================================================================================================================
# Purpose: Handle dataset creation, data loading, balancing with SMOTE, and data augmentation.


class FishDataset(Dataset):
    def __init__(self, images, labels, transform=None):

        self.images = self._preprocess_images(images)
        self.labels = labels.astype(np.int64)
        self.transform = transform #Here means: Medium,Heavy or Any

    def _preprocess_images(self, images):
        """Preprocess images to ensure proper format and normalization"""
        if images.max() > 1.5: #👉 The threshold 1.5 is just a safe cutoff to distinguish between the two cases.
            #Because some normalized images can have values slightly above 1.0 (e.g., after augmentations, rounding, or scaling bugs).
            images = images.astype(np.float32) / 255.0

        if len(images.shape) == 4 and images.shape[1] == 3: #If input is (batch, channels, height, width) → convert to (batch, height, width, channels) (common for TensorFlow).
            images = np.transpose(images, (0, 2, 3, 1))
        return images.astype(np.float32)

    def __len__(self):
        """Return the total number of samples in the dataset"""
        return len(self.images)




    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:  #Applies an Albumentations transform pipeline (it returns a dict, so you take ['image']).
            image = self.transform(image=image)['image'] #
        else:
            image = torch.from_numpy(image).permute(2, 0, 1)
        #With transform → advanced augmentations.
        #Without transform → just convert to PyTorch format.


        # Convert label to plain Python int to avoid CUDA tensor creation in workers.That wastes memory and slows down training.
        if isinstance(label, np.ndarray):
            label = int(label.item())
        elif hasattr(label, 'item'):
            label = int(label.item())
        else:
            label = int(label)


        return image, label  # Plain Python int, not torch.tensor
        # return image, torch.tensor(int(label), dtype=torch.long)




    # def __getitem__(self, idx):
    #     image = self.images[idx]  # H x W x C
    #     label = self.labels[idx]

    #     # Ensure image has 3 channels
    #     if image.ndim == 2:  # grayscale H x W
    #         image = np.stack([image]*3, axis=-1)
    #     elif image.shape[-1] == 4:  # RGBA
    #         image = image[:, :, :3]

    #     # Apply Albumentations transform if any
    #     if self.transform:
    #         image = self.transform(image=image)['image']  # may already be tensor

    #     # Convert to PyTorch tensor C x H x W if it's a numpy array
    #     if isinstance(image, np.ndarray):
    #         image = torch.from_numpy(image).permute(2, 0, 1).float()
    #     elif isinstance(image, torch.Tensor) and image.ndim == 3 and image.shape[0] != 3:
    #         # If transform returns H x W x C tensor, permute to C x H x W
    #         image = image.permute(2, 0, 1).float()
    #     # else assume it's already C x H x W

    #     # Convert label to tensor
    #     label = int(label) if not isinstance(label, torch.Tensor) else label.long()

    #     return image, torch.tensor(label, dtype=torch.long)




        # class MyClass:
        #     def greet(self):
        #         print("Hello!")
        # obj = MyClass()
        # print(hasattr(obj, 'greet'))   # True, because obj has a method greet
        # print(hasattr(obj, 'name'))    # False, no attribute called name
        # # Using hasattr with .item()
        # import torch
        # x = torch.tensor(5)  # scalar tensor
        # print(hasattr(x, 'item'))      # True
        # print(x.item())                # 5

        # return image, torch.tensor(label, dtype=torch.long)  # <-- ensure label is tensor




class DataManager:
    @staticmethod  #In Python, @staticmethod is used to define a method that belongs to a class but doesn’t access self or cls.

    # class DataManager:
    # staticmethod
    # def greet(name):
    #     return f"Hello, {name}!"
    # # Call without creating an instance
    # print(DataManager.greet("Imran"))  # Output: Hello, Imran!
    # # Call with an instance
    # dm = DataManager()
    # print(dm.greet("Imran"))           # Output: Hello, Imran!

    # class MyClass:
    #     count = 0

    #     staticmethod
    #     def greet(name):
    #         return f"Hello, {name}!"

    #     classmethod
    #     def increment_count(cls):
    #         cls.count += 1
    #         return cls.count

    # # Static method
    # print(MyClass.greet("Imran"))      # Hello, Imran!
    # # Class method
    # print(MyClass.increment_count())   # 1
    # print(MyClass.increment_count())   # 2
    #Static method → independent of class/instance.
    #Class method → works with the class itself (cls), can modify class variables.


    def get_transforms(is_training=True, augmentation_strength='medium'):
        """Get data transforms with configurable augmentation strength"""
        if is_training:
            if augmentation_strength == 'light':
                return A.Compose([
                    A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                    A.HorizontalFlip(p=0.3),
                    A.RandomRotate90(p=0.3),
                    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
                    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ToTensorV2()
                ])
            elif augmentation_strength == 'heavy':
                return A.Compose([
                    A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                    A.HorizontalFlip(p=0.7),
                    A.VerticalFlip(p=0.5),
                    A.RandomRotate90(p=0.7),
                    # A.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.3, rotate_limit=45, p=0.8),
                    # A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=0.8),
                    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
                    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),
                    # A.HueSaturationValue(hue_shift_limit=30, sat_shift_limit=40, val_shift_limit=30, p=0.7),
                    A.GaussianBlur(blur_limit=(3, 9), p=0.5),
                    A.GaussNoise(var_limit=(10.0, 80.0), p=0.4),
                    A.CoarseDropout(max_holes=12, max_height=25, max_width=25, p=0.5),
                    A.ElasticTransform(p=0.3),
                    A.GridDistortion(p=0.3),
                    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

                    A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3, p=0.3),
                    A.RandomRain(blur_value=3, p=0.2),
                    A.ColorJitter(hue=0.1, p=0.5),

                    ToTensorV2()
                ])
            else:  # medium
                return A.Compose([
                    A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                    A.HorizontalFlip(p=0.5),
                    A.VerticalFlip(p=0.3),
                    A.RandomRotate90(p=0.5),
                    # A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.7),
                    # A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
                    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.1),
                    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),
                    # A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.6),
                    A.GaussianBlur(blur_limit=(3, 7), p=0.4),
                    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
                    A.CoarseDropout(max_holes=8, max_height=20, max_width=20, p=0.4),
                    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ToTensorV2()
                ])
        else:
            return A.Compose([
                A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])


    @staticmethod
    def load_and_balance_data():
        """Load data and apply SMOTE"""
        print("Loading and preprocessing data...")

        # # Check GPU availability
        # print("GPU Available:", torch.cuda.is_available())
        # print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


        # # Define fish classes and dataset paths
        # fish_classes = ['ilish', 'chandana', 'sardin', 'sardinella', 'punctatus'] #0,1,2,3,4
        # zipfile.ZipFile('/content/drive/MyDrive/Hilsha/data_fish_org_8407.zip').extractall('/content/.hidden_fish')
        # data_dir = '/content/.hidden_fish'

        # image_limits = {
        #     'ilish': 3000,
        #     'chandana': 1185,
        #     'sardin': 2899,
        #     'sardinella': 370,
        #     'punctatus': 953
        # }

        # # Settings
        # total_images = sum(image_limits.values())
        # batch_size = 100
        # num_threads = 4


        # # Output paths
        # output_dir = '/content/drive/MyDrive/Hilsha'
        # os.makedirs(output_dir, exist_ok=True)
        # labels_file = os.path.join(output_dir, 'Y_labels.npy')
        # xdata_file = os.path.join(output_dir, 'X_data.npy')

        # save_lock = threading.Lock()  # for thread-safe writes -> Prevents race conditions when multiple threads write to the same list.

        # # Function to gather image paths
        # def get_image_paths(class_name, max_images):
        #     path = os.path.join(data_dir, class_name)
        #     files = sorted(os.listdir(path))
        #     random.shuffle(files)
        #     return [os.path.join(path, f) for f in files[:max_images]]

        # # Load and preprocess batch
        # def load_and_preprocess_batch(image_paths, start_idx, batch_size, class_idx):
        #     end_idx = min(start_idx + batch_size, len(image_paths))
        #     batch_paths = image_paths[start_idx:end_idx]
        #     batch_images = []

        #     for img_path in batch_paths:
        #         img = Image.open(img_path).resize((224, 224)).convert('RGB')
        #         img_tensor = torch.tensor(np.array(img), dtype=torch.uint8).permute(2, 0, 1)  # C x H x W
        #         batch_images.append(img_tensor)

        #     batch_tensor = torch.stack(batch_images)  # B x C x H x W
        #     batch_labels = np.full((len(batch_images),), class_idx, dtype=np.int32)
        #     return batch_tensor, batch_labels

        # # Process one batch and return tensors & labels (no file saving)
        # def process_batch(image_paths, start_idx, batch_size, class_idx):
        #     return load_and_preprocess_batch(image_paths, start_idx, batch_size, class_idx)

        # def preprocess_and_save_all(overwrite=True):
        #     if os.path.exists(labels_file) and os.path.exists(xdata_file) and not overwrite:
        #         print("Preprocessed data already exists. Set overwrite=True to reprocess.")
        #         return

        #     all_images = []
        #     all_labels = []
        #     processed_count = 0

        #     for idx, class_name in enumerate(fish_classes):
        #         print(f"\nProcessing class: {class_name}")
        #         image_paths = get_image_paths(class_name, image_limits[class_name])
        #         total_batches = (len(image_paths) + batch_size - 1) // batch_size
        #         #It ensures ceiling division — rounding up, not down.
        #         # Normal division: 103 / 20 = 5.15 → floor division // 20 = 5 (❌ missing last 3 images)
        #         # This trick: (103 + 20 - 1) // 20 = 122 // 20 = 6 ✅

        #         with ThreadPoolExecutor(max_workers=num_threads) as executor:
        #             futures = []
        #             for start in range(0, len(image_paths), batch_size):
        #                 futures.append(executor.submit(process_batch, image_paths, start, batch_size, idx))

        #             for future in tqdm(as_completed(futures), total=total_batches, desc=class_name):#taqaddum (تقدّم) – Arabic for "progress".
        #                 # futures: List of tasks (from ThreadPoolExecutor or ProcessPoolExecutor).
        #                 # as_completed(futures): Yields each future as it finishes (not in order).

        #                 batch_tensor, batch_labels = future.result()
        #                 with save_lock: #Locks this section so that only one thread can update the shared lists safely.
        #                     all_images.append(batch_tensor)
        #                     all_labels.append(batch_labels)
        #                     processed_count += batch_tensor.size(0)
        #                     print(f"Processed batch with {batch_tensor.size(0)} images, total processed: {processed_count}/{total_images}")
        #                 gc.collect()

        #     # Combine all tensors and labels
        #     X = torch.cat(all_images, dim=0).numpy()
        #     Y = np.concatenate(all_labels, axis=0)

        #     # Save final arrays
        #     np.save(xdata_file, X, allow_pickle=False)#Malicious .npy -> import os;os.system("rm -rf /")  # ← Dangerous command
        #     np.save(labels_file, Y, allow_pickle=False)

        #     print(f"\n✅ Done! Saved {processed_count} images in {xdata_file}")
        #     print(f"X_data shape: {X.shape}, Y_labels shape: {Y.shape}")

        #     if processed_count != total_images:
        #         raise ValueError(f"Expected {total_images} images, but processed {processed_count}")

        # # Run preprocessing and save directly to X_data.npy and Y_labels.npy
        # preprocess_and_save_all(overwrite=True)







        X = np.load(Config.DATA_FILE)
        Y = np.load(Config.LABELS_FILE)







        # # Your data path
        # output_dir = '/content/drive/MyDrive/Hilsha'
        # data_file = os.path.join(output_dir, 'X_data.npy')
        # labels_file = os.path.join(output_dir, 'Y_labels.npy')

        # # Readable size format
        # def sizeof_fmt(num, suffix='B'):
        #     for unit in ['', 'K', 'M', 'G', 'T']:
        #         if abs(num) < 1024.0:
        #             return f"{num:3.2f} {unit}{suffix}"
        #         num /= 1024.0
        #     return f"{num:.2f} T{suffix}"

        # # Main loader
        # def load_preprocessed_data(as_torch=True, normalize=True, to_device=None):
        #     # Check file existence #cpu,cuda (CUDA stands for Compute Unified Device Architecture.)
        #     for path in [data_file, labels_file]:
        #         if not os.path.exists(path):
        #             raise FileNotFoundError(f"Missing: {path}")

        #     # Print file sizes
        #     print(f"📁 X_data.npy: {sizeof_fmt(os.path.getsize(data_file))}")
        #     print(f"📁 Y_labels.npy: {sizeof_fmt(os.path.getsize(labels_file))}")

        #     # Load with mmap
        #     X = np.load(data_file, mmap_mode='r')
        #     Y = np.load(labels_file, mmap_mode='r')

        #     print(f"✅ X shape: {X.shape}, dtype: {X.dtype}")
        #     print(f"✅ Y shape: {Y.shape}, dtype: {Y.dtype}")

        #     # Sanity check
        #     if len(X) != len(Y):
        #         raise ValueError("Mismatch between number of samples in X and Y")

        #     # Convert to torch
        #     if as_torch:
        #         X = torch.from_numpy(X)
        #         Y = torch.from_numpy(Y)

        #         if normalize and X.dtype == torch.uint8:
        #             X = X.float() / 255.0

        #         if to_device:
        #             X = X.to(to_device)
        #             Y = Y.to(to_device)

        #         print(f"🧠 Torch tensors ready on {to_device or 'CPU'}")

        #     return X, Y

        # # 🔁 Example call
        # X, Y = load_preprocessed_data(
        #     as_torch=True,
        #     normalize=True,
        #     to_device='cuda' if torch.cuda.is_available() else 'cpu'
        # )

        # print(f"\nOriginal data shape: {X.shape}")
        # # print(f"Original class distribution: {np.bincount(Y)}")
        # class_dist = np.bincount(Y.cpu().numpy()) if torch.is_tensor(Y) else np.bincount(Y)
        # print(f"Original class distribution: {class_dist}")






        # print("Applying SMOTE for class balancing...")
        # X_flat = X.reshape(X.shape[0], -1)
        # smote = SMOTE(random_state=Config.SEED, k_neighbors=min(5, np.bincount(Y).min()-1))
        # X_balanced_flat, Y_balanced = smote.fit_resample(X_flat, Y)
        # X_balanced = X_balanced_flat.reshape(-1, *X.shape[1:])
        # print(f"Balanced data shape: {X_balanced.shape}")
        # print(f"Balanced class distribution: {np.bincount(Y_balanced)}")
        # return X_balanced, Y_balanced



        # Remove SMOTE completely and use WeightedRandomSampler only
        # Using WeightedRandomSampler instead of SMOTE
        # Compute weights and create sampler during DataLoader, not here
        # return X, Y
        # # Example data
        # X = torch.randn(100, 3, 32, 32)  # 100 images
        # Y = torch.randint(0, 5, (100,))  # 5 classes, imbalanced
        # # Compute class weights
        # class_counts = torch.bincount(Y)
        # class_weights = 1.0 / class_counts.float()
        # sample_weights = class_weights[Y]  # assign weight to each sample
        # # Create sampler
        # sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
        # # Create DataLoader
        # dataset = TensorDataset(X, Y)
        # loader = DataLoader(dataset, batch_size=16, sampler=sampler)



        print("Applying SMOTE for class balancing...")
        # Apply SMOTE with reduced k_neighbors and combine with WeightedRandomSampler
        X_flat = X.reshape(X.shape[0], -1)
        smote = SMOTE(random_state=Config.SEED, k_neighbors=3, sampling_strategy='not majority')
        # smote = SMOTE(random_state=Config.SEED, k_neighbors=2, sampling_strategy= 'auto')
        X_balanced_flat, Y_balanced = smote.fit_resample(X_flat, Y)
        X_balanced = X_balanced_flat.reshape(-1, *X.shape[1:])
        # Ensures WeightedRandomSampler is still used in DataLoader
        print(f"Balanced data shape: {X_balanced.shape}")
        print(f"Balanced class distribution: {np.bincount(Y_balanced)}")
        return X_balanced, Y_balanced
        # Benefit: Using a smaller k_neighbors=3 reduces the risk of generating unnatural
        # image artifacts, while sampling_strategy='not majority' balances classes more conservatively.
        # Retaining WeightedRandomSampler in the DataLoader further ensures balanced sampling during
        # training, maintaining smoothness and preventing accuracy drops by avoiding over-reliance
        # on SMOTE-generated samples.



        # print("Applying SMOTE for class balancing...")

        # X_flat = X.cpu().numpy().reshape(X.shape[0], -1) if torch.is_tensor(X) else X.reshape(X.shape[0], -1)
        # Y_np = Y.cpu().numpy() if torch.is_tensor(Y) else Y

        # smote = SMOTE(random_state=Config.SEED, k_neighbors=3, sampling_strategy='not majority')
        # X_balanced_flat, Y_balanced = smote.fit_resample(X_flat, Y_np)
        # X_balanced = X_balanced_flat.reshape(-1, *X.shape[1:])

        # print(f"Balanced data shape: {X_balanced.shape}")
        # print(f"Balanced class distribution: {np.bincount(Y_balanced)}")
        # return X_balanced, Y_balanced






    @staticmethod
    def create_data_loaders(X, Y, test_size=0.2, batch_size=None, augmentation_strength='medium'):


        X_temp, X_test, y_temp, y_test = train_test_split(X, Y, test_size=test_size, random_state=Config.SEED, stratify=Y)
        X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=Config.SEED, stratify=y_temp)

        print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
        print(f"Using optimized batch size: {batch_size}")



        train_dataset = FishDataset(X_train, y_train,DataManager.get_transforms(True, augmentation_strength))
        val_dataset = FishDataset(X_val, y_val, DataManager.get_transforms(False))
        test_dataset = FishDataset(X_test, y_test, DataManager.get_transforms(False))


        class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        #compute_class_weight('balanced', ...) gives higher weight to minority classes.
        sample_weights = [class_weights[y] for y in y_train]
        sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)
        # Samples with higher weights are more likely to be picked in each batch.
        # replacement=True allows oversampling of minority classes. ✅


        # Conditionally set prefetch_factor based on num_workers
        prefetch_factor = 2 if Config.DATALOADER_NUM_WORKERS > 0 else None
        pin_memory=Config.PIN_MEMORY if 'cuda' in Config.DEVICE else False
        num_workers = Config.DATALOADER_NUM_WORKERS if torch.cuda.is_available() else 0
        use_prefetch = num_workers > 0


        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            # sampler=sampler, #Imbalanced dataset → use sampler.Balanced dataset → use shuffle=True.
            shuffle=True,
            num_workers=num_workers,
            pin_memory=torch.cuda.is_available(),
            prefetch_factor=2 if use_prefetch else None,  # Only use prefetch_factor when num_workers > 0
            # persistent_workers=Config.DATALOADER_NUM_WORKERS > 0,
            # persistent_workers=False,
            worker_init_fn=worker_init_fn  # Add this
        )
        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=torch.cuda.is_available(),
            prefetch_factor=2 if use_prefetch else None,  # Only use prefetch_factor when num_workers > 0
            # persistent_workers=Config.DATALOADER_NUM_WORKERS > 0,
            # persistent_workers=False,
            worker_init_fn=worker_init_fn  # Add this
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=torch.cuda.is_available(),
            prefetch_factor=2 if use_prefetch else None,  # Only use prefetch_factor when num_workers > 0
            # persistent_workers=Config.DATALOADER_NUM_WORKERS > 0,
            # persistent_workers=False, #False is slow but exact reproductivity ensures & workers reset each epoch).
            worker_init_fn=worker_init_fn  # Add this
        )

        return train_loader, val_loader, test_loader, (X_val, y_val), (X_test, y_test)



# ---
# 10. HYPERPARAMETER OPTIMIZATION
# ================================================================================================================================
# Purpose: Tune model hyperparameters using Optuna.
# Add these imports at top of file
import psutil  # ADDED: For CPU memory monitoring
import gc      # ADDED: For garbage collection

class ExpandedHyperparameterOptimizer:
    def __init__(self, model_name, train_loader, val_loader):
        self.model_name = model_name
        self.train_loader = train_loader
        self.val_loader = val_loader

        self.total_batches_per_epoch = len(train_loader)
        self.batch_times = []
        self.best_val_acc = 0.0
        self.best_val_f1 = 0.0
        self.best_trial = 0.0

        # ADDED: Memory management attributes
        # self.memory_check_interval = 40
        # self.force_cleanup_threshold = 95

        # self.val_f1 = 0.0
        # self.history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'val_f1': [], 'learning_rates': []}



    # def _update_progress(self, batch_idx, batch_loss, batch_acc, trial_params, is_training=True):
    #     """Display progress during hyperparameter tuning"""
    #     batch_time = time.time()
    #     self.batch_times.append(batch_time)

    #     if len(self.batch_times) > 1:
    #         avg_batch_time = np.mean(np.diff(self.batch_times[-10:]))
    #     else:
    #         avg_batch_time = 1.0

    #     remaining_batches = self.total_batches_per_epoch - (batch_idx + 1)
    #     eta_epoch = remaining_batches * avg_batch_time

    #     progress_pct = (batch_idx + 1) / self.total_batches_per_epoch * 100
    #     bar_length = 30
    #     filled_length = int(bar_length * (batch_idx + 1) // self.total_batches_per_epoch)
    #     bar = '█' * filled_length + '-' * (bar_length - filled_length)

    #     eta_str = str(timedelta(seconds=int(eta_epoch)))

    #     mode = "TRAIN" if is_training else "VAL  "

    #     # print(f"\r{mode} |{bar}| {progress_pct:5.1f}% | "
    #     #       f"Batch: {batch_idx+1:4d}/{self.total_batches_per_epoch} | "
    #     #       f"Loss: {batch_loss:.4f} | Acc: {batch_acc:.4f} | "
    #     #       f"ETA: {eta_str} | Params: {self._format_params(trial_params)}", end='', flush=True)

    #     import sys
    #     # Clear previous lines if not the first batch
    #     if batch_idx > 0:
    #         print("\033[3A\033[J", end='')  # Move up 3 lines and clear from cursor down
    #     # Line 1: Mode, progress bar, percentage, and batch info
    #     print(f"{mode} |{bar}| {progress_pct:5.1f}% | Batch: {batch_idx+1:4d}/{self.total_batches_per_epoch}")
    #     # Line 2: Loss, Accuracy, and ETA
    #     print(f"Loss: {batch_loss:.4f} | Acc: {batch_acc:.4f} | ETA: {eta_str}")
    #     # Line 3: Parameters
    #     print(f"Params: {self._format_params(trial_params)}")
    #     sys.stdout.flush()



    # def _format_params(self, params):
    #     """Format hyperparameters for display"""
    #     formatted = []
    #     for key, value in params.items():
    #         if key in ['lr', 'weight_decay']:
    #             formatted.append(f"{key}: {value:.4f}")
    #         elif key in ['dropout', 'hidden_dim_multiplier', 'label_smoothing']:
    #             formatted.append(f"{key}: {value:.4f}")
    #         else:
    #             formatted.append(f"{key}: {value}")
    #     return ", ".join(formatted)



    # best_val_f1 = 0.0  # Track best F1 in this trial                        # Added
    # best_val_acc = 0.0  # Track corresponding acc for the best F1           # Added
    # patience_counter = 0  # Local patience for early stopping in trial      # Added



    def objective(self, trial):
        # """Optuna objective function with hyperparameters"""
        # gpu_memory = get_available_gpu_memory()
        # if gpu_memory > 24:
        #     batch_size_options = [32, 64, 96, 128]
        # elif gpu_memory > 12:
        #     batch_size_options = [32, 64, 96]
        # elif gpu_memory > 6:
        #     batch_size_options = [32, 64]
        # else:
        #     batch_size_options = [16, 32]


        lr = trial.suggest_float('lr', 1e-6, 5e-4, log=True)
        batch_size = trial.suggest_categorical("batch_size", [32, 64, 96, 128, 256])
        # batch_size = trial.suggest_categorical('batch_size', batch_size_options)  # Use dynamic options
        weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
        dropout = trial.suggest_float('dropout', 0.1, 0.5)
        hidden_dim_multiplier = trial.suggest_float('hidden_dim_multiplier', 0.25, 1.0)
        augmentation_strength = trial.suggest_categorical('augmentation_strength', ['light', 'medium', 'heavy'])
        optimizer_type = trial.suggest_categorical('optimizer_type', ['adamw'])#, 'adam', 'sgd'])
        scheduler_type = trial.suggest_categorical('scheduler_type',['cosine', 'plateau'])#, 'step', 'exponential'])
        label_smoothing = trial.suggest_float('label_smoothing', 0.05, 0.15)


        trial_params = {
            'lr': lr, 'weight_decay': weight_decay, 'dropout': dropout,
            'hidden_dim_multiplier': hidden_dim_multiplier, 'augmentation_strength': augmentation_strength,
            'batch_size': batch_size, 'optimizer_type': optimizer_type, 'scheduler_type': scheduler_type,
            'label_smoothing': label_smoothing
        }


        try:
            print(f"\nTrial {trial.number+1}/{Config.OPTUNA_TRIALS} parameters for {self.model_name}:")
            for key, value in trial_params.items():
                if key in ['lr', 'weight_decay']:
                    print(f"  {key}: {value:.4f}")
                elif key in ['dropout', 'hidden_dim_multiplier', 'label_smoothing']:
                    print(f"  {key}: {value:.4f}")
                else:
                    print(f"  {key}: {value}")

            # # Conditionally set prefetch_factor based on num_workers
            prefetch_factor = 2 if Config.DATALOADER_NUM_WORKERS > 0 else None

            temp_train_loader = DataLoader(
                self.train_loader.dataset,
                batch_size=batch_size,
                sampler=self.train_loader.sampler,
                num_workers=Config.DATALOADER_NUM_WORKERS,
                pin_memory=Config.PIN_MEMORY,
                prefetch_factor=prefetch_factor,
                persistent_workers=Config.DATALOADER_NUM_WORKERS > 0,
                worker_init_fn=worker_init_fn  # Add this
            )
            temp_val_loader = DataLoader(
                self.val_loader.dataset,
                batch_size=batch_size,
                shuffle=False,
                num_workers=Config.DATALOADER_NUM_WORKERS,
                pin_memory=Config.PIN_MEMORY,
                prefetch_factor=prefetch_factor,
                persistent_workers=Config.DATALOADER_NUM_WORKERS > 0,
                worker_init_fn=worker_init_fn  # Add this
            )
            # temp_train_loader = self.train_loader
            # temp_val_loader = self.val_loader



            model = ModelFactory.create_model(
                self.model_name,
                dropout_rate=dropout,
                hidden_dim_multiplier=hidden_dim_multiplier
            )
            model = model.to(Config.DEVICE)


            if optimizer_type == 'adam':
                optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
            elif optimizer_type == 'adamw':
                optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
            else:
                optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay,momentum=0.9, nesterov=True)

            criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

            if scheduler_type == 'cosine':
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.OPTUNA_EPOCHS)
            elif scheduler_type == 'step':
                scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.7)
            else:
                scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)



            is_best = False
            best_val_f1 = 0.0  # Local variable for trial-specific best F1 score
            best_val_acc = 0.0  # Local variable for trial-specific best accuracy
            patience_counter = 0.0  # Initialize patience counter

            for optuna_epoch in range(Config.OPTUNA_EPOCHS):
                self.batch_times = []
                model.train()

                train_correct = 0
                train_total = 0
                train_loss = 0

                train_pbar = tqdm(temp_train_loader, desc=f"Optuna Epoch {optuna_epoch+1}/{Config.OPTUNA_EPOCHS} Training", leave=False)

                # Corrected training loop in ExpandedHyperparameterOptimizer.objective
                for batch_idx, (images, labels) in enumerate(train_pbar): #train_pbar
                    if images.dim() == 3:
                        images = images.unsqueeze(0)
                    if images.dim() != 4:
                        raise ValueError(f"Expected 4D tensor, got {images.shape}")


                    # # ADDED: Memory monitoring every 40 batches
                    # if batch_idx % self.memory_check_interval == 0:
                    #     try:
                    #         cpu_memory = psutil.virtual_memory()
                    #         cpu_percent = cpu_memory.percent
                    #         if torch.cuda.is_available():
                    #             gpu_percent = (torch.cuda.memory_reserved() / torch.cuda.get_device_properties(0).total_memory) * 100
                    #         else:
                    #             gpu_percent = 0

                    #         # Force cleanup at 95% threshold
                    #         if cpu_percent >= self.force_cleanup_threshold or gpu_percent >= self.force_cleanup_threshold:
                    #             torch.cuda.empty_cache()
                    #             torch.cuda.synchronize()
                    #             gc.collect()
                    #             print(f"\nForce cleanup: CPU {cpu_percent:.1f}%, GPU {gpu_percent:.1f}%")
                    #     except:
                    #         pass



                    # Move data to device with optimized memory format
                    images = images.to(Config.DEVICE, non_blocking=True, memory_format=torch.channels_last)
                    # images = images.contiguous(memory_format=torch.channels_last)
                    labels = labels.to(Config.DEVICE, non_blocking=True)
                    # images, labels = images.to(Config.DEVICE, non_blocking=True), labels.to(Config.DEVICE, non_blocking=True)
                    # images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                    # if torch.cuda.is_available():
                    #     torch.cuda.empty_cache()
                    # device = Config.DEVICE if hasattr(Config, 'DEVICE') and Config.DEVICE else torch.device('cpu')
                    # images, labels = images.to(device), labels.to(device)
                    # Fix shape before moving to device
                    # if len(images.shape) == 3:
                    #     images = images.unsqueeze(0)
                    # images = images.to(Config.DEVICE)
                    # labels = torch.tensor(labels).to(Config.DEVICE) if not isinstance(labels, torch.Tensor) else labels.to(Config.DEVICE)

                    optimizer.zero_grad(set_to_none=True)
                    with torch.cuda.amp.autocast(enabled=Config.USE_MIXED_PRECISION):
                        outputs = model(images)
                        loss = criterion(outputs, labels)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()

                    batch_loss = loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()
                    train_loss += batch_loss
                    batch_acc = train_correct / train_total

                    #Show the Training Progress
                    # self._update_progress(batch_idx, batch_loss, batch_acc, trial_params, is_training=True)
                    train_pbar.set_postfix({'loss': f'{batch_loss:.4f}', 'acc': f'{batch_acc:.4f}'})

                    # ENHANCED: Memory cleanup
                    del images, labels, outputs, loss, predicted
                    # if batch_idx % 40 == 0:
                    #     torch.cuda.empty_cache()
                    #     gc.collect()  # ADDED: Garbage collection

                model.eval()

                val_correct = 0
                val_total = 0
                val_loss = 0
                val_predictions = []
                val_labels = []

                val_pbar = tqdm(temp_val_loader, desc=f"Optuna Epoch {optuna_epoch+1}/{Config.OPTUNA_EPOCHS} Validation", total=len(temp_val_loader), leave=False)



                from sklearn.metrics import f1_score
                # from torchmetrics.functional import F1Score
                # GPU-based torchmetrics (commented out)
                # f1_metric = F1Score(task="multiclass", num_classes=Config.NUM_CLASSES, average="macro").to(Config.DEVICE)

                with torch.no_grad():
                    for batch_idx, (images, labels) in enumerate(val_pbar): #val_pbar
                        # # ADDED: Memory check every 5 batches during validation
                        # if batch_idx % 40 == 0:
                        #     try:
                        #         cpu_percent = psutil.virtual_memory().percent
                        #         gpu_percent = (torch.cuda.memory_reserved() / torch.cuda.get_device_properties(0).total_memory) * 100 if torch.cuda.is_available() else 0
                        #         if cpu_percent >= self.force_cleanup_threshold or gpu_percent >= self.force_cleanup_threshold:
                        #             torch.cuda.empty_cache()
                        #             gc.collect()
                        #     except:
                        #         pass
                        images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                        outputs = model(images)
                        loss = criterion(outputs, labels)
                        batch_loss = loss.item()
                        val_loss += batch_loss
                        _, predicted = torch.max(outputs.data, 1)
                        # outputs = tensor([[1.2, 0.5, 2.1],[0.3, 4.1, 1.0]])
                        # _, predicted = torch.max(outputs, 1)
                        # print(predicted)  # tensor([2, 1]) → class 2 for first sample, class 1 for second
                        val_total += labels.size(0)
                        val_correct += (predicted == labels).sum().item()

                        # Store predictions and labels on CPU for sklearn F1Score calculation
                        val_predictions.extend(predicted.cpu().numpy())
                        val_labels.extend(labels.cpu().numpy())

                        # GPU tensor storage (commented out)
                        # val_predictions.append(predicted)   # keep tensors on GPU
                        # val_labels.append(labels)           # keep tensors on GPU

                        batch_acc = val_correct / val_total
                        #Show the Validation Progress
                        val_pbar.set_postfix({'loss': f'{batch_loss:.4f}', 'acc': f'{batch_acc:.4f}'})
                        # ADDED: Memory cleanup for validation
                        del images, labels, outputs, loss, predicted
                        # if batch_idx % 40 == 0:  # Less frequent cleanup in validation
                        #     torch.cuda.empty_cache()

                # CPU-based F1Score calculation using sklearn
                # val_f1_score = f1_score(val_labels, val_predictions, average='macro')

                # GPU-based torchmetrics calculation (commented out)
                # val_predictions_tensor = torch.cat(val_predictions).cpu().numpy()
                # val_labels_tensor = torch.cat(val_labels).cpu().numpy()

                train_acc = train_correct / train_total
                val_acc = val_correct / val_total



                # self._update_progress(batch_idx, batch_loss, batch_acc, trial_params, is_training=False)  # ✅ is_training=False
                # val_f1 = f1_score(val_labels, val_predictions, average='macro')
                scheduler.step()


                #from torchmetrics.classification import F1Score
                #initialize before training loop
                # f1_metric = F1Score(task="multiclass", num_classes=Config.NUM_CLASSES, average="macro").to(Config.DEVICE)
                # val_f1 = f1_metric.compute().item()   # stays on GPU, returns scalar
                val_f1 = f1_score(val_labels, val_predictions, average='macro')


                #For Opturna Report Purpose
                trial.report(val_f1, step=optuna_epoch)

                print(f"\nTrial {trial.number+1} Optuna Epoch {optuna_epoch+1} completed for Model: {self.model_name}"
                      f"\nOptuna Epoch {optuna_epoch+1}/{Config.OPTUNA_EPOCHS} Summary: "
                      f"TL: {train_loss/len(temp_train_loader):.6f}, "
                      f"VL: {val_loss/len(temp_val_loader):.6f}, "
                      f"TA: {train_acc:.4f}, "
                      f"VA: {val_acc:.4f}, "
                      f"VF1: {val_f1:.4f}\n")


                if val_f1 >= best_val_f1 * 1.001:  # Use self.best_val_f1 for class-level tracking
                    # self.best_trial = trial.number + 1
                    best_val_f1 = val_f1
                    best_val_acc = val_acc
                    patience_counter = 0
                    is_best = True
                    # Optional: Save model checkpoint if needed
                    # torch.save(model.state_dict(), f"{Config.OUTPUT_DIR}/trial_best_model/trial_{trial.number}_best.pt")
                else:
                    patience_counter += 1

                #Early stopping for Epoch Level
                if patience_counter >= Config.PATIENCE:
                    print(f"Early stopping at epoch {optuna_epoch + 1}: No improvement >= 0.1% in val_f1 for {Config.PATIENCE} validations")
                    break


                #Early stopping for Trial Level
                logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

                try:
                    # F1-based pruning (Optuna standard)
                    if optuna_epoch >= 4:
                        trial.report(val_f1, optuna_epoch)

                        if trial.should_prune():
                            best_f1 = max([t.value for t in trial.study.trials if t.value is not None], default=val_f1)
                            msg = (f"⛔ Trial {trial.number + 1} stopped at epoch {optuna_epoch + 1}: "  #logging.info(
                                  f"val_f1={val_f1:.4f} is below best_f1={best_f1:.4f}."
                                  f"Pruning trial to focus on better candidates."
                            )
                            print(msg)               # show message
                            logging.info(msg)        # log message
                            trial.set_user_attr("pruned_reason", msg)
                            raise optuna.TrialPruned()

                    # Custom accuracy-based stopping rule
                    if optuna_epoch + 1 >= 5 and val_f1 < 0.90:
                        msg = (f"⛔ Trial {trial.number + 1} stopped at epoch {optuna_epoch + 1}: "
                              f"val_f1={val_f1:.4f} did not reach even 30% after 5 epochs.So Skipping this epoch....")
                        print(msg)                   # show message
                        logging.info(msg)            # log message
                        trial.set_user_attr("pruned_reason", msg)
                        raise optuna.TrialPruned()


                except optuna.TrialPruned as e:
                    msg = (f"⛔ Trial {trial.number + 1} stopped at epoch {optuna_epoch + 1}: "
                           f"val_f1={val_f1:.4f} is below best_f1={best_f1:.4f}. "
                           f"Pruning trial to focus on better candidates.")
                    print(msg)   # or logging.info(msg)
                    #pass
                    raise




            # Update class-level tracking
            if best_val_f1 > self.best_val_f1  :
                self.best_val_f1 = best_val_f1
                self.best_val_acc = best_val_acc
                self.best_trial = trial.number + 1

                # print(f"\n\033[1;31mNew best result found for Trial {self.best_trial}:\033[0m")
                if (trial.number+1!=1):
                    print(f"\n\033[1;31mNew best result found for Trial {self.best_trial}:\033[0m")
                else:
                    print(f"\n\033[1;31mThis is the First Trial. So No Scope for Judgement.\033[0m")

                print(f"Validation Accuracy: {self.best_val_acc:.4f}")
                print(f"Validation F1 Score: {self.best_val_f1:.4f}")

            else:
                print(f"\nTrial {trial.number+1} did not improved the accuracy.") # overall best (Current best F1: {self.best_val_f1:.4f})")


            if (trial.number+1>1):
                print(f"\n\033[1;31mTrial {self.best_trial} holds best result up to this:\033[0m")
                print(f"Validation Accuracy: {self.best_val_acc:.4f}")
                print(f"Validation F1 Score: {self.best_val_f1:.4f}")

            # ADDED: Trial cleanup before return
            try:
                del model, optimizer, scheduler, criterion
                del temp_train_loader, temp_val_loader
                del val_predictions, val_labels
                torch.cuda.empty_cache()
                gc.collect()
            except:
                pass

            return self.best_val_f1  # Return the last epoch's F1 for Optuna to maximize

        except Exception as e:
            import traceback
            print(f"\nTrial {trial.number+1} Skipped with : {str(e)}\n{traceback.format_exc()} cause F1 Not Improving!!!!")

            # ADDED: Emergency cleanup on error
            try:
                del model, optimizer, scheduler, criterion
                torch.cuda.empty_cache()
                gc.collect()
            except:
                pass

            return 0.0 #✅This is a safe pattern for Optuna trials when an unexpected error occurs and you want to skip the trial without crashing your training loop.



    def optimize(self):
        print(f"Optimizing hyperparameters for {self.model_name} : ")

        if not OPTUNA_AVAILABLE:
            return {
                'lr': Config.LEARNING_RATE,
                'dropout': 0.5,
                'weight_decay': Config.WEIGHT_DECAY,
                'hidden_dim_multiplier': 0.5,
                'augmentation_strength': 'medium',
                'batch_size': Config.BATCH_SIZE,
                'optimizer_type': 'adamw',
                'scheduler_type': 'cosine',
                'label_smoothing': 0.1
            }

        # Successive Halving Pruner
        pruner = optuna.pruners.SuccessiveHalvingPruner(
            min_resource=3,        # first rung uses 10 epochs
            reduction_factor=4,     # top 1/4 survive each rung
            min_early_stopping_rate=0,  # optional, can start from first rung
        )
        study = optuna.create_study(direction='maximize',pruner=pruner)

        # study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=4))
        # study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner(min_resource=1, reduction_factor=4))
        # The number of epochs per rung is controlled by min_resource and how you report steps (trial.report(val, step=epoch)).


        optuna.logging.set_verbosity(optuna.logging.WARNING)  # Suppress info-level logs
        study.optimize(self.objective, n_trials=Config.OPTUNA_TRIALS, timeout=3600)#, callbacks=[callback])

        best_params = study.best_trial.params

        # print(f"\nBest params for {self.model_name}:")
        # for key, value in best_params.items():
        #     if key in ['lr', 'weight_decay']:
        #         print(f"  {key}: {value:.4f}")
        #     elif key in ['dropout', 'hidden_dim_multiplier', 'label_smoothing']:
        #         print(f"  {key}: {value:.6f}")
        #     else:
        #         print(f"  {key}: {value}")
        # print(f"Best validation accuracy: {study.best_trial.value:.4f}")

        return best_params





# ---
# 11. MODEL EVALUATION (FIXED)
# =============================================================================
class ModelEvaluator:
    @staticmethod
    def evaluate_model(model, test_loader, model_name):
        try:
            print(f"\nEvaluating For Model: {model_name}...")
            model.eval()

            all_preds = []
            all_labels = []
            misclassified = []
            total_loss = 0
            criterion = nn.CrossEntropyLoss()

            all_probs = []

            if test_loader is None or len(test_loader.dataset) == 0:
                print(f"Warning: No test data available for {model_name}")
                return ModelEvaluator._create_empty_result(model_name)

            with torch.no_grad():
                for batch_idx, (images, labels) in enumerate(test_loader):
                    # CPU version
                    # images, labels = images.cpu(), labels.cpu()
                    #GPU version
                    # Keep everything on the same device as the model
                    images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)



                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    total_loss += loss.item()

                    _, predicted = torch.max(outputs.data, 1)

                    # all_preds.extend(predicted.numpy())
                    # all_labels.extend(labels.numpy())
                    # To this:
                    all_preds.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())



                    # Convert logits to probabilities
                    prob = torch.softmax(outputs, dim=1)
                    # Store probabilities
                    all_probs.append(prob.detach().cpu().numpy())




                    # Identify misclassified
                    incorrect_mask = predicted != labels
                    if incorrect_mask.any():
                        incorrect_images = images[incorrect_mask]
                        incorrect_labels = labels[incorrect_mask]
                        incorrect_preds = predicted[incorrect_mask]
                        for img, true_label, pred_label in zip(incorrect_images, incorrect_labels, incorrect_preds):
                            misclassified.append({
                                'image': img,
                                'true_label': int(true_label),
                                'pred_label': int(pred_label)
                            })
                    del images, labels, outputs, loss, predicted




                    # ---------------- GPU VERSION (commented) ----------------
                    # images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                    # outputs = model(images)
                    # loss = criterion(outputs, labels)
                    # _, predicted = torch.max(outputs.data, 1)
                    # all_preds.append(predicted)
                    # all_labels.append(labels)
                    # --------------------------------------------------------



            # CPU metrics
            # Convert to single array
            all_probs = np.concatenate(all_probs, axis=0)  # shape: (num_samples, num_classes)
            # all_probs = [
            #     (32, 5),   # batch 1: 32 samples × 5 classes
            #     (32, 5),   # batch 2: 32 samples × 5 classes
            #     (16, 5)    # batch 3: 16 samples × 5 classes
            # ]
            # result shape = (32 + 32 + 16, 5) = (80, 5)




            all_preds = np.array(all_preds)
            all_labels = np.array(all_labels)
            avg_loss = total_loss / len(test_loader)
            accuracy = accuracy_score(all_labels, all_preds)
            f1_macro = f1_score(all_labels, all_preds, average='macro', zero_division=0)
            f1_weighted = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
            precision_macro = precision_score(all_labels, all_preds, average='macro', zero_division=0)
            recall_macro = recall_score(all_labels, all_preds, average='macro', zero_division=0)
            f1_per_class = f1_score(all_labels, all_preds, average=None, zero_division=0)
            precision_per_class = precision_score(all_labels, all_preds, average=None, zero_division=0)
            recall_per_class = recall_score(all_labels, all_preds, average=None, zero_division=0)
            conf_matrix = confusion_matrix(all_labels, all_preds)

            print(f"{model_name} Evaluation: Acc = {accuracy:.4f}, F1 (Macro) = {f1_macro:.4f}, Loss = {avg_loss:.4f}")
            print(f"Data shapes - Labels: {all_labels.shape}, Predictions: {all_preds.shape}")

            return {
                'all_preds': all_preds,
                'all_labels': all_labels,
                'all_probs': all_probs,


                'model_name': model_name,
                'accuracy': accuracy,
                'f1_macro': f1_macro,
                'f1_weighted': f1_weighted,
                'precision_macro': precision_macro,
                'recall_macro': recall_macro,
                'f1_per_class': f1_per_class,
                'precision_per_class': precision_per_class,
                'recall_per_class': recall_per_class,
                'predictions': all_preds,
                'true_labels': all_labels,
                # 'probabilities': None,  # CPU version
                'probabilities': all_probs,  # correctly collected probabilities
                'conf_matrix': conf_matrix,
                'loss': avg_loss,
                'misclassified': misclassified
            }

        except Exception as e:
            print(f"Evaluation error for {model_name}: {e}")
            return ModelEvaluator._create_empty_result(model_name)

    @staticmethod
    def _create_empty_result(model_name):
        return {
            'model_name': model_name,
            'accuracy': 0.0,
            'f1_macro': 0.0,
            'f1_weighted': 0.0,
            'precision_macro': 0.0,
            'recall_macro': 0.0,
            'f1_per_class': np.zeros(Config.NUM_CLASSES),
            'precision_per_class': np.zeros(Config.NUM_CLASSES),
            'recall_per_class': np.zeros(Config.NUM_CLASSES),
            'predictions': np.array([]),
            'true_labels': np.array([]),
            'probabilities': np.array([]),
            'conf_matrix': np.zeros((Config.NUM_CLASSES, Config.NUM_CLASSES)),
            'loss': 0.0,
            'misclassified': []
        }




# ---
# 12. MODEL TRAINING
# =============================================================================
# Purpose: Train individual models with optimized hyperparameters.

class EnhancedModelTrainer:
    def __init__(self, model, model_name, hyperparameters):
        self.model = model.to(Config.DEVICE)
        self.model_name = model_name
        self.hyperparameters = hyperparameters
        self.best_val_acc = 0.0
        self.best_val_f1 = 0.0
        self.patience_counter = 0

        # Resource management
        self.resource_manager = ResourceManager()
        self.memory_check_interval = 15



        self._setup_training_components()



        # Initialize history
        self.history = {
            'train_loss': [], 'train_acc': [], 'val_loss': [],
            'val_acc': [], 'val_f1': [], 'learning_rates': []
        }




    def _setup_training_components(self):
        """Setup optimizer, criterion, and scheduler"""
        allowed_keys = ['lr', 'weight_decay', 'dropout', 'hidden_dim_multiplier',
                       'augmentation_strength', 'batch_size', 'optimizer_type',
                       'scheduler_type', 'label_smoothing']
        self.hyperparameters = {k: v for k, v in self.hyperparameters.items() if k in allowed_keys}

        # Optimizer setup
        lr = self.hyperparameters.get('lr', Config.LEARNING_RATE)
        weight_decay = self.hyperparameters.get('weight_decay', Config.WEIGHT_DECAY)
        optimizer_type = self.hyperparameters.get('optimizer_type', 'adamw')

        if optimizer_type == 'adamw':
            self.optimizer = optim.AdamW(
                self.model.parameters(), lr=lr, weight_decay=weight_decay,
                fused=torch.cuda.is_available()
            )
        elif optimizer_type == 'adam':
            self.optimizer = optim.Adam(
                self.model.parameters(), lr=lr, weight_decay=weight_decay,
                fused=torch.cuda.is_available()
            )
        else:
            self.optimizer = optim.SGD(
                self.model.parameters(), lr=lr, weight_decay=weight_decay,
                momentum=0.9, nesterov=True
            )



        # Criterion
        label_smoothing = self.hyperparameters.get('label_smoothing', 0.1)
        self.criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

        # Scheduler
        scheduler_type = self.hyperparameters.get('scheduler_type', 'cosine')
        if scheduler_type == 'cosine':
            self.scheduler = optim.lr_scheduler.CosineAnnealingLR(
                self.optimizer, T_max=Config.EPOCHS, eta_min=1e-6
            )
        elif scheduler_type == 'plateau':
            self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, mode='min', factor=0.5, patience=5
            )
        else:
            self.scheduler = optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.95)


        # Mixed precision scaler
        self.scaler = torch.cuda.amp.GradScaler(enabled=Config.USE_MIXED_PRECISION)



    def train_epoch(self, train_loader, progress_tracker):
        """Enhanced training epoch with smart memory management"""
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        batch_count = len(train_loader)

        tqdm.write(f"Total Data: {len(train_loader.dataset):,}  Training : "
                   f"{batch_count:,} batches, Batch_size: {train_loader.batch_size}")

        try:
            for batch_idx, (images, labels) in enumerate(train_loader):
                try:
                    # Smart memory management
                    if batch_idx % self.memory_check_interval == 0:
                        if self.resource_manager.should_cleanup_aggressive():
                            self.resource_manager.aggressive_cleanup()
                            tqdm.write(f"  Memory cleanup at batch {batch_idx}")

                    # Move data to device with optimized memory format
                    images = images.to(Config.DEVICE, non_blocking=True, memory_format=torch.channels_last)
                    # images = images.contiguous(memory_format=torch.channels_last)
                    labels = labels.to(Config.DEVICE, non_blocking=True)

                    # Forward pass
                    self.optimizer.zero_grad(set_to_none=True)
                    with torch.cuda.amp.autocast(enabled=Config.USE_MIXED_PRECISION):
                        outputs = self.model(images)
                        loss = self.criterion(outputs, labels)

                    # Backward pass
                    self.scaler.scale(loss).backward()
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                    self.scaler.step(self.optimizer)
                    self.scaler.update()

                    # Calculate metrics
                    _, predicted = torch.max(outputs, 1)
                    batch_acc = (predicted == labels).float().mean().item()
                    batch_loss = loss.item()

                    # Update totals
                    total_loss += batch_loss * images.size(0)
                    total += images.size(0)
                    correct += (predicted == labels).sum().item()

                    # Update progress tracker
                    progress_tracker.update_batch(batch_idx, batch_loss, batch_acc, is_training=True, total_batches=batch_count)

                    # Progress logging every 50 batches
                    # if batch_idx % 40 == 0 and batch_idx > 0:
                    #     avg_loss = total_loss / total
                    #     avg_acc = correct / total
                    #     stats = self.resource_manager.get_memory_stats()
                    #     tqdm.write(f"  Batch {batch_idx:4d}/{batch_count} - "
                    #                f"Loss: {avg_loss:.4f}, Acc: {avg_acc:.4f}, "
                    #                f"GPU: {stats['gpu_percent']:.1f}%, CPU: {stats['cpu_percent']:.1f}%")

                    # Memory cleanup for large batches
                    del outputs, loss, predicted, images, labels

                except Exception as e:
                    tqdm.write(f"Error in batch {batch_idx}: {str(e)}")
                    self.resource_manager.aggressive_cleanup()
                    continue

            # Final cleanup
            self.resource_manager.aggressive_cleanup()

            return total_loss / max(1, total), correct / max(1, total)

        except Exception as e:
            tqdm.write(f"Training epoch failed: {str(e)}")
            self.resource_manager.aggressive_cleanup()
            return float('inf'), 0.0



    def validate_epoch(self, val_loader, progress_tracker):
        """Enhanced validation epoch with memory optimization"""
        self.model.eval()
        total_loss = 0
        total_samples = 0
        all_predictions = []
        all_labels = []
        all_probs = []
        batch_count = len(val_loader)

        tqdm.write(f"Validation epoch: {len(val_loader.dataset):,} samples, "
                   f"{batch_count:,} batches, batch_size: {val_loader.batch_size}")

        try:
            with torch.no_grad():
                for batch_idx, (images, labels) in enumerate(val_loader):
                    try:
                        images = images.to(Config.DEVICE, non_blocking=True, memory_format=torch.channels_last)
                        # images = images.contiguous(memory_format=torch.channels_last)
                        labels = labels.to(Config.DEVICE, non_blocking=True)

                        with torch.cuda.amp.autocast(enabled=Config.USE_MIXED_PRECISION):
                            outputs = self.model(images)
                            loss = self.criterion(outputs, labels)

                        _, predicted = torch.max(outputs, 1)
                        batch_acc = (predicted == labels).float().mean().item()
                        batch_loss = loss.item()

                        # Store results
                        total_loss += batch_loss * images.size(0)
                        total_samples += images.size(0)
                        all_predictions.extend(predicted.cpu().numpy())
                        all_labels.extend(labels.cpu().numpy())
                        all_probs.append(torch.softmax(outputs, dim=1).cpu().numpy())

                        # Update progress
                        progress_tracker.update_batch(batch_idx, batch_loss, batch_acc,
                                                    is_training=False, total_batches=batch_count)

                        # Memory cleanup
                        del outputs, loss, predicted, images, labels

                        # # Periodic memory management
                        # if batch_idx % 20 == 0:
                        #     self.resource_manager.aggressive_cleanup()

                    except Exception as e:
                        tqdm.write(f"Error in validation batch {batch_idx}: {str(e)}")
                        continue

            # Calculate final metrics
            all_probs = np.concatenate(all_probs, axis=0) if all_probs else np.array([])
            val_acc = accuracy_score(all_labels, all_predictions)
            val_f1 = f1_score(all_labels, all_predictions, average='macro', zero_division=0)

            return total_loss / max(1, total_samples), val_acc, val_f1, all_predictions, all_labels, all_probs

        except Exception as e:
            tqdm.write(f"Validation epoch failed: {str(e)}")
            self.resource_manager.aggressive_cleanup()
            return float('inf'), 0.0, 0.0, [], [], np.array([])



    def train_main_model(self, train_loader, val_loader, test_loader=None):
        """Main model training with comprehensive logging"""
        if not train_loader or len(train_loader.dataset) == 0:
            tqdm.write(f"Skipping {self.model_name}: No training data")
            return None, None

        if not val_loader or len(val_loader.dataset) == 0:
            tqdm.write(f"Skipping {self.model_name}: No validation data")
            return None, None

        tqdm.write(f"\nTraining {self.model_name}")
        tqdm.write(f"Training samples: {len(train_loader.dataset):,}")
        tqdm.write(f"Validation samples: {len(val_loader.dataset):,}")
        tqdm.write(f"Total epochs: {Config.EPOCHS}")
        tqdm.write(f"Batch size: {train_loader.batch_size}")

        # Print hyperparameters
        tqdm.write("Hyperparameters:")
        for key, value in self.hyperparameters.items():
            if key in ['lr', 'weight_decay', 'dropout', 'hidden_dim_multiplier', 'label_smoothing']:
                tqdm.write(f"  {key}: {value:.4f}")
            else:
                tqdm.write(f"  {key}: {value}")

        # Setup model for training
        self.model = self.model.to(Config.DEVICE, memory_format=torch.channels_last)

        # Add this line after model creation:
        # self.model = torch.compile(self.model, mode="max-autotune")

        # torch.backends.cudnn.benchmark = True
        # torch.backends.cudnn.deterministic = False


        # Progress tracker
        progress_tracker = TrainingProgressTracker(self.model_name, Config.EPOCHS, len(train_loader))

        # Training loop
        training_start_time = time.time()

        # # Before loop
        # single_models = {}
        # single_results = {}

        for epoch in range(Config.EPOCHS):
            epoch_start_time = time.time()
            tqdm.write(f"\nEpoch {epoch + 1}/{Config.EPOCHS}")

            progress_tracker.start_epoch(epoch)

            # Training phase
            train_loss, train_acc = self.train_epoch(train_loader, progress_tracker)

            # Validation phase
            val_loss, val_acc, val_f1, predictions, labels, probs = self.validate_epoch(val_loader, progress_tracker)

            # Learning rate scheduling
            if isinstance(self.scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                self.scheduler.step(val_loss)
            else:
                self.scheduler.step()

            # Progress tracking
            is_best = val_f1 > self.best_val_f1 * 1.001
            current_lr = self.optimizer.param_groups[0]['lr']

            progress_tracker.finish_epoch(train_loss, train_acc, val_loss, val_acc, val_f1, is_best=is_best, lr=current_lr)

            # Store history
            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            self.history['val_f1'].append(val_f1)
            self.history['learning_rates'].append(current_lr)

            # Early stopping and best model saving
            # if is_best:
            #     self.best_val_f1 = val_f1
            #     self.best_val_acc = val_acc
            #     self.patience_counter = 0
            #     # torch.save(f"{Config.OUTPUT_DIR}/best_model/{self.model_name}_best.pt")
            #     # Save complete model with architecture and hyperparameters
            #     torch.save({
            #         'state_dict': self.model.state_dict(),
            #         'hyperparameters': self.hyperparameters,
            #         'model_name': self.model_name
            #     }, f"{Config.OUTPUT_DIR}/best_model/{self.model_name}_best.pt")

            #     tqdm.write(f"New best model saved: F1={val_f1:.4f}, Acc={val_acc:.4f}")
            # else:
            #     self.patience_counter += 1
            # Early stopping and best model saving
            if is_best:
                self.best_val_f1 = val_f1
                self.best_val_acc = val_acc
                self.patience_counter = 0

                # Create comprehensive checkpoint
                checkpoint = {
                    # Model architecture and weights
                    'state_dict': self.model.state_dict(),
                    'model_name': self.model_name,
                    'hyperparameters': self.hyperparameters,

                    # Training metadata
                    'epoch': epoch + 1,
                    'best_val_f1': val_f1,
                    'best_val_acc': val_acc,
                    'optimizer_state': self.optimizer.state_dict(),
                    'scheduler_state': self.scheduler.state_dict() if self.scheduler else None,

                    # Architecture info for reconstruction
                    'num_classes': Config.NUM_CLASSES,
                    'dropout_rate': self.hyperparameters.get('dropout', 0.5),
                    'hidden_dim_multiplier': self.hyperparameters.get('hidden_dim_multiplier', 0.5),

                    # Save format version for future compatibility
                    'save_format_version': '1.0'
                }

                # Save with error handling
                try:
                    torch.save(checkpoint, f"{Config.OUTPUT_DIR}/best_model/{self.model_name}_best.pt")
                    tqdm.write(f"✅ Best model saved: F1={val_f1:.4f}, Acc={val_acc:.4f}")
                except Exception as e:
                    tqdm.write(f"❌ Error saving model {self.model_name}: {e}")
            else:
                self.patience_counter += 1




            # Epoch summary
            epoch_time = time.time() - epoch_start_time
            tqdm.write(f"\nEpoch {epoch + 1} Summary:")
            tqdm.write(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
            tqdm.write(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")
            tqdm.write(f"  Learning Rate: {current_lr:.6f}")
            tqdm.write(f"  Epoch Time: {epoch_time:.1f}s")
            tqdm.write(f"  Best F1 so far: {self.best_val_f1:.4f}")

            # Early stopping check
            if self.patience_counter >= Config.PATIENCE:
                total_time = time.time() - training_start_time
                tqdm.write(f"Early stopping at epoch {epoch + 1}")
                tqdm.write(f"Total training time: {total_time:.1f}s")
                break

        # # Load best model for evaluation
        # best_model_path = f"{Config.OUTPUT_DIR}/best_model/{self.model_name}_best.pt"
        # if os.path.exists(best_model_path):
        #     # self.model.load_state_dict(torch.load(best_model_path, map_location=Config.DEVICE))

        #     tqdm.write(f"Loaded best model from {best_model_path}")
        # else:
        #     tqdm.write("Warning: Best model not found, using current weights")
        # Load best model for evaluation with robust error handling
        best_model_path = f"{Config.OUTPUT_DIR}/best_model/{self.model_name}_best.pt"
        if os.path.exists(best_model_path):
            try:
                checkpoint = torch.load(best_model_path, map_location=Config.DEVICE, weights_only=False)

                if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
                    # New comprehensive format
                    self.model.load_state_dict(checkpoint['state_dict'])
                    tqdm.write(f"✅ Loaded best model (v{checkpoint.get('save_format_version', '1.0')})")
                else:
                    # Legacy format or direct state_dict
                    self.model.load_state_dict(checkpoint)
                    tqdm.write(f"✅ Loaded best model (legacy format)")

            except Exception as e:
                tqdm.write(f"⚠️ Could not load best model: {e}")
                tqdm.write(f"Continuing with current weights...")
        else:
            tqdm.write(f"⚠️ Best model file not found, using current weights")





        # Final evaluation
        evaluator = ModelEvaluator()
        eval_loader = test_loader if test_loader else val_loader

        with torch.no_grad():
            result = evaluator.evaluate_model(self.model, eval_loader, self.model_name)

        # Save results
        # result_to_save = {
        #     'history': {k: torch.tensor(v, dtype=torch.float32) for k, v in self.history.items()},
        #     'result': {
        #         'accuracy': torch.tensor(result.get('accuracy', 0.0), dtype=torch.float32),
        #         'f1': torch.tensor(result.get('f1_macro', 0.0), dtype=torch.float32),
        #         'conf_matrix': torch.tensor(result.get('conf_matrix', np.array([[]])), dtype=torch.int64),
        #         'true_labels': torch.tensor(result.get('true_labels', []), dtype=torch.int64),
        #         'predictions': torch.tensor(result.get('predictions', []), dtype=torch.int64),
        #         'probabilities': torch.tensor(result.get('probabilities', np.zeros((0, Config.NUM_CLASSES))), dtype=torch.float32),
        #         'misclassified': [{'image': item['image'].cpu(), 'true_label': item['true_label'], 'pred_label': item['pred_label']}
        #                         for item in result.get('misclassified', [])] if result.get('misclassified') else []
        #     }
        # }

        result_to_save = {
            'history': {k: torch.as_tensor(v, dtype=torch.float32).detach().cpu() for k, v in self.history.items()},
            'result': {
                'accuracy': torch.as_tensor(result.get('accuracy', 0.0), dtype=torch.float32).detach().cpu(),
                'f1': torch.as_tensor(result.get('f1_macro', 0.0), dtype=torch.float32).detach().cpu(),
                'conf_matrix': torch.as_tensor(result.get('conf_matrix', np.array([[]])), dtype=torch.int64).detach().cpu(),
                'true_labels': torch.as_tensor(result.get('true_labels', []), dtype=torch.int64).detach().cpu(),
                'predictions': torch.as_tensor(result.get('predictions', []), dtype=torch.int64).detach().cpu(),
                # 'probabilities': torch.as_tensor(result.get('probabilities', np.zeros((0, Config.NUM_CLASSES))), dtype=torch.float32).detach().cpu(),
                'probabilities': torch.as_tensor(
                    result.get('probabilities') if result.get('probabilities') is not None else np.zeros((0, Config.NUM_CLASSES)),
                    dtype=torch.float32
                ).detach().cpu(),
                # 'probabilities': torch.as_tensor(result.get('probabilities') if result.get('probabilities') is not None else np.zeros((0, Config.NUM_CLASSES)), dtype=torch.float32).detach().cpu(),
                'misclassified': [
                    {'image': item['image'].detach().cpu(),
                    'true_label': item['true_label'],
                    'pred_label': item['pred_label']}
                    for item in result.get('misclassified', [])
                ] if result.get('misclassified') else []
            }
        }



        main_result_path = f"{Config.OUTPUT_DIR}/model_results/{self.model_name}_main_results.pt"
        torch.save(result_to_save, main_result_path)
        tqdm.write(f"Results saved to {main_result_path}")

        # Training summary
        total_training_time = time.time() - training_start_time
        tqdm.write(f"\nTraining Summary for {self.model_name}:")
        tqdm.write(f"  Final Accuracy: {result.get('accuracy', 0.0):.4f}")
        tqdm.write(f"  Final F1 Score: {result.get('f1_macro', 0.0):.4f}")
        tqdm.write(f"  Best Validation F1: {self.best_val_f1:.4f}")
        tqdm.write(f"  Total Training Time: {total_training_time:.1f}s")
        tqdm.write(f"  Epochs Completed: {epoch + 1}/{Config.EPOCHS}")

        # Cleanup
        del result_to_save, result, evaluator, eval_loader
        self.resource_manager.aggressive_cleanup()

        # return self.history, None
        # return None, None  # Don't return data since it's saved and also change calling function from main




    def train_kfold(self, train_loader, val_loader, test_loader, n_folds=3):
        """K-fold cross-validation with detailed logging"""
        if n_folds <= 0:
            tqdm.write(f"Skipping k-fold for {self.model_name}: n_folds <= 0")
            return []


        from torch.utils.data import ConcatDataset
        combined_dataset = ConcatDataset([
            train_loader.dataset,
            val_loader.dataset
            # test_loader.dataset if test_loader is not None else []
        ])
        dataset = combined_dataset
        test_loader_fold=test_loader

        # dataset = train_loader.dataset
        total_samples = len(dataset)
        min_samples_per_fold = 500

        # Adjust folds based on data availability
        if total_samples < n_folds * min_samples_per_fold:
            n_folds = max(1, total_samples // min_samples_per_fold)
            tqdm.write(f"Adjusted to {n_folds} folds due to insufficient data")

        if n_folds < 2:
            tqdm.write(f"Skipping k-fold: need at least {min_samples_per_fold*2} samples, have {total_samples}")
            return []

        # Calculate fold statistics
        samples_per_fold = total_samples // n_folds
        train_samples_per_fold = total_samples - samples_per_fold

        tqdm.write(f"\nK-fold Cross-Validation Setup for {self.model_name}:")
        tqdm.write(f"  Total samples: {total_samples:,}")
        tqdm.write(f"  Number of folds: {n_folds}")
        tqdm.write(f"  Samples per fold (validation): {samples_per_fold:,}")
        tqdm.write(f"  Samples per fold (training): {train_samples_per_fold:,}")

        # Create fold indices
        fold_indices = []
        for i in range(n_folds):
            val_start = i * samples_per_fold
            val_end = min(val_start + samples_per_fold, total_samples)
            val_idx = list(range(val_start, val_end))
            train_idx = list(range(0, val_start)) + list(range(val_end, total_samples))
            fold_indices.append((train_idx, val_idx))

            tqdm.write(f"  Fold {i+1}: Train={len(train_idx):,}, Val={len(val_idx):,}")

        # Optimize settings for k-fold
        # fold_batch_size = self.resource_manager.optimize_batch_size(64, 1.0)
        # Use the ResourceManager method directly
        base_batch_size = self.hyperparameters.get('batch_size', Config.BATCH_SIZE)
        # Use the same complexity calculation as in main()
        model_complexity_map = {
            'efficientnet': 1.5,
            'resnet': 1.0,
            'vgg': 0.8,
            'mobilenet': 0.6,
            'densenet': 1.3,
            'convnext': 1.4
        }
        model_complexity = model_complexity_map.get(self.model_name.split('_')[0].lower(), 1.0)
        fold_batch_size = self.resource_manager.optimize_batch_size(base_batch_size, model_complexity)
        # fold_batch_size = self.resource_manager.optimize_batch_size(base_batch_size, model_name=self.model_name)

        prefetch_factor = min(4, max(2, self.resource_manager.cpu_memory_gb // 10))
        num_workers = min(8, mp.cpu_count() // 2)

        tqdm.write(f"K-fold optimized settings:")
        tqdm.write(f"  Batch size: {fold_batch_size}")
        tqdm.write(f"  Workers: {num_workers}")
        tqdm.write(f"  Prefetch factor: {prefetch_factor}")

        fold_results = []
        total_kfold_start = time.time()

        for fold, (train_idx, val_idx) in enumerate(fold_indices, 1):
            fold_start_time = time.time()
            tqdm.write(f"\nTraining Fold {fold}/{n_folds}")

            try:
                # Create fold-specific data loaders
                train_subsampler = SubsetRandomSampler(train_idx)
                val_subsampler = SubsetRandomSampler(val_idx)

                train_loader_fold = DataLoader(
                    dataset,
                    batch_size=fold_batch_size,
                    sampler=train_subsampler,
                    num_workers=num_workers,
                    pin_memory=torch.cuda.is_available(),
                    prefetch_factor=prefetch_factor,
                    persistent_workers=num_workers > 0,
                    worker_init_fn=worker_init_fn if 'worker_init_fn' in globals() else None
                )

                val_loader_fold = DataLoader(
                    dataset,
                    batch_size=fold_batch_size,
                    sampler=val_subsampler,
                    num_workers=num_workers,
                    pin_memory=torch.cuda.is_available(),
                    prefetch_factor=prefetch_factor,
                    persistent_workers=num_workers > 0,
                    worker_init_fn=worker_init_fn if 'worker_init_fn' in globals() else None
                )

                # Create fold model
                fold_model = ModelFactory.create_model(
                    self.model_name,
                    num_classes=Config.NUM_CLASSES,
                    dropout_rate=self.hyperparameters.get('dropout', 0.5),
                    hidden_dim_multiplier=self.hyperparameters.get('hidden_dim_multiplier', 0.5)
                ).to(Config.DEVICE, memory_format=torch.channels_last)

                # Create fold trainer
                fold_trainer = EnhancedModelTrainer(fold_model, f"{self.model_name}_fold_{fold}", self.hyperparameters)

                # Train fold
                # fold_history, fold_result =
                fold_trainer.train_main_model(train_loader_fold, val_loader_fold, test_loader=None)
                fold_time = time.time() - fold_start_time

                # Save trained model
                fold_model_path = f"{Config.OUTPUT_DIR}/kfold_results/{self.model_name}_fold_{fold}_model.pt"
                torch.save(fold_model.state_dict(), fold_model_path)


                # Load the model back (optional but ensures evaluation uses saved model)
                fold_model.load_state_dict(torch.load(fold_model_path))
                fold_model.to(Config.DEVICE)

                # Evaluate the fold
                evaluator = ModelEvaluator()
                eval_loader = test_loader_fold  # or test_loader if you have it
                with torch.no_grad():
                    fold_result = evaluator.evaluate_model(fold_model, eval_loader, f"{self.model_name}_fold_{fold}")

                # # Handle fold results
                # if fold_history is None or fold_result is None:
                #     tqdm.write(f"Fold {fold} training failed")
                #     fold_result = {
                #         'accuracy': 0.0, 'f1_macro': 0.0, 'conf_matrix': np.array([[]]),
                #         'true_labels': [], 'predictions': [], 'probabilities': np.zeros((0, Config.NUM_CLASSES)),
                #         'misclassified': []
                #     }
                #     fold_history = {'val_loss': [], 'val_acc': []}


                # # Store results
                # fold_results.append({
                #     'history': {
                #         'val_loss': fold_history.get('val_loss', []),
                #         'val_acc': fold_history.get('val_acc', [])
                #     },
                #     'result': {
                #         'accuracy': torch.tensor(fold_result.get('accuracy', 0.0), dtype=torch.float32),
                #         'f1': torch.tensor(fold_result.get('f1_macro', 0.0), dtype=torch.float32),
                #         'conf_matrix': torch.tensor(fold_result.get('conf_matrix', np.array([[]])), dtype=torch.int64),
                #         'misclassified': fold_result.get('misclassified', [])
                #     }
                # })

                # Save fold results
                # fold_result_to_save = {
                #     'history': {
                #         'val_loss': torch.tensor(fold_history.get('val_loss', []), dtype=torch.float32),
                #         'val_acc': torch.tensor(fold_history.get('val_acc', []), dtype=torch.float32)
                #     },
                #     'result': {
                #         'accuracy': torch.tensor(fold_result.get('accuracy', 0.0), dtype=torch.float32),
                #         'f1': torch.tensor(fold_result.get('f1_macro', 0.0), dtype=torch.float32),
                #         'conf_matrix': torch.tensor(fold_result.get('conf_matrix', np.array([[]])), dtype=torch.int64),
                #         'misclassified': fold_result.get('misclassified', [])
                #     }
                # }
                # Save fold results
                fold_result_to_save = {
                    'history': {  # assuming train_main_model updates self.history inside fold_trainer
                        'val_loss': torch.tensor(fold_trainer.history.get('val_loss', []), dtype=torch.float32),
                        'val_acc': torch.tensor(fold_trainer.history.get('val_acc', []), dtype=torch.float32)
                    },
                    'result': {
                        'accuracy': torch.tensor(fold_result.get('accuracy', 0.0), dtype=torch.float32),
                        'f1': torch.tensor(fold_result.get('f1_macro', 0.0), dtype=torch.float32),
                        'conf_matrix': torch.tensor(fold_result.get('conf_matrix', np.array([[]])), dtype=torch.int64),
                        'misclassified': fold_result.get('misclassified', [])
                    }
                }



                fold_result_path = f"{Config.OUTPUT_DIR}/kfold_results/{self.model_name}_fold_{fold}_results.pt"
                torch.save(fold_result_to_save, fold_result_path)

                tqdm.write(f"Fold {fold} completed in {fold_time:.1f}s - "
                          f"Accuracy: {fold_result.get('accuracy', 0.0):.4f}, "
                          f"F1: {fold_result.get('f1_macro', 0.0):.4f}")

                # Cleanup fold
                del fold_trainer, fold_model #, fold_history, fold_result
                del train_loader_fold, val_loader_fold, train_subsampler, val_subsampler
                self.resource_manager.aggressive_cleanup()

            except Exception as e:
                tqdm.write(f"Error in fold {fold}: {str(e)}")
                import traceback
                traceback.print_exc()
                continue

        total_kfold_time = time.time() - total_kfold_start
        # successful_folds = len(fold_results)

        tqdm.write(f"\nK-fold Summary for {self.model_name}:")
        # tqdm.write(f"  Successful folds: {successful_folds}/{n_folds}")
        tqdm.write(f"  Total k-fold time: {total_kfold_time:.1f}s")
        # tqdm.write(f"  Average time per fold: {total_kfold_time/max(1,successful_folds):.1f}s")

        # return []
        # return fold_results



    def cleanup_trainer(self):
        """Complete cleanup of trainer resources"""
        try:
            if hasattr(self, 'model'):
                del self.model
            if hasattr(self, 'optimizer'):
                del self.optimizer
            if hasattr(self, 'scheduler'):
                del self.scheduler
            if hasattr(self, 'criterion'):
                del self.criterion
            if hasattr(self, 'scaler'):
                del self.scaler

            self.history.clear()
            self.resource_manager.aggressive_cleanup()

        except Exception as e:
            tqdm.write(f"Cleanup error: {e}")






# orig_numpy = torch.Tensor.numpy
# def patched_numpy(self, *args, **kwargs):
#     if self.is_cuda:
#         raise RuntimeError(f"Tried to call .numpy() on CUDA tensor at line ???. Move it to CPU first!")
#     return orig_numpy(self, *args, **kwargs)

# torch.Tensor.numpy = patched_numpy

class ResourceManager:
    """Smart resource management for optimal GPU/CPU utilization"""

    def __init__(self):
        self.gpu_memory_gb = 20
        self.cpu_memory_gb = 50
        self.max_gpu_usage = 0.85  # 85% of 20GB = 17GB
        self.max_cpu_usage = 0.90  # 80% of 50GB = 40GB

    def get_memory_stats(self):
        """Get current memory usage statistics"""
        stats = {'cpu_percent': psutil.virtual_memory().percent}

        if torch.cuda.is_available():
            stats['gpu_allocated_gb'] = torch.cuda.memory_allocated() / (1024**3)
            stats['gpu_reserved_gb'] = torch.cuda.memory_reserved() / (1024**3)
            stats['gpu_percent'] = (stats['gpu_reserved_gb'] / self.gpu_memory_gb) * 100
        else:
            stats.update({'gpu_allocated_gb': 0, 'gpu_reserved_gb': 0, 'gpu_percent': 0})

        return stats

    def should_cleanup_aggressive(self):
        """Check if aggressive cleanup is needed"""
        stats = self.get_memory_stats()
        return (stats['gpu_percent'] > 90 or stats['cpu_percent'] > 90)

    def aggressive_cleanup(self):
        """Perform comprehensive memory cleanup"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        gc.collect()
        time.sleep(0.1)  # Brief pause for system cleanup

    def optimize_batch_size(self, base_size, model_complexity=1.0):
        """Calculate optimal batch size based on current memory state"""
        stats = self.get_memory_stats()

        # Reduce batch size if memory usage is high
        memory_factor = max(0.4, 1.0 - (stats['gpu_percent'] / 100))
        optimal_size = int(base_size * memory_factor / model_complexity)

        return max(32, min(256, optimal_size))  # Keep within reasonable bounds


def safe_gpu_convert(data):
    """Safely convert GPU tensors to CPU numpy arrays or lists"""
    if torch.is_tensor(data):
        if data.is_cuda:
            data = data.detach().cpu()
        if data.dim() == 0:
            return data.item()
        elif data.dtype == torch.bool or data.dtype == torch.uint8:
            return data.numpy()
        else:
            return data.numpy() if data.dim() > 0 else data.item()
    elif isinstance(data, list):
        return [safe_gpu_convert(item) for item in data]
    elif isinstance(data, dict):
        return {k: safe_gpu_convert(v) for k, v in data.items()}
    elif isinstance(data, np.ndarray):
        return data  # Already numpy
    else:
        return data


# def safe_gpu_convert(data):
#     """Safely convert GPU tensors to CPU numpy arrays or lists"""
#     if torch.is_tensor(data):
#         if data.is_cuda:
#             data = data.detach().cpu()
#         if data.dim() == 0:
#             return data.item()
#         elif data.dtype == torch.bool or data.dtype == torch.uint8:
#             return data.numpy()
#         else:
#             return data.numpy() if data.dim() > 0 else data.item()
#     elif isinstance(data, list):
#         return [safe_gpu_convert(item) for item in data]
#     elif isinstance(data, dict):
#         converted = {}
#         for k, v in data.items():
#             # Special handling for data that should remain as lists for boolean evaluation
#             if k in ['misclassified', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'val_f1']:
#                 if torch.is_tensor(v):
#                     converted[k] = v.detach().cpu().numpy().tolist() if v.dim() > 0 else v.item()
#                 else:
#                     converted[k] = safe_gpu_convert(v)
#             else:
#                 converted[k] = safe_gpu_convert(v)
#         return converted
#     elif isinstance(data, np.ndarray):
#         return data  # Already numpy
#     else:
#         return data


def main():
    print("\nStarting Fish Species Classification Pipeline...")
    print("="*70)

    # Environment setup for maximum performance
    setup_environment()



    # Set optimal thread count for CPU utilization
    print(f"CPU : {os.cpu_count()}")
    print(f"USER: {os.getenv('USER')}")
    print(f"HOME: {os.getenv('HOME')}")
    torch.set_num_threads(min(16, os.cpu_count()))
    os.environ['OMP_NUM_THREADS'] = str(min(16, os.cpu_count()))



    # OpenMP → Open Multi-Processing.
    # OpenMP is a widely used API for parallel programming on CPUs.
    # Many scientific libraries (like NumPy, PyTorch, TensorFlow,
    # -OpenCV, Scikit-learn) rely on OpenMP to run operations in parallel across CPU cores.
    # In main() after line 720:
    torch.backends.cudnn.benchmark = True
    # Auto-benchmarks and selects fastest cuDNN algorithms for your specific hardware and input sizes
    # Takes ~5-10 seconds at start but can provide 10-20% speedup throughout training
    torch.backends.cudnn.deterministic = False
    # Allows cuDNN to use fastest available algorithms even if they produce slightly different results each run
    # Trades reproducibility for speed - essential for maximum performance in production training
    # torch.backends.cuda.matmul.allow_tf32 = True
    # Uses TensorFloat-32 (19-bit precision) instead of full Float-32 for matrix multiplications
    # Provides ~1.6x speedup on Ampere GPUs with negligible accuracy loss for deep learning
    # torch.backends.cudnn.allow_tf32 = True
    # Enables TF32 precision for cuDNN operations (convolutions, pooling, batch norm)
    # Accelerates CNN layers by 1.3-1.6x on RTX 30/40 series and A100+ GPUs
    torch.backends.cuda.enable_flash_sdp(True)
    # Activates Flash Attention algorithm for memory-efficient attention computations
    # Reduces GPU memory usage by 2-4x and increases attention speed, crucial for transformer layers



    # GPU optimizations
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.backends.cuda.matmul.allow_tf32 = True

        gpu_props = torch.cuda.get_device_properties(0)
        print(f"GPU: {gpu_props.name}, Memory: {gpu_props.total_memory / 1024**3:.1f}GB")

    # CPU info
    print(f"CPU Cores: {os.cpu_count()}, Using threads: {torch.get_num_threads()}")

    # # Ensure all output directories exist
    # os.makedirs(f"{Config.OUTPUT_DIR}/models", exist_ok=True)
    # os.makedirs(f"{Config.OUTPUT_DIR}/kfold_results", exist_ok=True)
    # os.makedirs(f"{Config.OUTPUT_DIR}/visualizations", exist_ok=True)
    # os.makedirs(f"{Config.OUTPUT_DIR}/model_results", exist_ok=True)
    # os.makedirs(f"{Config.OUTPUT_DIR}/best_model", exist_ok=True)

    # Initialize resource manager
    resource_manager = ResourceManager()

    # Load and balance data (only once)
    print("\nLoading and balancing data...")
    X, Y = DataManager.load_and_balance_data()
    print(f"Total samples after balancing: {len(X):,}, Labels: {len(Y):,}")

    # Validate data consistency
    if len(X) != len(Y):
        raise ValueError(f"Inconsistent data: X has {len(X)} samples, Y has {len(Y)} labels")
    if len(X) == 0:
        raise ValueError("No data available after loading and balancing")


    # Initialize components
    visualizer = EnhancedVisualizations()
    single_models = {}
    single_results = {}
    histories = {}

    # Process each model individually
    for model_name in Config.MODELS:
        print(f"\n{'='*70}")
        print(f"PROCESSING MODEL: {model_name}")
        print(f"{'='*70}")

        # Pre-model memory state
        pre_stats = resource_manager.get_memory_stats()
        print(f"Pre-training memory - GPU: {pre_stats['gpu_allocated_gb']:.2f}GB ({pre_stats['gpu_percent']:.1f}%), "
              f"CPU: {pre_stats['cpu_percent']:.1f}%")





        try:
            # Step 1: Create initial data loaders for hyperparameter optimization
            print(f"\n1. CREATING INITIAL DATA LOADERS FOR {model_name}")
            print("-"*50)

            train_loader, val_loader, test_loader, val_data, test_data = DataManager.create_data_loaders(X, Y)

            # Validate data loaders
            print(f"Train loader: {len(train_loader.dataset) if train_loader else 0} samples")
            print(f"Val loader: {len(val_loader.dataset) if val_loader else 0} samples")
            print(f"Test loader: {len(test_loader.dataset) if test_loader else 0} samples")

            if not train_loader or len(train_loader.dataset) == 0:
                print(f"Skipping {model_name}: No training data available")
                continue #To Go Next Model
            if not val_loader or len(val_loader.dataset) == 0:
                print(f"Skipping {model_name}: No validation data available")
                continue




            # Step 2: Optimize hyperparameters
            print(f"\n2. HYPERPARAMETER OPTIMIZATION FOR {model_name}")
            print("-"*50)

            optuna.logging.set_verbosity(optuna.logging.WARNING)
            torch.serialization.add_safe_globals([np.core.multiarray.scalar])
            torch.set_num_threads(2)  # Reduce CPU threads for GPU workload

            # Debug data loader sizes
            print(f"Train loader size: {len(train_loader.dataset)} samples, {len(train_loader)} batches")
            print(f"Val loader size: {len(val_loader.dataset)} samples, {len(val_loader)} batches")

            try:
                optimizer = ExpandedHyperparameterOptimizer(model_name, train_loader, val_loader)
                best_params = optimizer.optimize()
                del optimizer
                resource_manager.aggressive_cleanup()
            except Exception as e:
                print(f"Error in hyperparameter optimization for {model_name}: {str(e)}")
                import traceback
                # traceback.print_exc()
                traceback.format_exc()
                print(traceback.format_exc())
                #Or Log it
                tb_log = traceback.format_exc()
                logger.error(tb_log)
                best_params = {}  # Fallback to default hyperparameters

            print(f"\n\033[1;31m{'='*70}\033[0m")
            print(f"\033[1;31m{model_name.upper()} BEST PARAMETERS:\033[0m")
            for key, value in best_params.items():
                if key in ['lr', 'weight_decay', 'dropout', 'hidden_dim_multiplier', 'label_smoothing']:
                    print(f"\033[1;31m  {key}: {value:.4f}\033[0m")
                else:
                    print(f"\033[1;31m  {key}: {value}\033[0m")
            print(f"\033[1;31m{'='*70}\033[0m")





            # Step 3: Create data loaders with optimized parameters
            print(f"\n3. RECREATING DATA LOADERS WITH OPTIMIZED PARAMETERS")
            print("-"*50)

            # Clear previous loaders
            del train_loader, val_loader, test_loader
            resource_manager.aggressive_cleanup()

            # Calculate optimal batch size based on model complexity
            model_complexity_map = {
                'efficientnet_': 1.5,  # matches efficientnet_b0, efficientnet_b7 etc.
                'resnet': 1.0,         # matches resnet18, resnet50 etc.
                'vgg': 0.8,            # matches vgg16, vgg19 etc.
                'mobilenet_': 0.6,     # matches mobilenet_v2, mobilenet_v3 etc.
                'densenet': 1.3,       # matches densenet121, densenet201 etc.
                'convnext_': 1.4       # matches convnext_tiny, convnext_base etc.
            }

            model_complexity = model_complexity_map.get(model_name.split('_')[0].lower(), 1.0) #model_name gets from for loop.
            #"EfficientNet_B0".split('_')[0].lower() → "efficientnet"
            base_batch_size = best_params.get('batch_size', Config.BATCH_SIZE)
            optimized_batch_size = resource_manager.optimize_batch_size(base_batch_size, model_complexity)

            # Recreate with optimized batch size and GPU-optimized settings
            train_loader, val_loader, test_loader, val_data, test_data = DataManager.create_data_loaders(
                X, Y,
                test_size=0.2,
                batch_size=optimized_batch_size,
                augmentation_strength=best_params.get('augmentation_strength', 'medium')
            )

            # Optimize data loaders for GPU
            optimal_workers = min(16, max(8, os.cpu_count() // 2))
            for loader in [train_loader, val_loader, test_loader]:
                if loader:
                    loader.pin_memory = torch.cuda.is_available()
                    loader.num_workers = optimal_workers
                    loader.prefetch_factor = 4 if optimal_workers > 0 else 2

            # Validate recreated data loaders
            print(f"Recreated - Train: {len(train_loader.dataset)}, Val: {len(val_loader.dataset)}, Test: {len(test_loader.dataset)}")
            print(f"Optimized batch size: {optimized_batch_size} (complexity factor: {model_complexity})")

            # Additional validation for val_data and test_data tuples
            if val_data is not None and len(val_data) == 2:
                print(f"Val data tuple: X={len(val_data[0])}, Y={len(val_data[1])}")
                if len(val_data[0]) != len(val_data[1]):
                    print(f"WARNING: Validation data inconsistency: {len(val_data[0])} samples vs {len(val_data[1])} labels")

            if test_data is not None and len(test_data) == 2:
                print(f"Test data tuple: X={len(test_data[0])}, Y={len(test_data[1])}")
                if len(test_data[0]) != len(test_data[1]):
                    print(f"WARNING: Test data inconsistency: {len(test_data[0])} samples vs {len(test_data[1])} labels")





            # Step 4: Train main model
            print(f"\n4. MAIN MODEL TRAINING FOR {model_name}")
            print("-"*50)

            model = ModelFactory.create_model(
                model_name,
                num_classes=Config.NUM_CLASSES,
                dropout_rate=best_params.get('dropout', 0.5),
                hidden_dim_multiplier=best_params.get('hidden_dim_multiplier', 0.5)
            ).to(Config.DEVICE, memory_format=torch.channels_last)  # GPU optimization

            # Model info
            total_model_params = sum(p.numel() for p in model.parameters())
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            print(f"Model created: {total_model_params:,} total params, {trainable_params:,} trainable")

            # Additional GPU optimizations
            torch.backends.cudnn.benchmark = True
            torch.backends.cudnn.deterministic = False
            # Tells cuDNN (NVIDIA’s deep learning library) to find the fastest algorithm for your hardware and input size.
            # Benchmarking finds the fastest algorithm.
            # Non-determinism allows cuDNN to use even faster (but slightly variable) methods.
            # Together → maximum training speed, but with non-reproducible results (F1 score may vary slightly between runs).
            # For speed → benchmark=True, deterministic=False (your case).
            # For reproducibility → benchmark=False, deterministic=True.

            trainer = EnhancedModelTrainer(model, model_name, best_params)
            # history, result = trainer.train_main_model(train_loader, val_loader, test_loader)
            trainer.train_main_model(train_loader, val_loader, test_loader)

            # if history is None or result is None: #No Need cause Data is saved to Disk Directly
            #     print(f"Training failed for {model_name}, skipping...")
            #     continue

            print(f"Main model training completed for {model_name}")





            # Step 5: K-fold cross-validation
            print(f"\n5. K-FOLD CROSS-VALIDATION FOR {model_name}")
            print("-"*50)

            # Ensure we have sufficient data for k-fold
            total_samples = len(train_loader.dataset)
            min_samples_per_fold = 500
            max_folds = total_samples // min_samples_per_fold
            n_folds = min(3, max_folds) if max_folds > 1 else 0

            if n_folds > 1:
                print(f"Performing {n_folds}-fold cross-validation...")
                # fold_results =
                trainer.train_kfold(train_loader, val_loader, test_loader,n_folds=n_folds)
                print(f"K-fold validation completed for {model_name}")
            else:
                print(f"Skipping k-fold validation for {model_name}: insufficient data (need >{min_samples_per_fold*2} samples)")
                fold_results = []





            # Step 6: Generate ALL visualizations for this model
            print(f"\n6. GENERATING COMPLETE VISUALIZATIONS FOR {model_name}")
            print("-"*50)

            # Load saved results for plotting (ensuring consistency)
            main_result_path = f"{Config.OUTPUT_DIR}/model_results/{model_name}_main_results.pt"

            try:
                if os.path.exists(main_result_path):
                    saved_data = torch.load(main_result_path, map_location=Config.DEVICE, weights_only=False)
                    history_for_viz = saved_data['history']
                    result_for_viz = saved_data['result']
                    print(f"Using saved results for {model_name} visualization")
                else:
                    # # Use current result data for visualization
                    # history_for_viz = history
                    # result_for_viz = result
                    result_for_viz = {}
                    history_for_viz = {}
                    def add_strikethrough(text):
                        return ''.join(c + '\u0336' for c in text)
                    print(f"{add_strikethrough('Using current')} No results for {model_name} visualization")

                print(f"Result keys: {list(result_for_viz.keys()) if result_for_viz else 'None'}")

                # Convert all data to CPU/numpy for visualization (GPU-safe conversion)
                history_viz = safe_gpu_convert(history_for_viz)
                result_viz = safe_gpu_convert(result_for_viz)
                # history_viz = history_for_viz
                # result_viz = result_for_viz



                # Validate result data
                true_labels = result_viz.get('true_labels', np.array([]))
                predictions = result_viz.get('predictions', np.array([]))
                probabilities = result_viz.get('probabilities', np.array([]))

                # ADD THE CONVERSION LINES HERE:
                if torch.is_tensor(true_labels):
                    true_labels = true_labels.detach().cpu().numpy()
                if torch.is_tensor(predictions):
                    predictions = predictions.detach().cpu().numpy()
                if torch.is_tensor(probabilities):
                    probabilities = probabilities.detach().cpu().numpy()
                # ADD THESE ADDITIONAL CONVERSIONS:
                # Convert history data to safe formats
                for key in ['train_loss', 'train_acc', 'val_loss', 'val_acc', 'val_f1']:
                    if key in history_viz and torch.is_tensor(history_viz[key]):
                        history_viz[key] = history_viz[key].detach().cpu().numpy().tolist()




                print(f"Data validation for {model_name}:")
                print(f"  true_labels: {type(true_labels)}, shape: {getattr(true_labels, 'shape', len(true_labels) if hasattr(true_labels, '__len__') else 'scalar')}")
                print(f"  predictions: {type(predictions)}, shape: {getattr(predictions, 'shape', len(predictions) if hasattr(predictions, '__len__') else 'scalar')}")
                print(f"  probabilities: {type(probabilities)}, shape: {getattr(probabilities, 'shape', 'unknown')}")

                # Check if we have valid data for plotting
                if (true_labels.size > 0 and predictions.size > 0 and true_labels.shape[0] == predictions.shape[0]):
                # if (len(true_labels) > 0 and len(predictions) > 0 and len(true_labels) == len(predictions)):
                    plots_generated = 0

                    # Training history plot
                    # if history_viz.get('train_loss') and len(history_viz['train_loss']) > 0:
                    # CORRECTION:
                    train_loss = history_viz.get('train_loss')
                    if train_loss is not None and hasattr(train_loss, '__len__') and len(train_loss) > 0:
                        try:
                            # Add this conversion:
                            if history_viz.get('train_loss') is not None:
                                if isinstance(history_viz['train_loss'], np.ndarray):
                                    history_viz['train_loss'] = history_viz['train_loss'].tolist()
                            if history_viz.get('train_acc') is not None:
                                if isinstance(history_viz['train_acc'], np.ndarray):
                                    history_viz['train_acc'] = history_viz['train_acc'].tolist()
                            if history_viz.get('val_loss') is not None:
                                if isinstance(history_viz['val_loss'], np.ndarray):
                                    history_viz['val_loss'] = history_viz['val_loss'].tolist()
                            if history_viz.get('val_acc') is not None:
                                if isinstance(history_viz['val_acc'], np.ndarray):
                                    history_viz['val_acc'] = history_viz['val_acc'].tolist()

                            visualizer.plot_single_model_history(history_viz, model_name)
                            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_history.png", dpi=150, bbox_inches='tight')
                            plt.close('all')
                            print(f"Training history plot generated for {model_name}")
                            plots_generated += 1
                        except Exception as e:
                            print(f"Error plotting training history for {model_name}: {e}")

                    try:
                        # ROC Curves
                        visualizer.plot_roc_curves(result_viz, model_name)
                        plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_roc.png", dpi=150, bbox_inches='tight')
                        plt.close('all')
                        print(f"ROC curves plot generated for {model_name}")
                        plots_generated += 1

                        # Confusion Matrix
                        visualizer.plot_confusion_matrix(result_viz, model_name)
                        plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_confusion.png", dpi=150, bbox_inches='tight')
                        plt.close('all')
                        print(f"Confusion matrix plot generated for {model_name}")
                        plots_generated += 1

                        # Misclassified Images
                        # CORRECTION:
                        misclassified = result_viz.get('misclassified', [])
                        if misclassified is not None and len(misclassified) > 0:
                        # if result_viz.get('misclassified') and len(result_viz['misclassified']) > 0:
                            # Convert any GPU tensors in misclassified images to CPU
                            for item in result_viz['misclassified']:
                                if isinstance(item, dict) and 'image' in item:
                                    if torch.is_tensor(item['image']):
                                        item['image'] = item['image'].detach().cpu()
                                    elif isinstance(item['image'], np.ndarray):
                                        # Optionally transpose if channel-first
                                        if item['image'].shape[0] in [1,3]:
                                            item['image'] = np.transpose(item['image'], (1, 2, 0))

                            visualizer.plot_misclassified_images(result_viz['misclassified'], model_name)
                            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_misclassified.png", dpi=150, bbox_inches='tight')
                            plt.close('all')
                            print(f"Misclassified images plot generated for {model_name}")
                            plots_generated += 1
                        else:
                            print(f"No misclassified images to plot for {model_name}")

                        # XAI Visualization
                        if model is not None and test_loader is not None and len(test_loader.dataset) > 0:
                            try:
                                visualizer.plot_single_model_xai(model, model_name, test_loader)
                                plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_xai.png", dpi=150, bbox_inches='tight')
                                plt.close('all')
                                print(f"XAI visualization generated for {model_name}")
                                plots_generated += 1
                            except Exception as xai_error:
                                print(f"XAI visualization error for {model_name}: {xai_error}")
                        else:
                            print(f"Skipping XAI visualization for {model_name}: model or test_loader not available")

                    except Exception as plot_error:
                        print(f"Visualization error for {model_name}: {plot_error}")
                        import traceback
                        traceback.print_exc()

                    print(f"Generated {plots_generated} main visualization plots for {model_name}")

                else:
                    print(f"Skipping main visualizations for {model_name}: data validation failed")
                    # print(f"  true_labels length: {len(true_labels) if hasattr(true_labels, '__len__') else 'scalar'}")
                    # print(f"  predictions length: {len(predictions) if hasattr(predictions, '__len__') else 'scalar'}")
                    # CORRECTION:
                    print(f"  true_labels length: {true_labels.shape[0] if hasattr(true_labels, 'shape') else 'scalar'}")
                    print(f"  predictions length: {predictions.shape[0] if hasattr(predictions, 'shape') else 'scalar'}")



                # K-fold results plotting (load from saved files)
                if n_folds > 1:
                    print(f"Loading and plotting k-fold results for {model_name}...")
                    fold_results_loaded = []

                    for fold in range(1, n_folds + 1):
                        fold_result_path = f"{Config.OUTPUT_DIR}/kfold_results/{model_name}_fold_{fold}_results.pt"
                        if os.path.exists(fold_result_path):
                            try:
                                fold_data = torch.load(fold_result_path, map_location=Config.DEVICE, weights_only=False)

                                # Convert GPU tensors to CPU/numpy
                                fold_converted = {
                                    'history': safe_gpu_convert(fold_data['history']),
                                    'result': safe_gpu_convert(fold_data['result'])
                                }
                                fold_results_loaded.append(fold_converted)

                            except Exception as e:
                                print(f"Error loading fold {fold} results: {e}")

                    # if fold_results_loaded and len(fold_results_loaded) > 0:
                    # CORRECTION:
                    if fold_results_loaded is not None and len(fold_results_loaded) > 0:
                        try:
                            visualizer.plot_kfold_results(fold_results_loaded, model_name, test_loader)
                            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_kfold.png", dpi=150, bbox_inches='tight')
                            plt.close('all')
                            print(f"K-fold results plot generated for {model_name}")
                        except Exception as e:
                            print(f"Error plotting k-fold results for {model_name}: {e}")
                            import traceback
                            traceback.print_exc()
                    else:
                        print(f"No k-fold results to plot for {model_name}")

                print(f"All visualizations completed for {model_name}")

            except Exception as e:
                print(f"Error in visualization process for {model_name}: {e}")
                import traceback
                traceback.print_exc()






            # Step 7: Store results and save model
            print(f"\n7. STORING RESULTS FOR {model_name}")
            print("-"*50)

            # # Store results for later ensemble use
            # histories[model_name] = history
            # single_results[model_name] = result
            # single_models[model_name] = model

            # Save model for ensemble
            model_state_dict = model.state_dict()
            torch.save(model_state_dict, f"{Config.OUTPUT_DIR}/models/{model_name}_for_ensemble.pt")
            print(f"Model saved for ensemble: {model_name}")


            # Step 8: Complete memory cleanup for this model
            print(f"\n8. MEMORY CLEANUP FOR {model_name}")
            print("-"*50)

            # Clear all variables specific to this model
            trainer.cleanup_trainer()
            # del trainer, history, result, model_state_dict
            del train_loader, val_loader, test_loader

            # Clear visualization data
            if 'history_viz' in locals():
                del history_viz
            if 'result_viz' in locals():
                del result_viz
            if 'fold_results_loaded' in locals():
                del fold_results_loaded
            if 'history_for_viz' in locals():
                del history_for_viz
            if 'result_for_viz' in locals():
                del result_for_viz

            # Force garbage collection and GPU cleanup
            plt.close('all')
            resource_manager.aggressive_cleanup()


            # Post-model memory state
            post_stats = resource_manager.get_memory_stats()
            memory_freed = pre_stats['gpu_allocated_gb'] - post_stats['gpu_allocated_gb']

            print(f"Memory cleanup completed for {model_name}:")
            print(f"  GPU memory freed: {memory_freed:.2f}GB")
            print(f"  Current GPU usage: {post_stats['gpu_percent']:.1f}%")
            print(f"  Current CPU usage: {post_stats['cpu_percent']:.1f}%")

            print(f"✓ {model_name} PROCESSING COMPLETED!")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"Error processing {model_name}: {e}")
            import traceback
            traceback.print_exc()

            # Emergency cleanup
            try:
                if 'trainer' in locals():
                    trainer.cleanup_trainer()
                    del trainer
                if 'model' in locals():
                    del model
                resource_manager.aggressive_cleanup()
            except:
                pass
            continue





    single_models = {}
    single_results = {}

    def load_model_safely(model_name, model_path):
        """Safely load model with multiple fallback strategies"""
        try:
            checkpoint = torch.load(model_path, map_location=Config.DEVICE, weights_only=False)

            if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
                # New comprehensive format - use saved architecture info
                model = ModelFactory.create_model(
                    model_name,
                    num_classes=checkpoint.get('num_classes', Config.NUM_CLASSES),
                    dropout_rate=checkpoint.get('dropout_rate', 0.5),
                    hidden_dim_multiplier=checkpoint.get('hidden_dim_multiplier', 0.5)
                )
                model.to(Config.DEVICE)
                model.load_state_dict(checkpoint['state_dict'])
                return model, f"new format v{checkpoint.get('save_format_version', '1.0')}"

            elif isinstance(checkpoint, dict):
                # Legacy format - try with hyperparameters
                hyperparams = checkpoint.get('hyperparameters', {})
                model = ModelFactory.create_model(
                    model_name,
                    num_classes=Config.NUM_CLASSES,
                    dropout_rate=hyperparams.get('dropout', 0.5),
                    hidden_dim_multiplier=hyperparams.get('hidden_dim_multiplier', 0.5)
                )
                model.to(Config.DEVICE)
                model.load_state_dict(checkpoint)
                return model, "legacy nested format"

            else:
                # Try direct loading with parameter combinations
                param_combinations = [
                    (0.5, 0.5), (0.3, 0.5), (0.7, 0.5),
                    (0.5, 0.3), (0.5, 0.7), (0.5, 1.0), (0.5, 1.5)
                ]

                for dropout, hidden_mult in param_combinations:
                    try:
                        model = ModelFactory.create_model(
                            model_name,
                            num_classes=Config.NUM_CLASSES,
                            dropout_rate=dropout,
                            hidden_dim_multiplier=hidden_mult
                        )
                        model.to(Config.DEVICE)
                        model.load_state_dict(checkpoint)
                        return model, f"direct format (dropout={dropout}, hidden={hidden_mult})"
                    except RuntimeError:
                        continue

                return None, "failed all combinations"

        except Exception as e:
            return None, f"loading error: {str(e)}"

    # Load models with enhanced error handling
    for model_name in Config.MODELS:
        model_path = os.path.join(Config.OUTPUT_DIR, "best_model", f"{model_name}_best.pt")

        if not os.path.exists(model_path):
            print(f"⚠️  Model file not found: {model_name}")
            continue

        model, load_info = load_model_safely(model_name, model_path)

        if model is not None:
            model.eval()
            single_models[model_name] = model
            print(f"✅ Loaded {model_name} ({load_info})")

            # Load evaluation results for visualization compatibility
            result_path = f"{Config.OUTPUT_DIR}/model_results/{model_name}_main_results.pt"
            if os.path.exists(result_path):
                try:
                    saved_results = torch.load(result_path, map_location='cpu', weights_only=False)
                    result_data = saved_results.get('result', {})

                    single_results[model_name] = {
                        'model_name': model_name,
                        'accuracy': float(result_data.get('accuracy', 0.0)),
                        'f1_macro': float(result_data.get('f1', 0.0)),
                        'f1_weighted': float(result_data.get('f1', 0.0)),  # Add f1_weighted key
                        'predictions': result_data.get('predictions', []),
                        'true_labels': result_data.get('true_labels', []),
                        'probabilities': result_data.get('probabilities', np.zeros((0, Config.NUM_CLASSES))),
                        'conf_matrix': result_data.get('conf_matrix', np.zeros((Config.NUM_CLASSES, Config.NUM_CLASSES))),
                        'misclassified': result_data.get('misclassified', [])
                    }

                    # Convert tensors to numpy for visualization compatibility
                    for key in ['predictions', 'true_labels', 'probabilities', 'conf_matrix']:
                        if torch.is_tensor(single_results[model_name][key]):
                            single_results[model_name][key] = single_results[model_name][key].numpy()

                    print(f"📊 Results loaded: Acc={single_results[model_name]['accuracy']:.4f}")

                except Exception as e:
                    print(f"⚠️  Could not load results for {model_name}: {e}")
                    single_results[model_name] = {'model_name': model_name, 'accuracy': 0.0, 'f1_macro': 0.0, 'f1_weighted': 0.0}
            else:
                print(f"⚠️  No results file for {model_name}")
                single_results[model_name] = {'model_name': model_name, 'accuracy': 0.0, 'f1_macro': 0.0, 'f1_weighted': 0.0}
        else:
            print(f"❌ Failed to load {model_name}: {load_info}")

    print(f"\n🎯 Ensemble ready: {len(single_models)} models, {len(single_results)} result sets")




    # Step: Ensemble processing (only if we have trained models)
    if single_models:
        print("\n" + "="*70)
        print("ALL MODELS PROCESSED - STARTING ENSEMBLE ANALYSIS")
        print("="*70)

        try:
            if (val_data is not None and len(val_data) == 2 and len(val_data[0]) > 0 and len(val_data[1]) > 0 and len(val_data[0]) == len(val_data[1])):

                ensemble_manager = EnsembleManager(single_models, val_data)
                ensemble_results, best_ensemble = ensemble_manager.test_ensemble_combinations()

                # Generate visualizations for ensemble methods
                print("\nGenerating visualizations for ensemble methods...")
                for ensemble_name, ensemble_result in ensemble_results.items():
                    try:
                        # Convert GPU tensors to CPU for visualization
                        ensemble_result_viz = safe_gpu_convert(ensemble_result)

                        true_labels_ens = ensemble_result_viz.get('true_labels', [])
                        predictions_ens = ensemble_result_viz.get('predictions', [])

                        if (len(true_labels_ens) > 0 and
                            len(predictions_ens) > 0 and
                            len(true_labels_ens) == len(predictions_ens)):

                            visualizer.plot_roc_curves(ensemble_result_viz, ensemble_name)
                            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{ensemble_name}_roc.png", dpi=150, bbox_inches='tight')
                            plt.close('all')

                            visualizer.plot_confusion_matrix(ensemble_result_viz, ensemble_name)
                            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{ensemble_name}_confusion.png", dpi=150, bbox_inches='tight')
                            plt.close('all')

                            # F1 per class visualization
                            from sklearn.metrics import f1_score
                            f1_per_class = f1_score(ensemble_result_viz['true_labels'],
                                                   ensemble_result_viz['predictions'], average=None)
                            fig, ax = plt.subplots(figsize=(10, 6))
                            ax.bar(range(Config.NUM_CLASSES), f1_per_class, color='lightgreen', alpha=0.8)
                            ax.set_xticks(range(Config.NUM_CLASSES))
                            ax.set_xticklabels(Config.CLASS_LABELS, rotation=45, ha='right')
                            ax.set_title(f'F1 Scores per Class - {ensemble_name}',
                                       fontsize=14, fontweight='bold', pad=15)
                            ax.set_xlabel('Class', fontsize=12, fontweight='bold')
                            ax.set_ylabel('F1 Score', fontsize=12, fontweight='bold')
                            ax.grid(True, alpha=0.3)
                            for i, v in enumerate(f1_per_class):
                                ax.text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom', fontsize=10)
                            save_path = f"{Config.OUTPUT_DIR}/visualizations/{ensemble_name}_f1_per_class.png"
                            plt.savefig(save_path, dpi=300, bbox_inches='tight',
                                      facecolor='white', edgecolor='none')
                            plt.close()
                            print(f"F1 scores per class saved: {save_path}")
                        else:
                            print(f"Skipping visualizations for {ensemble_name}: invalid data")
                    except Exception as e:
                        print(f"Error generating ensemble visualizations for {ensemble_name}: {e}")

                    # Clear memory
                    if 'ensemble_result_viz' in locals():
                        del ensemble_result_viz
                    resource_manager.aggressive_cleanup()

                # Comment out the problematic ensemble test evaluation
                print("Skipping ensemble test evaluation (method not implemented)")
                # if (best_ensemble and test_data is not None and len(test_data) == 2 and
                #     len(test_data[0]) == len(test_data[1]) and len(test_data[0]) > 0):
                #     print("Evaluating best ensemble on test set...")
                #     # Ensemble test evaluation code commented out

            else:
                print("Skipping ensemble analysis: no valid validation data")

        except Exception as e:
            print(f"Error in ensemble processing: {e}")
            import traceback
            traceback.print_exc()




    # Step 10: Generate 4 comprehensive visualizations (only if we have results)
    if single_results:
        print("\n" + "="*70)
        print("GENERATING 4 COMPREHENSIVE ANALYSIS")
        print("="*70)

        try:
            # Convert all results to CPU/numpy for final plotting
            single_results_viz = safe_gpu_convert(single_results)
            ensemble_results_viz = safe_gpu_convert(ensemble_results) if 'ensemble_results' in locals() else {}

            # Plot overall model comparison
            if single_results_viz and len(single_results_viz) > 0:
                visualizer.plot_model_comparison(single_results_viz,ensemble_results_viz)
                plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/model_comparison.png", dpi=300, bbox_inches='tight')
                plt.close('all')

            # Generate comprehensive report
            visualizer.generate_comprehensive_report(
                single_results_viz,
                ensemble_results_viz,
                best_ensemble if 'best_ensemble' in locals() else None
            )
            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/comprehensive_report.png", dpi=300, bbox_inches='tight')
            plt.close('all')

            # Additional visualizations
            if 'test_loader' in locals() and test_loader is not None and len(test_loader.dataset) > 0:
                try:
                    visualizer.plot_comparative_xai(
                        single_models,
                        ensemble_results_viz,
                        test_loader,
                        max_images=2
                    )
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/comparative_xai.png", dpi=300, bbox_inches='tight')
                    plt.close('all')

                    visualizer.plot_lrp_grid(single_models, test_loader)
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/lrp_grid.png", dpi=300, bbox_inches='tight')
                    plt.close('all')
                except Exception as e:
                    print(f"Error in additional visualizations: {e}")

            print("Comprehensive analysis completed successfully")

        except Exception as e:
            print(f"Error in comprehensive analysis: {e}")
            import traceback
            traceback.print_exc()


    # Final cleanup
    print("\nFinal cleanup and summary...")
    try:
        # Clean up all remaining variables
        if 'single_models' in locals():
            del single_models
        if 'test_loader' in locals():
            del test_loader
        if 'histories' in locals():
            del histories
        if 'single_results' in locals():
            del single_results
        if 'single_results_viz' in locals():
            del single_results_viz
        if 'ensemble_results' in locals():
            del ensemble_results
        if 'ensemble_results_viz' in locals():
            del ensemble_results_viz
        if 'best_ensemble' in locals():
            del best_ensemble
        if 'visualizer' in locals():
            del visualizer

        resource_manager.aggressive_cleanup()

        # Final memory stats
        final_stats = resource_manager.get_memory_stats()
        print(f"Final GPU memory: {final_stats['gpu_allocated_gb']:.2f}GB allocated ({final_stats['gpu_percent']:.1f}%)")
        print(f"Final CPU usage: {final_stats['cpu_percent']:.1f}%")

    except Exception as e:
        print(f"Error in final cleanup: {e}")

    print("\n" + "="*70)
    print("PIPELINE COMPLETED!")
    print("="*70)
    print("\nGenerated Files:")
    print(f"- Model checkpoints: {Config.OUTPUT_DIR}/models/")
    print(f"- Training results: {Config.OUTPUT_DIR}/model_results/")
    print(f"- K-fold results: {Config.OUTPUT_DIR}/kfold_results/")
    print(f"- Visualizations: {Config.OUTPUT_DIR}/visualizations/")
    print(f"- Best models: {Config.OUTPUT_DIR}/best_model/")


# Environment setup and multiprocessing configuration
import torch.multiprocessing as mp
if __name__ == "__main__":
    # # Environment setup for maximum performance with 20GB GPU + 50GB CPU
    # os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
    # os.environ['OMP_NUM_THREADS'] = '16'  # Max CPU utilization
    # os.environ['MKL_NUM_THREADS'] = '16'
    # os.environ['NUMEXPR_NUM_THREADS'] = '16'

    # # Additional GPU optimizations
    # if torch.cuda.is_available():
    #     torch.backends.cudnn.enabled = True
    #     torch.backends.cudnn.benchmark = True
    #     torch.backends.cudnn.deterministic = False
    #     torch.backends.cuda.matmul.allow_tf32 = True
    #     torch.backends.cudnn.allow_tf32 = True

    # Multiprocessing setup for CUDA compatibility
    # try:
        # import torch.multiprocessing as mp
    #     mp.set_start_method('spawn', force=True)
    # except RuntimeError:
    #     pass  # Already set

    # print("Environment optimizations applied:")
    # print(f"CUDNN Enabled: {torch.backends.cudnn.enabled}")
    # print(f"CUDNN Benchmark: {torch.backends.cudnn.benchmark}")
    # print(f"Allow TF32 Matmul: {torch.backends.cuda.matmul.allow_tf32}")
    # print(f"OMP Threads: {os.environ.get('OMP_NUM_THREADS', 'default')}")

    main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 7.2 MB/s eta 0:00:00



# Optuna Trial
==================================================

In [ ]:
"""
13.Fish Species Classification Pipeline - Hyperparameter Optimization
===================================================================================================================

This module handles hyperparameter optimization using Optuna for all models.
It will find the best parameters for each model and save them to JSON files.
"""

import os
import json
import torch
import optuna
import numpy as np
import gc
import time
import psutil
import warnings
warnings.filterwarnings('ignore')

# orig_numpy = torch.Tensor.numpy
# def patched_numpy(self, *args, **kwargs):
#     if self.is_cuda:
#         raise RuntimeError(f"Tried to call .numpy() on CUDA tensor at line ???. Move it to CPU first!")
#     return orig_numpy(self, *args, **kwargs)

# torch.Tensor.numpy = patched_numpy

class ResourceManager:
    """Smart resource management for optimal GPU/CPU utilization"""

    def __init__(self):
        self.gpu_memory_gb = 20
        self.cpu_memory_gb = 50
        self.max_gpu_usage = 0.85  # 85% of 20GB = 17GB
        self.max_cpu_usage = 0.90  # 80% of 50GB = 40GB

    def get_memory_stats(self):
        """Get current memory usage statistics"""
        stats = {'cpu_percent': psutil.virtual_memory().percent}

        if torch.cuda.is_available():
            stats['gpu_allocated_gb'] = torch.cuda.memory_allocated() / (1024**3)
            stats['gpu_reserved_gb'] = torch.cuda.memory_reserved() / (1024**3)
            stats['gpu_percent'] = (stats['gpu_reserved_gb'] / self.gpu_memory_gb) * 100
        else:
            stats.update({'gpu_allocated_gb': 0, 'gpu_reserved_gb': 0, 'gpu_percent': 0})

        return stats

    def should_cleanup_aggressive(self):
        """Check if aggressive cleanup is needed"""
        stats = self.get_memory_stats()
        return (stats['gpu_percent'] > 90 or stats['cpu_percent'] > 90)

    def aggressive_cleanup(self):
        """Perform comprehensive memory cleanup"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        gc.collect()
        time.sleep(0.1)  # Brief pause for system cleanup

    def optimize_batch_size(self, base_size, model_complexity=1.0):
        """Calculate optimal batch size based on current memory state"""
        stats = self.get_memory_stats()

        # Reduce batch size if memory usage is high
        memory_factor = max(0.4, 1.0 - (stats['gpu_percent'] / 100))
        optimal_size = int(base_size * memory_factor / model_complexity)

        return max(32, min(256, optimal_size))  # Keep within reasonable bounds


def safe_gpu_convert(data):
    """Safely convert GPU tensors to CPU numpy arrays or lists"""
    if torch.is_tensor(data):
        if data.is_cuda:
            data = data.detach().cpu()
        if data.dim() == 0:
            return data.item()
        elif data.dtype == torch.bool or data.dtype == torch.uint8:
            return data.numpy()
        else:
            return data.numpy() if data.dim() > 0 else data.item()
    elif isinstance(data, list):
        return [safe_gpu_convert(item) for item in data]
    elif isinstance(data, dict):
        return {k: safe_gpu_convert(v) for k, v in data.items()}
    elif isinstance(data, np.ndarray):
        return data  # Already numpy
    else:
        return data


def main():
    print("\nStarting Fish Species Hyperparameter Optimization...")
    print("="*70)

    # Environment setup for maximum performance
    setup_environment()


    # Set optimal thread count for CPU utilization
    print(f"CPU : {os.cpu_count()}")
    print(f"USER: {os.getenv('USER')}")
    print(f"HOME: {os.getenv('HOME')}")
    torch.set_num_threads(min(16, os.cpu_count()))
    os.environ['OMP_NUM_THREADS'] = str(min(16, os.cpu_count()))

    # OpenMP → Open Multi-Processing.
    # OpenMP is a widely used API for parallel programming on CPUs.
    # Many scientific libraries (like NumPy, PyTorch, TensorFlow,
    # -OpenCV, Scikit-learn) rely on OpenMP to run operations in parallel across CPU cores.
    # In main() after line 720:
    torch.backends.cudnn.benchmark = True
    # Auto-benchmarks and selects fastest cuDNN algorithms for your specific hardware and input sizes
    # Takes ~5-10 seconds at start but can provide 10-20% speedup throughout training
    torch.backends.cudnn.deterministic = False
    # Allows cuDNN to use fastest available algorithms even if they produce slightly different results each run
    # Trades reproducibility for speed - essential for maximum performance in production training
    # torch.backends.cuda.matmul.allow_tf32 = True
    # Uses TensorFloat-32 (19-bit precision) instead of full Float-32 for matrix multiplications
    # Provides ~1.6x speedup on Ampere GPUs with negligible accuracy loss for deep learning
    # torch.backends.cudnn.allow_tf32 = True
    # Enables TF32 precision for cuDNN operations (convolutions, pooling, batch norm)
    # Accelerates CNN layers by 1.3-1.6x on RTX 30/40 series and A100+ GPUs
    torch.backends.cuda.enable_flash_sdp(True)
    # Activates Flash Attention algorithm for memory-efficient attention computations
    # Reduces GPU memory usage by 2-4x and increases attention speed, crucial for transformer layers

    # GPU optimizations
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.backends.cuda.matmul.allow_tf32 = True

        gpu_props = torch.cuda.get_device_properties(0)
        print(f"GPU: {gpu_props.name}, Memory: {gpu_props.total_memory / 1024**3:.1f}GB")

    # CPU info
    print(f"CPU Cores: {os.cpu_count()}, Using threads: {torch.get_num_threads()}")

    # Ensure all output directories exist
    os.makedirs(f"{Config.OUTPUT_DIR}/hyperparameters", exist_ok=True)

    # Initialize resource manager
    resource_manager = ResourceManager()

    # Load and balance data (only once)
    print("\nLoading and balancing data...")
    X, Y = DataManager.load_and_balance_data()
    print(f"Total samples after balancing: {len(X):,}, Labels: {len(Y):,}")

    # Validate data consistency
    if len(X) != len(Y):
        raise ValueError(f"Inconsistent data: X has {len(X)} samples, Y has {len(Y)} labels")
    if len(X) == 0:
        raise ValueError("No data available after loading and balancing")

    # Dictionary to store all best parameters
    all_best_params = {}

    # Process each model individually for hyperparameter optimization
    for model_name in Config.MODELS:
        print(f"\n{'='*70}")
        print(f"HYPERPARAMETER OPTIMIZATION FOR: {model_name}")
        print(f"{'='*70}")

        # Pre-model memory state
        pre_stats = resource_manager.get_memory_stats()
        print(f"Pre-training memory - GPU: {pre_stats['gpu_allocated_gb']:.2f}GB ({pre_stats['gpu_percent']:.1f}%), "
              f"CPU: {pre_stats['cpu_percent']:.1f}%")

        try:
            # Step 1: Create initial data loaders for hyperparameter optimization
            print(f"\n1. CREATING INITIAL DATA LOADERS FOR {model_name}")
            print("-"*50)

            train_loader, val_loader, test_loader, val_data, test_data = DataManager.create_data_loaders(X, Y)

            # Validate data loaders
            print(f"Train loader: {len(train_loader.dataset) if train_loader else 0} samples")
            print(f"Val loader: {len(val_loader.dataset) if val_loader else 0} samples")
            print(f"Test loader: {len(test_loader.dataset) if test_loader else 0} samples")

            if not train_loader or len(train_loader.dataset) == 0:
                print(f"Skipping {model_name}: No training data available")
                continue #To Go Next Model
            if not val_loader or len(val_loader.dataset) == 0:
                print(f"Skipping {model_name}: No validation data available")
                continue

            # Step 2: Optimize hyperparameters
            print(f"\n2. HYPERPARAMETER OPTIMIZATION FOR {model_name}")
            print("-"*50)

            optuna.logging.set_verbosity(optuna.logging.WARNING)
            torch.serialization.add_safe_globals([np.core.multiarray.scalar])
            torch.set_num_threads(2)  # Reduce CPU threads for GPU workload

            # Debug data loader sizes
            print(f"Train loader size: {len(train_loader.dataset)} samples, {len(train_loader)} batches")
            print(f"Val loader size: {len(val_loader.dataset)} samples, {len(val_loader)} batches")

            try:
                optimizer = ExpandedHyperparameterOptimizer(model_name, train_loader, val_loader)
                best_params = optimizer.optimize()
                all_best_params[model_name] = best_params
                del optimizer
                resource_manager.aggressive_cleanup()
            except Exception as e:
                print(f"Error in hyperparameter optimization for {model_name}: {str(e)}")
                import traceback
                # traceback.print_exc()
                traceback.format_exc()
                print(traceback.format_exc())
                #Or Log it
                tb_log = traceback.format_exc()
                # logger.error(tb_log)
                best_params = {}  # Fallback to default hyperparameters
                all_best_params[model_name] = best_params

            print(f"\n\033[1;31m{'='*70}\033[0m")
            print(f"\033[1;31m{model_name.upper()} BEST PARAMETERS:\033[0m")
            for key, value in best_params.items():
                if key in ['lr', 'weight_decay', 'dropout', 'hidden_dim_multiplier', 'label_smoothing']:
                    print(f"\033[1;31m  {key}: {value:.4f}\033[0m")
                else:
                    print(f"\033[1;31m  {key}: {value}\033[0m")
            print(f"\033[1;31m{'='*70}\033[0m")

            # Save best parameters to JSON file
            params_file = f"{Config.OUTPUT_DIR}/hyperparameters/{model_name}_best_params.json"
            with open(params_file, 'w') as f:
                json.dump(best_params, f, indent=4)
            print(f"Best parameters saved to: {params_file}")

            # Memory cleanup
            del train_loader, val_loader, test_loader
            resource_manager.aggressive_cleanup()

            # Post-model memory state
            post_stats = resource_manager.get_memory_stats()
            memory_freed = pre_stats['gpu_allocated_gb'] - post_stats['gpu_allocated_gb']

            print(f"Memory cleanup completed for {model_name}:")
            print(f"  GPU memory freed: {memory_freed:.2f}GB")
            print(f"  Current GPU usage: {post_stats['gpu_percent']:.1f}%")
            print(f"  Current CPU usage: {post_stats['cpu_percent']:.1f}%")

            print(f"✓ {model_name} HYPERPARAMETER OPTIMIZATION COMPLETED!")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"Error processing {model_name}: {e}")
            import traceback
            traceback.print_exc()

            # Emergency cleanup
            try:
                if 'optimizer' in locals():
                    del optimizer
                resource_manager.aggressive_cleanup()
            except:
                pass
            continue

    # Save all best parameters to master file
    master_params_file = f"{Config.OUTPUT_DIR}/hyperparameters/all_best_params.json"
    with open(master_params_file, 'w') as f:
        json.dump(all_best_params, f, indent=4)

    print(f"All best parameters saved to: {master_params_file}")

    # Final cleanup
    print("\nFinal cleanup and summary...")
    try:
        # Clean up all remaining variables
        if 'all_best_params' in locals():
            del all_best_params

        resource_manager.aggressive_cleanup()

        # Final memory stats
        final_stats = resource_manager.get_memory_stats()
        print(f"Final GPU memory: {final_stats['gpu_allocated_gb']:.2f}GB allocated ({final_stats['gpu_percent']:.1f}%)")
        print(f"Final CPU usage: {final_stats['cpu_percent']:.1f}%")

    except Exception as e:
        print(f"Error in final cleanup: {e}")

    print("\n" + "="*70)
    print("HYPERPARAMETER OPTIMIZATION COMPLETED!")
    print("="*70)
    print("\nGenerated Files:")
    print(f"- Best parameters: {Config.OUTPUT_DIR}/hyperparameters/")


# Environment setup and multiprocessing configuration
import torch.multiprocessing as mp
if __name__ == "__main__":
    main()

#Model Training

In [ ]:
"""
14.Fish Species Classification Pipeline - Model Training
==================================================

This module handles the main model training using optimized hyperparameters.
It loads the best parameters from JSON files and trains models, saving them as .pt and .keras files.
"""

import os
import json
import torch
import numpy as np
import gc
import time
import psutil
import warnings
warnings.filterwarnings('ignore')

# orig_numpy = torch.Tensor.numpy
# def patched_numpy(self, *args, **kwargs):
#     if self.is_cuda:
#         raise RuntimeError(f"Tried to call .numpy() on CUDA tensor at line ???. Move it to CPU first!")
#     return orig_numpy(self, *args, **kwargs)

# torch.Tensor.numpy = patched_numpy

class ResourceManager:
    """Smart resource management for optimal GPU/CPU utilization"""

    def __init__(self):
        self.gpu_memory_gb = 20
        self.cpu_memory_gb = 50
        self.max_gpu_usage = 0.85  # 85% of 20GB = 17GB
        self.max_cpu_usage = 0.90  # 80% of 50GB = 40GB

    def get_memory_stats(self):
        """Get current memory usage statistics"""
        stats = {'cpu_percent': psutil.virtual_memory().percent}

        if torch.cuda.is_available():
            stats['gpu_allocated_gb'] = torch.cuda.memory_allocated() / (1024**3)
            stats['gpu_reserved_gb'] = torch.cuda.memory_reserved() / (1024**3)
            stats['gpu_percent'] = (stats['gpu_reserved_gb'] / self.gpu_memory_gb) * 100
        else:
            stats.update({'gpu_allocated_gb': 0, 'gpu_reserved_gb': 0, 'gpu_percent': 0})

        return stats

    def should_cleanup_aggressive(self):
        """Check if aggressive cleanup is needed"""
        stats = self.get_memory_stats()
        return (stats['gpu_percent'] > 90 or stats['cpu_percent'] > 90)

    def aggressive_cleanup(self):
        """Perform comprehensive memory cleanup"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        gc.collect()
        time.sleep(0.1)  # Brief pause for system cleanup

    def optimize_batch_size(self, base_size, model_complexity=1.0):
        """Calculate optimal batch size based on current memory state"""
        stats = self.get_memory_stats()

        # Reduce batch size if memory usage is high
        memory_factor = max(0.4, 1.0 - (stats['gpu_percent'] / 100))
        optimal_size = int(base_size * memory_factor / model_complexity)

        return max(32, min(256, optimal_size))  # Keep within reasonable bounds


def safe_gpu_convert(data):
    """Safely convert GPU tensors to CPU numpy arrays or lists"""
    if torch.is_tensor(data):
        if data.is_cuda:
            data = data.detach().cpu()
        if data.dim() == 0:
            return data.item()
        elif data.dtype == torch.bool or data.dtype == torch.uint8:
            return data.numpy()
        else:
            return data.numpy() if data.dim() > 0 else data.item()
    elif isinstance(data, list):
        return [safe_gpu_convert(item) for item in data]
    elif isinstance(data, dict):
        return {k: safe_gpu_convert(v) for k, v in data.items()}
    elif isinstance(data, np.ndarray):
        return data  # Already numpy
    else:
        return data


def main():
    print("\nStarting Fish Species Model Training...")
    print("="*70)

    # Environment setup for maximum performance
    setup_environment()


    # Set optimal thread count for CPU utilization
    print(f"CPU : {os.cpu_count()}")
    print(f"USER: {os.getenv('USER')}")
    print(f"HOME: {os.getenv('HOME')}")
    torch.set_num_threads(min(16, os.cpu_count()))
    os.environ['OMP_NUM_THREADS'] = str(min(16, os.cpu_count()))

    # OpenMP → Open Multi-Processing.
    # OpenMP is a widely used API for parallel programming on CPUs.
    # Many scientific libraries (like NumPy, PyTorch, TensorFlow,
    # -OpenCV, Scikit-learn) rely on OpenMP to run operations in parallel across CPU cores.
    # In main() after line 720:
    torch.backends.cudnn.benchmark = True
    # Auto-benchmarks and selects fastest cuDNN algorithms for your specific hardware and input sizes
    # Takes ~5-10 seconds at start but can provide 10-20% speedup throughout training
    torch.backends.cudnn.deterministic = False
    # Allows cuDNN to use fastest available algorithms even if they produce slightly different results each run
    # Trades reproducibility for speed - essential for maximum performance in production training
    # torch.backends.cuda.matmul.allow_tf32 = True
    # Uses TensorFloat-32 (19-bit precision) instead of full Float-32 for matrix multiplications
    # Provides ~1.6x speedup on Ampere GPUs with negligible accuracy loss for deep learning
    # torch.backends.cudnn.allow_tf32 = True
    # Enables TF32 precision for cuDNN operations (convolutions, pooling, batch norm)
    # Accelerates CNN layers by 1.3-1.6x on RTX 30/40 series and A100+ GPUs
    torch.backends.cuda.enable_flash_sdp(True)
    # Activates Flash Attention algorithm for memory-efficient attention computations
    # Reduces GPU memory usage by 2-4x and increases attention speed, crucial for transformer layers

    # GPU optimizations
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.backends.cuda.matmul.allow_tf32 = True

        gpu_props = torch.cuda.get_device_properties(0)
        print(f"GPU: {gpu_props.name}, Memory: {gpu_props.total_memory / 1024**3:.1f}GB")

    # CPU info
    print(f"CPU Cores: {os.cpu_count()}, Using threads: {torch.get_num_threads()}")

    # Ensure all output directories exist
    os.makedirs(f"{Config.OUTPUT_DIR}/models", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/kfold_results", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/model_results", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/best_model", exist_ok=True)

    # Initialize resource manager
    resource_manager = ResourceManager()

    # Load and balance data (only once)
    print("\nLoading and balancing data...")
    X, Y = DataManager.load_and_balance_data()
    print(f"Total samples after balancing: {len(X):,}, Labels: {len(Y):,}")

    # Validate data consistency
    if len(X) != len(Y):
        raise ValueError(f"Inconsistent data: X has {len(X)} samples, Y has {len(Y)} labels")
    if len(X) == 0:
        raise ValueError("No data available after loading and balancing")

    # Load all best parameters from hyperparameter optimization
    master_params_file = f"{Config.OUTPUT_DIR}/hyperparameters/all_best_params.json"
    if os.path.exists(master_params_file):
        with open(master_params_file, 'r') as f:
            all_best_params = json.load(f)
        print(f"Loaded best parameters for {len(all_best_params)} models")
    else:
        print("No hyperparameters found, using default parameters")
        all_best_params = {}

    # Process each model individually
    for model_name in Config.MODELS:
        print(f"\n{'='*70}")
        print(f"TRAINING MODEL: {model_name}")
        print(f"{'='*70}")

        # Pre-model memory state
        pre_stats = resource_manager.get_memory_stats()
        print(f"Pre-training memory - GPU: {pre_stats['gpu_allocated_gb']:.2f}GB ({pre_stats['gpu_percent']:.1f}%), "
              f"CPU: {pre_stats['cpu_percent']:.1f}%")

        try:
            # Load best parameters for this model
            if model_name in all_best_params:
                best_params = all_best_params[model_name]
                print(f"Using optimized parameters for {model_name}")
            else:
                # Load individual parameter file as fallback
                params_file = f"{Config.OUTPUT_DIR}/hyperparameters/{model_name}_best_params.json"
                if os.path.exists(params_file):
                    with open(params_file, 'r') as f:
                        best_params = json.load(f)
                    print(f"Loaded individual parameters for {model_name}")
                else:
                    # Default parameters as last resort
                    best_params = {
                        'lr': 0.001,
                        'weight_decay': 0.0001,
                        'dropout': 0.5,
                        'batch_size': 32,
                        'hidden_dim_multiplier': 0.5,
                        'augmentation_strength': 'medium'
                    }
                    print(f"Using default parameters for {model_name}")

            # Display parameters being used
            print(f"\n{model_name.upper()} TRAINING PARAMETERS:")
            for key, value in best_params.items():
                if key in ['lr', 'weight_decay', 'dropout', 'hidden_dim_multiplier', 'label_smoothing']:
                    print(f"  {key}: {value:.4f}")
                else:
                    print(f"  {key}: {value}")

            # Step 1: Create data loaders with optimized parameters
            print(f"\n1. CREATING DATA LOADERS WITH OPTIMIZED PARAMETERS FOR {model_name}")
            print("-"*50)

            # Calculate optimal batch size based on model complexity
            model_complexity_map = {
                'efficientnet_': 1.5,  # matches efficientnet_b0, efficientnet_b7 etc.
                'resnet': 1.0,         # matches resnet18, resnet50 etc.
                'vgg': 0.8,            # matches vgg16, vgg19 etc.
                'mobilenet_': 0.6,     # matches mobilenet_v2, mobilenet_v3 etc.
                'densenet': 1.3,       # matches densenet121, densenet201 etc.
                'convnext_': 1.4       # matches convnext_tiny, convnext_base etc.
            }

            model_complexity = model_complexity_map.get(model_name.split('_')[0].lower(), 1.0) #model_name gets from for loop.
            #"EfficientNet_B0".split('_')[0].lower() → "efficientnet"
            base_batch_size = best_params.get('batch_size', Config.BATCH_SIZE)
            optimized_batch_size = resource_manager.optimize_batch_size(base_batch_size, model_complexity)

            # Create with optimized batch size and GPU-optimized settings
            train_loader, val_loader, test_loader, val_data, test_data = DataManager.create_data_loaders(
                X, Y,
                test_size=0.2,
                batch_size=optimized_batch_size,
                augmentation_strength=best_params.get('augmentation_strength', 'medium')
            )

            # Optimize data loaders for GPU
            optimal_workers = min(16, max(8, os.cpu_count() // 2))
            for loader in [train_loader, val_loader, test_loader]:
                if loader:
                    loader.pin_memory = torch.cuda.is_available()
                    loader.num_workers = optimal_workers
                    loader.prefetch_factor = 4 if optimal_workers > 0 else 2

            # Validate data loaders
            print(f"Train: {len(train_loader.dataset)}, Val: {len(val_loader.dataset)}, Test: {len(test_loader.dataset)}")
            print(f"Optimized batch size: {optimized_batch_size} (complexity factor: {model_complexity})")

            if not train_loader or len(train_loader.dataset) == 0:
                print(f"Skipping {model_name}: No training data available")
                continue #To Go Next Model
            if not val_loader or len(val_loader.dataset) == 0:
                print(f"Skipping {model_name}: No validation data available")
                continue

            # Additional validation for val_data and test_data tuples
            if val_data is not None and len(val_data) == 2:
                print(f"Val data tuple: X={len(val_data[0])}, Y={len(val_data[1])}")
                if len(val_data[0]) != len(val_data[1]):
                    print(f"WARNING: Validation data inconsistency: {len(val_data[0])} samples vs {len(val_data[1])} labels")

            if test_data is not None and len(test_data) == 2:
                print(f"Test data tuple: X={len(test_data[0])}, Y={len(test_data[1])}")
                if len(test_data[0]) != len(test_data[1]):
                    print(f"WARNING: Test data inconsistency: {len(test_data[0])} samples vs {len(test_data[1])} labels")

            # Step 2: Train main model
            print(f"\n2. MAIN MODEL TRAINING FOR {model_name}")
            print("-"*50)

            model = ModelFactory.create_model(
                model_name,
                num_classes=Config.NUM_CLASSES,
                dropout_rate=best_params.get('dropout', 0.5),
                hidden_dim_multiplier=best_params.get('hidden_dim_multiplier', 0.5)
            ).to(Config.DEVICE, memory_format=torch.channels_last)  # GPU optimization

            # Model info
            total_model_params = sum(p.numel() for p in model.parameters())
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            print(f"Model created: {total_model_params:,} total params, {trainable_params:,} trainable")

            # Additional GPU optimizations
            torch.backends.cudnn.benchmark = True
            torch.backends.cudnn.deterministic = False
            # Tells cuDNN (NVIDIA's deep learning library) to find the fastest algorithm for your hardware and input size.
            # Benchmarking finds the fastest algorithm.
            # Non-determinism allows cuDNN to use even faster (but slightly variable) methods.
            # Together → maximum training speed, but with non-reproducible results (F1 score may vary slightly between runs).
            # For speed → benchmark=True, deterministic=False (your case).
            # For reproducibility → benchmark=False, deterministic=True.

            trainer = EnhancedModelTrainer(model, model_name, best_params)
            # history, result = trainer.train_main_model(train_loader, val_loader, test_loader)
            trainer.train_main_model(train_loader, val_loader, test_loader)

            # if history is None or result is None: #No Need cause Data is saved to Disk Directly
            #     print(f"Training failed for {model_name}, skipping...")
            #     continue

            print(f"Main model training completed for {model_name}")

            # Step 3: K-fold cross-validation
            print(f"\n3. K-FOLD CROSS-VALIDATION FOR {model_name}")
            print("-"*50)

            # Ensure we have sufficient data for k-fold
            total_samples = len(train_loader.dataset)
            min_samples_per_fold = 500
            max_folds = total_samples // min_samples_per_fold
            n_folds = min(3, max_folds) if max_folds > 1 else 0

            if n_folds > 1:
                print(f"Performing {n_folds}-fold cross-validation...")
                # fold_results =
                trainer.train_kfold(train_loader, val_loader, test_loader,n_folds=n_folds)
                print(f"K-fold validation completed for {model_name}")
            else:
                print(f"Skipping k-fold validation for {model_name}: insufficient data (need >{min_samples_per_fold*2} samples)")
                fold_results = []

            # Step 4: Save model in both .pt and .keras formats
            print(f"\n4. SAVING MODEL FILES FOR {model_name}")
            print("-"*50)

            # Save model for ensemble
            model_state_dict = model.state_dict()
            torch.save(model_state_dict, f"{Config.OUTPUT_DIR}/models/{model_name}_for_ensemble.pt")
            print(f"Model saved for ensemble: {model_name}")

            # Save model as .pt file (PyTorch format)
            torch.save({
                'model_state_dict': model.state_dict(),
                'model_name': model_name,
                'num_classes': Config.NUM_CLASSES,
                'hyperparameters': best_params,
                'architecture_info': {
                    'dropout_rate': best_params.get('dropout', 0.5),
                    'hidden_dim_multiplier': best_params.get('hidden_dim_multiplier', 0.5)
                }
            }, f"{Config.OUTPUT_DIR}/best_model/{model_name}_best.pt")
            print(f"Model saved as .pt file: {model_name}_best.pt")

            # Save model as .keras file (if TensorFlow/Keras conversion is available)
            try:
                # This would require conversion logic if needed
                # For now, we'll save a placeholder or skip
                print(f"Keras format saving for {model_name} - conversion logic needed")
            except Exception as e:
                print(f"Could not save {model_name} in Keras format: {e}")

            # Step 5: Memory cleanup for this model
            print(f"\n5. MEMORY CLEANUP FOR {model_name}")
            print("-"*50)

            # Clear all variables specific to this model
            trainer.cleanup_trainer()
            # del trainer, history, result, model_state_dict
            del train_loader, val_loader, test_loader
            if 'model' in locals():
                del model
            if 'model_state_dict' in locals():
                del model_state_dict

            # Force garbage collection and GPU cleanup
            resource_manager.aggressive_cleanup()

            # Post-model memory state
            post_stats = resource_manager.get_memory_stats()
            memory_freed = pre_stats['gpu_allocated_gb'] - post_stats['gpu_allocated_gb']

            print(f"Memory cleanup completed for {model_name}:")
            print(f"  GPU memory freed: {memory_freed:.2f}GB")
            print(f"  Current GPU usage: {post_stats['gpu_percent']:.1f}%")
            print(f"  Current CPU usage: {post_stats['cpu_percent']:.1f}%")

            print(f"✓ {model_name} TRAINING COMPLETED!")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"Error processing {model_name}: {e}")
            import traceback
            traceback.print_exc()

            # Emergency cleanup
            try:
                if 'trainer' in locals():
                    trainer.cleanup_trainer()
                    del trainer
                if 'model' in locals():
                    del model
                resource_manager.aggressive_cleanup()
            except:
                pass
            continue

    # Final cleanup
    print("\nFinal cleanup and summary...")
    try:
        # Clean up all remaining variables
        if 'X' in locals():
            del X
        if 'Y' in locals():
            del Y
        if 'all_best_params' in locals():
            del all_best_params

        resource_manager.aggressive_cleanup()

        # Final memory stats
        final_stats = resource_manager.get_memory_stats()
        print(f"Final GPU memory: {final_stats['gpu_allocated_gb']:.2f}GB allocated ({final_stats['gpu_percent']:.1f}%)")
        print(f"Final CPU usage: {final_stats['cpu_percent']:.1f}%")

    except Exception as e:
        print(f"Error in final cleanup: {e}")

    print("\n" + "="*70)
    print("MODEL TRAINING COMPLETED!")
    print("="*70)
    print("\nGenerated Files:")
    print(f"- Model checkpoints: {Config.OUTPUT_DIR}/models/")
    print(f"- Training results: {Config.OUTPUT_DIR}/model_results/")
    print(f"- K-fold results: {Config.OUTPUT_DIR}/kfold_results/")
    print(f"- Best models: {Config.OUTPUT_DIR}/best_model/")


# Environment setup and multiprocessing configuration
import torch.multiprocessing as mp
if __name__ == "__main__":
    main()

#Visualization

In [ ]:
"""
15.Fish Species Classification Pipeline - Visualization Module
========================================================

This module handles all visualization generation using saved models and results.
It loads saved .pt model files and generates comprehensive visualizations.
"""

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import gc
import time
import psutil
import warnings
warnings.filterwarnings('ignore')

# orig_numpy = torch.Tensor.numpy
# def patched_numpy(self, *args, **kwargs):
#     if self.is_cuda:
#         raise RuntimeError(f"Tried to call .numpy() on CUDA tensor at line ???. Move it to CPU first!")
#     return orig_numpy(self, *args, **kwargs)

# torch.Tensor.numpy = patched_numpy

class ResourceManager:
    """Smart resource management for optimal GPU/CPU utilization"""

    def __init__(self):
        self.gpu_memory_gb = 20
        self.cpu_memory_gb = 50
        self.max_gpu_usage = 0.85  # 85% of 20GB = 17GB
        self.max_cpu_usage = 0.90  # 80% of 50GB = 40GB

    def get_memory_stats(self):
        """Get current memory usage statistics"""
        stats = {'cpu_percent': psutil.virtual_memory().percent}

        if torch.cuda.is_available():
            stats['gpu_allocated_gb'] = torch.cuda.memory_allocated() / (1024**3)
            stats['gpu_reserved_gb'] = torch.cuda.memory_reserved() / (1024**3)
            stats['gpu_percent'] = (stats['gpu_reserved_gb'] / self.gpu_memory_gb) * 100
        else:
            stats.update({'gpu_allocated_gb': 0, 'gpu_reserved_gb': 0, 'gpu_percent': 0})

        return stats

    def should_cleanup_aggressive(self):
        """Check if aggressive cleanup is needed"""
        stats = self.get_memory_stats()
        return (stats['gpu_percent'] > 90 or stats['cpu_percent'] > 90)

    def aggressive_cleanup(self):
        """Perform comprehensive memory cleanup"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        gc.collect()
        time.sleep(0.1)  # Brief pause for system cleanup

    def optimize_batch_size(self, base_size, model_complexity=1.0):
        """Calculate optimal batch size based on current memory state"""
        stats = self.get_memory_stats()

        # Reduce batch size if memory usage is high
        memory_factor = max(0.4, 1.0 - (stats['gpu_percent'] / 100))
        optimal_size = int(base_size * memory_factor / model_complexity)

        return max(32, min(256, optimal_size))  # Keep within reasonable bounds


def safe_gpu_convert(data):
    """Safely convert GPU tensors to CPU numpy arrays or lists"""
    if torch.is_tensor(data):
        if data.is_cuda:
            data = data.detach().cpu()
        if data.dim() == 0:
            return data.item()
        elif data.dtype == torch.bool or data.dtype == torch.uint8:
            return data.numpy()
        else:
            return data.numpy() if data.dim() > 0 else data.item()
    elif isinstance(data, list):
        return [safe_gpu_convert(item) for item in data]
    elif isinstance(data, dict):
        return {k: safe_gpu_convert(v) for k, v in data.items()}
    elif isinstance(data, np.ndarray):
        return data  # Already numpy
    else:
        return data


def load_model_safely(model_name, model_path):
    """Safely load model with multiple fallback strategies"""
    try:
        checkpoint = torch.load(model_path, map_location=Config.DEVICE, weights_only=False)

        if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
            # New comprehensive format - use saved architecture info
            model = ModelFactory.create_model(
                model_name,
                num_classes=checkpoint.get('num_classes', Config.NUM_CLASSES),
                dropout_rate=checkpoint.get('dropout_rate', 0.5),
                hidden_dim_multiplier=checkpoint.get('hidden_dim_multiplier', 0.5)
            )
            model.to(Config.DEVICE)
            model.load_state_dict(checkpoint['state_dict'])
            return model, f"new format v{checkpoint.get('save_format_version', '1.0')}"

        elif isinstance(checkpoint, dict):
            # Legacy format - try with hyperparameters
            hyperparams = checkpoint.get('hyperparameters', {})
            model = ModelFactory.create_model(
                model_name,
                num_classes=Config.NUM_CLASSES,
                dropout_rate=hyperparams.get('dropout', 0.5),
                hidden_dim_multiplier=hyperparams.get('hidden_dim_multiplier', 0.5)
            )
            model.to(Config.DEVICE)
            model.load_state_dict(checkpoint)
            return model, "legacy nested format"

        else:
            # Try direct loading with parameter combinations
            param_combinations = [
                (0.5, 0.5), (0.3, 0.5), (0.7, 0.5),
                (0.5, 0.3), (0.5, 0.7), (0.5, 1.0), (0.5, 1.5)
            ]

            for dropout, hidden_mult in param_combinations:
                try:
                    model = ModelFactory.create_model(
                        model_name,
                        num_classes=Config.NUM_CLASSES,
                        dropout_rate=dropout,
                        hidden_dim_multiplier=hidden_mult
                    )
                    model.to(Config.DEVICE)
                    model.load_state_dict(checkpoint)
                    return model, f"direct format (dropout={dropout}, hidden={hidden_mult})"
                except RuntimeError:
                    continue

            return None, "failed all combinations"

    except Exception as e:
        return None, f"loading error: {str(e)}"


def main():
    print("\nStarting Fish Species Visualization Generation...")
    print("="*70)

    # Environment setup for maximum performance
    setup_environment()


    # Set optimal thread count for CPU utilization
    print(f"CPU : {os.cpu_count()}")
    print(f"USER: {os.getenv('USER')}")
    print(f"HOME: {os.getenv('HOME')}")
    torch.set_num_threads(min(16, os.cpu_count()))
    os.environ['OMP_NUM_THREADS'] = str(min(16, os.cpu_count()))

    # OpenMP → Open Multi-Processing.
    # OpenMP is a widely used API for parallel programming on CPUs.
    # Many scientific libraries (like NumPy, PyTorch, TensorFlow,
    # -OpenCV, Scikit-learn) rely on OpenMP to run operations in parallel across CPU cores.
    # In main() after line 720:
    torch.backends.cudnn.benchmark = True
    # Auto-benchmarks and selects fastest cuDNN algorithms for your specific hardware and input sizes
    # Takes ~5-10 seconds at start but can provide 10-20% speedup throughout training
    torch.backends.cudnn.deterministic = False
    # Allows cuDNN to use fastest available algorithms even if they produce slightly different results each run
    # Trades reproducibility for speed - essential for maximum performance in production training
    # torch.backends.cuda.matmul.allow_tf32 = True
    # Uses TensorFloat-32 (19-bit precision) instead of full Float-32 for matrix multiplications
    # Provides ~1.6x speedup on Ampere GPUs with negligible accuracy loss for deep learning
    # torch.backends.cudnn.allow_tf32 = True
    # Enables TF32 precision for cuDNN operations (convolutions, pooling, batch norm)
    # Accelerates CNN layers by 1.3-1.6x on RTX 30/40 series and A100+ GPUs
    torch.backends.cuda.enable_flash_sdp(True)
    # Activates Flash Attention algorithm for memory-efficient attention computations
    # Reduces GPU memory usage by 2-4x and increases attention speed, crucial for transformer layers

    # GPU optimizations
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.backends.cuda.matmul.allow_tf32 = True

        gpu_props = torch.cuda.get_device_properties(0)
        print(f"GPU: {gpu_props.name}, Memory: {gpu_props.total_memory / 1024**3:.1f}GB")

    # CPU info
    print(f"CPU Cores: {os.cpu_count()}, Using threads: {torch.get_num_threads()}")

    # Ensure visualization output directory exists
    os.makedirs(f"{Config.OUTPUT_DIR}/visualizations", exist_ok=True)

    # Initialize resource manager
    resource_manager = ResourceManager()

    # Load and balance data for visualization (if needed for test data)
    print("\nLoading data for visualization...")
    X, Y = DataManager.load_and_balance_data()
    print(f"Total samples: {len(X):,}, Labels: {len(Y):,}")

    # Initialize components
    visualizer = EnhancedVisualizations()
    single_models = {}
    single_results = {}

    # Load trained models and results
    print(f"\n{'='*70}")
    print("LOADING TRAINED MODELS AND RESULTS")
    print(f"{'='*70}")

    # Load models with enhanced error handling
    for model_name in Config.MODELS:
        model_path = os.path.join(Config.OUTPUT_DIR, "best_model", f"{model_name}_best.pt")

        if not os.path.exists(model_path):
            print(f"⚠️  Model file not found: {model_name}")
            continue

        model, load_info = load_model_safely(model_name, model_path)

        if model is not None:
            model.eval()
            single_models[model_name] = model
            print(f"✅ Loaded {model_name} ({load_info})")

            # Load evaluation results for visualization compatibility
            result_path = f"{Config.OUTPUT_DIR}/model_results/{model_name}_main_results.pt"
            if os.path.exists(result_path):
                try:
                    saved_results = torch.load(result_path, map_location='cpu', weights_only=False)
                    result_data = saved_results.get('result', {})

                    single_results[model_name] = {
                        'model_name': model_name,
                        'accuracy': float(result_data.get('accuracy', 0.0)),
                        'f1_macro': float(result_data.get('f1', 0.0)),
                        'f1_weighted': float(result_data.get('f1', 0.0)),  # Add f1_weighted key
                        'predictions': result_data.get('predictions', []),
                        'true_labels': result_data.get('true_labels', []),
                        'probabilities': result_data.get('probabilities', np.zeros((0, Config.NUM_CLASSES))),
                        'conf_matrix': result_data.get('conf_matrix', np.zeros((Config.NUM_CLASSES, Config.NUM_CLASSES))),
                        'misclassified': result_data.get('misclassified', [])
                    }

                    # Convert tensors to numpy for visualization compatibility
                    for key in ['predictions', 'true_labels', 'probabilities', 'conf_matrix']:
                        if torch.is_tensor(single_results[model_name][key]):
                            single_results[model_name][key] = single_results[model_name][key].numpy()

                    print(f"📊 Results loaded: Acc={single_results[model_name]['accuracy']:.4f}")

                except Exception as e:
                    print(f"⚠️  Could not load results for {model_name}: {e}")
                    single_results[model_name] = {'model_name': model_name, 'accuracy': 0.0, 'f1_macro': 0.0, 'f1_weighted': 0.0}
            else:
                print(f"⚠️  No results file for {model_name}")
                single_results[model_name] = {'model_name': model_name, 'accuracy': 0.0, 'f1_macro': 0.0, 'f1_weighted': 0.0}
        else:
            print(f"❌ Failed to load {model_name}: {load_info}")

    print(f"\n🎯 Loaded: {len(single_models)} models, {len(single_results)} result sets")

    # Create test data loader for visualizations that need it
    if len(single_models) > 0:
        print("\nCreating test data loader for visualizations...")
        _, _, test_loader, val_data, test_data = DataManager.create_data_loaders(X, Y, test_size=0.2)
        print(f"Test loader created with {len(test_loader.dataset)} samples")

    # Generate visualizations for each model
    for model_name in single_models.keys():
        print(f"\n{'='*70}")
        print(f"GENERATING VISUALIZATIONS FOR: {model_name}")
        print(f"{'='*70}")

        # Pre-visualization memory state
        pre_stats = resource_manager.get_memory_stats()
        print(f"Pre-visualization memory - GPU: {pre_stats['gpu_allocated_gb']:.2f}GB ({pre_stats['gpu_percent']:.1f}%), "
              f"CPU: {pre_stats['cpu_percent']:.1f}%")

        try:
            # Load saved results for plotting (ensuring consistency)
            main_result_path = f"{Config.OUTPUT_DIR}/model_results/{model_name}_main_results.pt"

            if os.path.exists(main_result_path):
                saved_data = torch.load(main_result_path, map_location=Config.DEVICE, weights_only=False)
                history_for_viz = saved_data['history']
                result_for_viz = saved_data['result']
                print(f"Using saved results for {model_name} visualization")
            else:
                result_for_viz = {}
                history_for_viz = {}
                def add_strikethrough(text):
                    return ''.join(c + '\u0336' for c in text)
                print(f"{add_strikethrough('Using current')} No results for {model_name} visualization")

            print(f"Result keys: {list(result_for_viz.keys()) if result_for_viz else 'None'}")

            # Convert all data to CPU/numpy for visualization (GPU-safe conversion)
            history_viz = safe_gpu_convert(history_for_viz)
            result_viz = safe_gpu_convert(result_for_viz)

            # Validate result data
            true_labels = result_viz.get('true_labels', np.array([]))
            predictions = result_viz.get('predictions', np.array([]))
            probabilities = result_viz.get('probabilities', np.array([]))

            # ADD THE CONVERSION LINES HERE:
            if torch.is_tensor(true_labels):
                true_labels = true_labels.detach().cpu().numpy()
            if torch.is_tensor(predictions):
                predictions = predictions.detach().cpu().numpy()
            if torch.is_tensor(probabilities):
                probabilities = probabilities.detach().cpu().numpy()
            # ADD THESE ADDITIONAL CONVERSIONS:
            # Convert history data to safe formats
            for key in ['train_loss', 'train_acc', 'val_loss', 'val_acc', 'val_f1']:
                if key in history_viz and torch.is_tensor(history_viz[key]):
                    history_viz[key] = history_viz[key].detach().cpu().numpy().tolist()

            print(f"Data validation for {model_name}:")
            print(f"  true_labels: {type(true_labels)}, shape: {getattr(true_labels, 'shape', len(true_labels) if hasattr(true_labels, '__len__') else 'scalar')}")
            print(f"  predictions: {type(predictions)}, shape: {getattr(predictions, 'shape', len(predictions) if hasattr(predictions, '__len__') else 'scalar')}")
            print(f"  probabilities: {type(probabilities)}, shape: {getattr(probabilities, 'shape', 'unknown')}")

            # Check if we have valid data for plotting
            if (true_labels.size > 0 and predictions.size > 0 and true_labels.shape[0] == predictions.shape[0]):
                plots_generated = 0

                # Training history plot
                train_loss = history_viz.get('train_loss')
                if train_loss is not None and hasattr(train_loss, '__len__') and len(train_loss) > 0:
                    try:
                        # Add this conversion:
                        if history_viz.get('train_loss') is not None:
                            if isinstance(history_viz['train_loss'], np.ndarray):
                                history_viz['train_loss'] = history_viz['train_loss'].tolist()
                        if history_viz.get('train_acc') is not None:
                            if isinstance(history_viz['train_acc'], np.ndarray):
                                history_viz['train_acc'] = history_viz['train_acc'].tolist()
                        if history_viz.get('val_loss') is not None:
                            if isinstance(history_viz['val_loss'], np.ndarray):
                                history_viz['val_loss'] = history_viz['val_loss'].tolist()
                        if history_viz.get('val_acc') is not None:
                            if isinstance(history_viz['val_acc'], np.ndarray):
                                history_viz['val_acc'] = history_viz['val_acc'].tolist()

                        visualizer.plot_single_model_history(history_viz, model_name)
                        plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_history.png", dpi=150, bbox_inches='tight')
                        plt.close('all')
                        print(f"Training history plot generated for {model_name}")
                        plots_generated += 1
                    except Exception as e:
                        print(f"Error plotting training history for {model_name}: {e}")

                try:
                    # ROC Curves
                    visualizer.plot_roc_curves(result_viz, model_name)
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_roc.png", dpi=150, bbox_inches='tight')
                    plt.close('all')
                    print(f"ROC curves plot generated for {model_name}")
                    plots_generated += 1

                    # Confusion Matrix
                    visualizer.plot_confusion_matrix(result_viz, model_name)
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_confusion.png", dpi=150, bbox_inches='tight')
                    plt.close('all')
                    print(f"Confusion matrix plot generated for {model_name}")
                    plots_generated += 1

                    # Misclassified Images
                    misclassified = result_viz.get('misclassified', [])
                    if misclassified is not None and len(misclassified) > 0:
                        # Convert any GPU tensors in misclassified images to CPU
                        for item in result_viz['misclassified']:
                            if isinstance(item, dict) and 'image' in item:
                                if torch.is_tensor(item['image']):
                                    item['image'] = item['image'].detach().cpu()
                                elif isinstance(item['image'], np.ndarray):
                                    # Optionally transpose if channel-first
                                    if item['image'].shape[0] in [1,3]:
                                        item['image'] = np.transpose(item['image'], (1, 2, 0))

                        visualizer.plot_misclassified_images(result_viz['misclassified'], model_name)
                        plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_misclassified.png", dpi=150, bbox_inches='tight')
                        plt.close('all')
                        print(f"Misclassified images plot generated for {model_name}")
                        plots_generated += 1
                    else:
                        print(f"No misclassified images to plot for {model_name}")

                    # XAI Visualization
                    if model_name in single_models and test_loader is not None and len(test_loader.dataset) > 0:
                        try:
                            model = single_models[model_name]
                            visualizer.plot_single_model_xai(model, model_name, test_loader)
                            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_xai.png", dpi=150, bbox_inches='tight')
                            plt.close('all')
                            print(f"XAI visualization generated for {model_name}")
                            plots_generated += 1
                        except Exception as xai_error:
                            print(f"XAI visualization error for {model_name}: {xai_error}")
                    else:
                        print(f"Skipping XAI visualization for {model_name}: model or test_loader not available")

                except Exception as plot_error:
                    print(f"Visualization error for {model_name}: {plot_error}")
                    import traceback
                    traceback.print_exc()

                print(f"Generated {plots_generated} main visualization plots for {model_name}")

            else:
                print(f"Skipping main visualizations for {model_name}: data validation failed")
                print(f"  true_labels length: {true_labels.shape[0] if hasattr(true_labels, 'shape') else 'scalar'}")
                print(f"  predictions length: {predictions.shape[0] if hasattr(predictions, 'shape') else 'scalar'}")

            # K-fold results plotting (load from saved files)
            print(f"Loading and plotting k-fold results for {model_name}...")
            fold_results_loaded = []

            # Check for k-fold results
            for fold in range(1, 4):  # Assuming max 3 folds
                fold_result_path = f"{Config.OUTPUT_DIR}/kfold_results/{model_name}_fold_{fold}_results.pt"
                if os.path.exists(fold_result_path):
                    try:
                        fold_data = torch.load(fold_result_path, map_location=Config.DEVICE, weights_only=False)

                        # Convert GPU tensors to CPU/numpy
                        fold_converted = {
                            'history': safe_gpu_convert(fold_data['history']),
                            'result': safe_gpu_convert(fold_data['result'])
                        }
                        fold_results_loaded.append(fold_converted)

                    except Exception as e:
                        print(f"Error loading fold {fold} results: {e}")

            if fold_results_loaded is not None and len(fold_results_loaded) > 0:
                try:
                    visualizer.plot_kfold_results(fold_results_loaded, model_name, test_loader)
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_kfold.png", dpi=150, bbox_inches='tight')
                    plt.close('all')
                    print(f"K-fold results plot generated for {model_name}")
                except Exception as e:
                    print(f"Error plotting k-fold results for {model_name}: {e}")
                    import traceback
                    traceback.print_exc()
            else:
                print(f"No k-fold results to plot for {model_name}")

            print(f"All visualizations completed for {model_name}")

            # Memory cleanup for this model's visualizations
            if 'history_viz' in locals():
                del history_viz
            if 'result_viz' in locals():
                del result_viz
            if 'fold_results_loaded' in locals():
                del fold_results_loaded
            if 'history_for_viz' in locals():
                del history_for_viz
            if 'result_for_viz' in locals():
                del result_for_viz

            # Force garbage collection and GPU cleanup
            plt.close('all')
            resource_manager.aggressive_cleanup()

            # Post-visualization memory state
            post_stats = resource_manager.get_memory_stats()
            memory_freed = pre_stats['gpu_allocated_gb'] - post_stats['gpu_allocated_gb']

            print(f"Memory cleanup completed for {model_name}:")
            print(f"  GPU memory freed: {memory_freed:.2f}GB")
            print(f"  Current GPU usage: {post_stats['gpu_percent']:.1f}%")
            print(f"  Current CPU usage: {post_stats['cpu_percent']:.1f}%")

            print(f"✓ {model_name} VISUALIZATIONS COMPLETED!")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"Error in visualization process for {model_name}: {e}")
            import traceback
            traceback.print_exc()

    # Step: Ensemble processing and visualizations (only if we have trained models)
    if single_models and 'val_data' in locals():
        print("\n" + "="*70)
        print("GENERATING ENSEMBLE VISUALIZATIONS")
        print("="*70)

        try:
            if (val_data is not None and len(val_data) == 2 and len(val_data[0]) > 0 and len(val_data[1]) > 0 and len(val_data[0]) == len(val_data[1])):

                ensemble_manager = EnsembleManager(single_models, val_data)
                ensemble_results, best_ensemble = ensemble_manager.test_ensemble_combinations()

                # Generate visualizations for ensemble methods
                print("\nGenerating visualizations for ensemble methods...")
                for ensemble_name, ensemble_result in ensemble_results.items():
                    try:
                        # Convert GPU tensors to CPU for visualization
                        ensemble_result_viz = safe_gpu_convert(ensemble_result)

                        true_labels_ens = ensemble_result_viz.get('true_labels', [])
                        predictions_ens = ensemble_result_viz.get('predictions', [])

                        if (len(true_labels_ens) > 0 and
                            len(predictions_ens) > 0 and
                            len(true_labels_ens) == len(predictions_ens)):

                            visualizer.plot_roc_curves(ensemble_result_viz, ensemble_name)
                            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{ensemble_name}_roc.png", dpi=150, bbox_inches='tight')
                            plt.close('all')

                            visualizer.plot_confusion_matrix(ensemble_result_viz, ensemble_name)
                            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{ensemble_name}_confusion.png", dpi=150, bbox_inches='tight')
                            plt.close('all')

                            # F1 per class visualization
                            from sklearn.metrics import f1_score
                            f1_per_class = f1_score(ensemble_result_viz['true_labels'],
                                                   ensemble_result_viz['predictions'], average=None)
                            fig, ax = plt.subplots(figsize=(10, 6))
                            ax.bar(range(Config.NUM_CLASSES), f1_per_class, color='lightgreen', alpha=0.8)
                            ax.set_xticks(range(Config.NUM_CLASSES))
                            ax.set_xticklabels(Config.CLASS_LABELS, rotation=45, ha='right')
                            ax.set_title(f'F1 Scores per Class - {ensemble_name}',
                                       fontsize=14, fontweight='bold', pad=15)
                            ax.set_xlabel('Class', fontsize=12, fontweight='bold')
                            ax.set_ylabel('F1 Score', fontsize=12, fontweight='bold')
                            ax.grid(True, alpha=0.3)
                            for i, v in enumerate(f1_per_class):
                                ax.text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom', fontsize=10)
                            save_path = f"{Config.OUTPUT_DIR}/visualizations/{ensemble_name}_f1_per_class.png"
                            plt.savefig(save_path, dpi=300, bbox_inches='tight',
                                      facecolor='white', edgecolor='none')
                            plt.close()
                            print(f"F1 scores per class saved: {save_path}")
                        else:
                            print(f"Skipping visualizations for {ensemble_name}: invalid data")
                    except Exception as e:
                        print(f"Error generating ensemble visualizations for {ensemble_name}: {e}")

                    # Clear memory
                    if 'ensemble_result_viz' in locals():
                        del ensemble_result_viz
                    resource_manager.aggressive_cleanup()

                # Comment out the problematic ensemble test evaluation
                print("Skipping ensemble test evaluation (method not implemented)")
                # if (best_ensemble and test_data is not None and len(test_data) == 2 and
                #     len(test_data[0]) == len(test_data[1]) and len(test_data[0]) > 0):
                #     print("Evaluating best ensemble on test set...")
                #     # Ensemble test evaluation code commented out

            else:
                print("Skipping ensemble analysis: no valid validation data")

        except Exception as e:
            print(f"Error in ensemble processing: {e}")
            import traceback
            traceback.print_exc()

    # Step: Generate comprehensive visualizations (only if we have results)
    if single_results:
        print("\n" + "="*70)
        print("GENERATING COMPREHENSIVE ANALYSIS VISUALIZATIONS")
        print("="*70)

        try:
            # Convert all results to CPU/numpy for final plotting
            single_results_viz = safe_gpu_convert(single_results)
            ensemble_results_viz = safe_gpu_convert(ensemble_results) if 'ensemble_results' in locals() else {}

            # Plot overall model comparison
            if single_results_viz and len(single_results_viz) > 0:
                visualizer.plot_model_comparison(single_results_viz,ensemble_results_viz)
                plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/model_comparison.png", dpi=300, bbox_inches='tight')
                plt.close('all')

            # Generate comprehensive report
            visualizer.generate_comprehensive_report(
                single_results_viz,
                ensemble_results_viz,
                best_ensemble if 'best_ensemble' in locals() else None
            )
            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/comprehensive_report.png", dpi=300, bbox_inches='tight')
            plt.close('all')

            # Additional visualizations
            if 'test_loader' in locals() and test_loader is not None and len(test_loader.dataset) > 0:
                try:
                    visualizer.plot_comparative_xai(
                        single_models,
                        ensemble_results_viz,
                        test_loader,
                        max_images=2
                    )
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/comparative_xai.png", dpi=300, bbox_inches='tight')
                    plt.close('all')

                    visualizer.plot_lrp_grid(single_models, test_loader)
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/lrp_grid.png", dpi=300, bbox_inches='tight')
                    plt.close('all')
                except Exception as e:
                    print(f"Error in additional visualizations: {e}")

            print("Comprehensive analysis completed successfully")

        except Exception as e:
            print(f"Error in comprehensive analysis: {e}")
            import traceback
            traceback.print_exc()

    # Final cleanup
    print("\nFinal cleanup and summary...")
    try:
        # Clean up all remaining variables
        if 'single_models' in locals():
            del single_models
        if 'test_loader' in locals():
            del test_loader
        if 'single_results' in locals():
            del single_results
        if 'single_results_viz' in locals():
            del single_results_viz
        if 'ensemble_results' in locals():
            del ensemble_results
        if 'ensemble_results_viz' in locals():
            del ensemble_results_viz
        if 'best_ensemble' in locals():
            del best_ensemble
        if 'visualizer' in locals():
            del visualizer
        if 'X' in locals():
            del X
        if 'Y' in locals():
            del Y

        resource_manager.aggressive_cleanup()

        # Final memory stats
        final_stats = resource_manager.get_memory_stats()
        print(f"Final GPU memory: {final_stats['gpu_allocated_gb']:.2f}GB allocated ({final_stats['gpu_percent']:.1f}%)")
        print(f"Final CPU usage: {final_stats['cpu_percent']:.1f}%")

    except Exception as e:
        print(f"Error in final cleanup: {e}")

    print("\n" + "="*70)
    print("VISUALIZATION GENERATION COMPLETED!")
    print("="*70)
    print("\nGenerated Files:")
    print(f"- All visualizations: {Config.OUTPUT_DIR}/visualizations/")


# Environment setup and multiprocessing configuration
import torch.multiprocessing as mp
if __name__ == "__main__":
    main()

#End